<div style="border: 2px solid #8A9AD0; margin: 1em 0.2em; padding: 0.5em;">

# Inferring Trajectories using Monocle3 (R)

by [Julia Jakiela](https://training.galaxyproject.org/hall-of-fame/wee-snufkin/), [Helena Rasche](https://training.galaxyproject.org/hall-of-fame/hexylena/), [Pavankumar Videm](https://training.galaxyproject.org/hall-of-fame/pavanvidem/), [Wendi Bacon](https://training.galaxyproject.org/hall-of-fame/nomadscientist/), [EOSC-Life](https://training.galaxyproject.org/hall-of-fame/eosc-life/)

CC-BY licensed content from the [Galaxy Training Network](https://training.galaxyproject.org/)

**Objectives**

- How do I perform a trajectory analysis using Monocle3 in R?
- What should I do when Galaxy's Monocle tools are not enough?
- How do I assign cell types to the clusters?
- How do I infer lineage relationships between clusters, without a time series?
- How do I perform batch correction and differential expression analysis in Monocle?

**Objectives**

- Identify which operations are necessary to transform an AnnData object into the files needed for Monocle
- Describe the Monocle3 functions in R
- Recognise steps that can be performed in R, but not with current Galaxy tools
- Repeat the Monocle3 workflow and choose appropriate parameter values
- Compare the outputs from Scanpy, Monocle in Galaxy and Monocle in R
- Describe differential expression analysis methods

**Time Estimation: 3H**
</div>


<h2 id="setting-up-the-environment-and-file-upload">Setting up the environment and file upload</h2>
<p>Once the installation is done, we should load the needed packages into our notebook. Navigate back to your <code style="color: inherit">notebook</code>. If you are using our prepopulated notebook, you can follow the tutorial from there. Otherwise, input the following into your fresh notebook.</p>


In [ ]:
install.packages("Rcpp")    # needed for reduce_dimension to avoid AnnoyAngular error; library(Rcpp) might work as well depending on the version

In [ ]:
library(monocle3)
library(biomaRt)
library(magrittr)     # needed for %>%
library(viridisLite)

In [ ]:
install.packages("anndata")

<p>Alright, the libraries are here - so now let’s get our AnnData file.</p>
<h2 id="upload-view-and-modify-the-files">Upload, view and modify the files</h2>
<p>We can download the file into Jupyter notebook also using a Zendo link. The first argument is the download link and the second one is the name under which the downloaded file is saved:</p>


In [ ]:
download.file('https://zenodo.org/record/7455590/files/AnnData_filtered.h5ad', 'AnnData.h5ad')

<p>Then we have to read in this h5ad file:</p>


In [ ]:
ann <- anndata::read_h5ad('AnnData.h5ad')

<p>Now we store all the information we need in AnnData object. However, Monocle uses <em>cell_data_set class</em> to hold expression data, which requires three input files: <code style="color: inherit">expression_matrix</code>, <code style="color: inherit">cell_metadata</code> and <code style="color: inherit">gene_metadata</code>. Therefore, we have to somehow ‘transfer’ that information from our AnnData object to Monocle’s cell_data_set (cds). AnnData stores a data matrix <code style="color: inherit">X</code> together with annotations of observations <code style="color: inherit">obs</code> and variables <code style="color: inherit">var</code>, so we can extract those parameters and use them for further analysis.</p>


In [ ]:
expression_matrix <- ann$X
cell_metadata <- ann$obs
gene_metadata <- ann$var

<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div style="font-weight:900;font-size: 125%">💡 </button></div>
<p>If you already have files containing the expression matrix, genes and cells metadata, you can upload them to JupyLab and generate a cds file from them instead. For example, if you first downloaded the files from Galaxy, your files will have the <code style="color: inherit">.tabular</code> extension. In this case, we will use the <code style="color: inherit">read.delim()</code> function to read them in. In this function, the first argument is the file path - in our case, the files are in the same folder as the notebook, so the file path is the same as the file name. You can always check that by right-clicking on the file and choosing <code style="color: inherit">Copy path</code>. The second argument, <code style="color: inherit">row.names=1</code> takes the column number of the data file from which to take the row names.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"># read in the files
cell_metadata &lt;- read.delim('cells.tabular', row.names=1)
gene_metadata &lt;- read.delim('genes.tabular', row.names=1)
expression_matrix &lt;- read.delim('expression.tabular', row.names=1)
</code></pre></div>  </div>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question"><i class="far fa-question-circle" aria-hidden="true"></i> Question</div>
<p>Why should we set <code style="color: inherit">row.names=1</code>?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution"></div>
<p>This allows us to ensure that the expression value matrix has the same number of columns as the <code style="color: inherit">cell_metadata</code> has rows and the same number of rows as the <code style="color: inherit">gene_metadata</code> has rows. Importantly, the row names of the <code style="color: inherit">cell_metadata</code> object should match the column names of the expression matrix and the row names of the <code style="color: inherit">gene_metadata</code> object should match the row names of the expression matrix.</p>
</details>
</blockquote>
</blockquote>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div style="font-weight:900;font-size: 125%">💡 </button></div>
<p>If you are working in RStudio Cloud, you have to click on ‘Upload’ button in the right bottom window toolbar and choose already downloaded data files to upload. You should now see all three files in this window. You might want to rename the files to make their names shorter.</p>
<figure id="figure-1"><img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAApkAAADkCAIAAABc7qgzAAAACXBIWXMAABJ0
AAASdAHeZh94AAAgAElEQVR42u2dCXgcR532J8/zPbubECBsroWELCw3C4Tl
2o+bACGBjbFR1sAuBFiWj10WlisXjm3ZliVbsWzLt3xfsWRbvnRYkiVbliXL
o1sjaaTRfYzGuu/LkiUfX3X3dE9VX9M9Gskzo/d93see6a6urqmuqV//q6pH
lhuTN2EYhmEYDl5bUAUwDMMwDJbDMAzDMBxILO/uG2hzdTQ7XY6GJhiGYRiG
A9wWJchbnNe7e/txmwPDMAzDQRmXt7o6unv6xqduBrh7Bgbrmlo6urr7BwaD
wg0trdc7g6a0c2pSD0Fx7YKlnD58LtIau7p7YRgOGctZ3uJ0kX8Dn+XEjoam
IOpAg6u0qI3Qvmrkc6Hvg+FQZjn5koPloAJqAyyHYRgsB8vBcrAcLIdhGCwH
y8FylBMsh2EYLAcVUBu4amA5DIPlYDmoAJaD5TAMLyyW54ZbiJYcbCav2w++
yL0JzwtIlmcu5wq3eK8jNKmQ+Qb38RbtsS9ARtbtWcx9+DcyA6GcfisMWA7D
sN9ZLjCb0eIj7QHIcsfeRfKCLssMYJa7u35aPmEgFFjuvnbC9RJs3/sCt2l5
RuCwXKNIfOHJRrAchuFAZ7nAbC0HEsvdPBAIx3esRlnun77YLMvdpxMK6QVd
YPk9jsszlilPx5eBLjnG2GEYDh6W68blzUcWM0G8ynZv9wezZLn77Qt765Qs
F3pkQVwCxUYpMRPrG4vsfWS5+y2PZDfX5SWkCEdvZ1jOF1jkusF8li1/gfl0
1IAB/ZGpQ4zfOviR5dQFkjaq3Q8t3rtHJaWfrppwCkU5+QIobiw0L5a7VO5P
JObGvZUqnLp2ngzZD8hcULAchsFyP7NcAPaLR5qnbmaH04wXDlmTPR9xubuj
VLKcDq3YQxR9sbLjnkOWU0jO3CuSkgK8Am8Zy4Tt1IF8Ak8XbzQfGtuKWwpm
nMP0sIG/WO65asJ2prSysQ2xBszMrRi+avJREHGAXVEYqpxUk6MvhHjPRH0W
puRcbbAXTvyAxschwHIYBst9ZDnD77w1/JtcOig3OxTvCw/oIWu6T2dIIOua
5SwXg3JzAPON5bIQTXVQN0M9GvN8BCawM54PXSda/BZDTLOj2b6vdaADaxV+
q10yObxNzD4YX/vG1h49wK5yQZmC8ck8w0Vc8Ra9sUy6FeBKq+A9nd6XlR9g
OQyD5Rpr3/iAW5vlwmuR/VSMLiVTGXufi7VvsuFKFVbJ+l+9MVK/jyor177R
kGYG/NW6eBWW712u3GsoH4qUDDlUg0iL0dFdf8bl7FVThLlaLNeqsdmtY1fc
+rCzJGqVrHq3xG1cniHlkCldPnYIhP7sYDkMg+XzFZczwGbx74naTeHcdx7M
kuXUGjqL/yM8zXVbdOSnMTxrqIRG81GwnJUnpeo6g/ljuUXrvmR+WU6dlK8Q
1cl7lYcU5CNDpLTkX3FahKsBd8npCzrbJzLAchgGy/0xxq7h5iNLZIyfJ5ab
GWNXdt8GVxvNjuXMyLCJMfZM2WIxL/nIhx/U73W069mv89BeWC6/arrPAsgv
rqEpElPPl4ulXf4Gczm0x9hVrtfyPSQT7vPyb5dlcumZsXq1WxCwHIbB8nuy
9s2dUslsfh79HsTlemvftGAp7ZqfuJzeTj1Zp1gORgpPpaFWq6msC2PzYZKJ
UaDKbDS9wk5en/MalyvQKH529bVvqhfXn78V45l8oStHc+2bG+FsDb/gXunG
v1286AWdRZckK8TlMAzfg2fSmFl2ceE680CauCBu3lmuO1YsDW9yCJENOBtc
9jX7MXZ2np5lhqFn0phl7ar5UB9NDOg16oe9jWCHfOf3mTT6crAz+nKWz105
5QsRGKwqLyg9Wq7yQZhlj7I7RbrCVVd+gOUwDJbj99jxy95ybMzNr53Ma23M
4gf+8HvsMAyD5WB5UFGBwFsRhc/RL4+C5WA5DMNgOVg+l9gz+bgdWA6WwzAM
loPl+JunKCdYDsMwWA4qgOW4amA5DIPlYDmogNoAy2EYBsvBcrAcLAfLYRgG
y8FysBzlBMthGAbLQQXUBq4aWA7DYHlwsByGYRiG4SBmOQzDMAzDxGA5DMMw
DIPl8Ow8Njk1gXrQrpxx1A+MdgLDYHmAe2B45MbNadSDVuWMTEygfmC0ExgG
ywPa17u7p2ZmUA9alTM4Mor6gdFOYBgsD2g3tbVN37qFetCqnL7BQdQPPEft
ZGxyKutq4atrN0Vu2evs6kFNwguI5cOTk1d72o61VR5sKTvnctQN9Y3J00yN
TozSJltQ11qurq27dfu2XndzY2Kwp6K3Lb23NaX/et7IsGtBVU53b59+/YyO
jXdY85uOv9X41sGW88k9DfVjNybRrhbal8hrO1G1o6n1xT3nvpVS9cyJgh2n
01GT8EJhec1Qzxev7H93RsxjaW8+ej6a+N3n34yuyaXAM37t5DNZux6lfe3E
10cnxlDdqrbZ7bfv3NG8cxpsbs55qS7j23Xp36pL/yb5t/7C8131by2cyuns
7tGpn95ax+Vn/jn9g3+X9g+PnX/fI6nvfzT1g+8ui44cGp9A01pQXyL9dkKb
BOIbdx/u6hsgr1MuX3sxy744p/F7mbX/L2Yft4Zu6mZGTn59aztqFQ5Zll/q
av705bjH0zcIIH8kdf3DKevelRz1UFLkS4Wn+m9MjIz2lZ7/yfXaxLHx8ZGx
Ucl15UdLUv9tZLQfNa50UWnZnbt3VXf1d1jrMpfUpHxF6aaCFfVNTTUNzVW1
jeX22pKKamtpZV5h2eX84qwr1sY2V8hUzvWOTq36ceVcuvTFTxGQp3/4PRzI
3/dIylMPp//j+5OffFf6v/1rU2NrV+9AV9+g0s7O3pGJG8FbLR09fT19/cRd
PX0dXT3t17ta2683tTjrGltq6horquvKKqpPnEtbUF8inXbCdGIFZT88kPLj
C7afx53Yfyrl5V1v/Udx20+LnEvzW17ceOhMRvYruxO+vD356xuPn7hwBb0T
HIIsbxkZ/OjFbRzI0zcQkN/lFV2X966kyHeeW/vOsxHRlRllGb901Z4eGhnu
7OmWubb0YMXF3wV+jWSHWywvHmlW2ZVL9iw+4v+7dWtxyV21PmhoyFmd8kzV
2c8Lbrz8s96GeHvSl6QtvQ1v3dXQW4kp81UtZm2uGknluK53qNZPf0tL1qc/
lPGRJ5p2xrYdPZD2ofcQkF9d8tz08NDlb37x3BPvLIhcNTk5qVo/Q6Pjw+P+
Y3neGotlycHm+Wulzs7uu96UcDp14XRkOu1ENju+bE/Cv+c3/crW8bua7t/b
2l6u7/tDXe9v7F2/KHX98Grz949lfyOl6vOnbJ84VvLs8q3dA0OABBxqLN/Z
VPTujBgpKJdY/hAP8necWfPwqRUVTZlDIyNKkAu2Zb8qz7b5yGKLxRKeS21s
P/iiX5HJ9bO01mQHGsuLilX7oLbKPeUnPyW4Pvvn0xNc993XdFLaaDv1uakx
l2o/fvjEOfXTcRXOUscYh+4Zy4uKtfro2h2x6R943LF2BfeB79xxnjh29QfP
3+i4ziVuaU75yJNnn3q4taz0zp07Jlguby3GPnXAsLy7t48E6C1trsaWtg1b
95Tba4tt1deKbTnXirOuXDufdeVc2sUWV6ciQ+5LRys8L9hYrt1OmKUVNyaT
LuX94qKN8HtZ6+Aq18jq66Mr20deaxr4bU33z0vaw3Kbnr1Q+6WzlZ8+Vvzr
TYf6hoa1v0dqnQnffuail4Bhv7H8u9ZjBOSfuRy3yHrshWvHhL7jmLPi+bzD
z+ceevvp1Q+eWrXV+laDPVnpJseFju6O8kuvaHwl6L5jLljuhd+ByfKK1O8X
H/sIsf38CzcnOj39dd3R0hNPC7t6m86p9ukH4s+EPMtz/+WZtPc/evmrnx1v
bhJwfuf2bfL/7enpyhWvnfm7t59+/MGiHVtu3rxpkuV0axEg5639GKjD5iNL
TLVDLyy/3qV63VucLq/x+sW8AlWWSxeFu9b+K2pAsZzE5fUtbf8Zvev15oG1
nWMb+m9sHJyM7r1BoP5yY/+vKzp/fK31uxfrv5ps/9qu8+cyLw9rrboQOy5Z
S+arDiyHA5vlH8jaQiLyDQ35qh3Eg6dXv+3Uqoji+OGeMqXbbDudbVUaLF+y
mOku5SwXvh7SN4TrEyWo8N8oT2LyVskbbZZLOdMdsQxafBcsnP2IDEKygnn6
9Dz+e26YfFosLzj6Yevh91WlLpqe7KermiCro2Y/2UXstG1VvRx7jyb6wnK6
/OxH0KoWWQxHbWfqXKcafe6jMz7ypDBHnvWFT4zW10q1U7n81XPv/VsC8lOP
ve3y6y+Pjo3NguUizqmhI83r3izv6KWs6OpS2ciMSxly6/VOn1memXNNn+WK
RpIbrvU1kYYxmI8gpdf4agjbNXDoW7UYj8uvFJb+fFnUyrYhAvLtYzO7Jm5F
Olr/cPbiH6/YfmPv+klh26Lshq+nVi8N31rpqB8cHdP5HnEdF/015/qfJfIe
yfM9yZW1KKEhiV8flUqG4Tlh+YeytnIsr7+q2kEQkD+QGL6udMtE32Gl221/
dLZVaLFc+IaLXwCmW+G+2MJ3QOpfuBdib5u3ZnH4GonfJLEKJDRYTsdJ9GsG
WtSxQhejVzBqhNbUEKUWy68d/vDV/e+RLNSzs3wjvbGldLPq5Yg7dNxXlksf
XxGrMdXC4l98rXoRdapxViz/5D8Ii91ynv3yhLNNYnn9jthz73uUgPzUow9c
eu3PwyMjs2O5oqmoX3epVkmn7H7NVZrYg8vjcnKIuz65TtzsmHZz+3V1lre5
WT5w807i9WnVNOnZV72wnKkEbpdQPPo22n1D46kK6SOQ9J6vp8VCv3bfx4i0
ll77oVqMsDy/rOqXsYdeOpb++/I2EojHDEzunLi1zloWu+/oiaTzOw7G//pY
2k+LnALLv3Qg+1ur4761Yseftx+53tOn3XEx9zfhefSwk9ZrcUbDcx+gXskw
PCcs/961Y4+ej34iPeYDGZvfn75J6Bc21+e/Nzn6ieTotyWG339y5dHKzRNd
cUq3l/6PHsubha+08K2guxX6+yxt92wUvzxCX0ASqN3SsjOg9F2w6sA+BS1Z
QCb/ZioK5j6X2X5Zk+Unnrmy+2HJQoW3lr5Jb3Ta41X76x0H4n2Py5sphtF9
txrjZcRS26hTjbPqo3MWfZuA/PIz/zwzPuYeY791i/v/9m3HxnUE5ImP3J+z
KWZkdHSWLKc26lx3d70xfbH8JkZ98Em1AvXd1KYefze1tgsvOiZv/3Xy8OkO
FZynXryiy3L2AtEVQjUedpxG45pq3e5oNjnfq8Ury8cmp/5j2bofZNf/vKT9
fx29y53D63snNg9NLd+0q6Gl1dXZ1exsfyN627/l1f/Lpfqvpdg/c7L840eL
P3qk6JP7r5xKu6jx0ei2LXREnqqQU9lTk/xR6rvUvqEw7F+W720ufex89MMp
6/42OepdSZFCv7DOceXBU6uEoPyR0yubmjaMt29U2ln0S32WU+2b+j7TI1Se
4TgpgQBvCeES1A30zoovjNRxUJ2UjPdUh6VeMB/XQGmxvDIvJnvnQ9k73yHY
HXgVr5e25Oz5u/7uRtU+fdu+t2bPcrrqtKtFLVxjBhW1q3F2fXT17h0pf/9w
+Z9/K4C8LnbDxW98YaS2hotKy4rP/v2jiU++q+zKlcnJKb+x3MB1Z2pAPqIu
5ZwrT2WS5Q0tTiYcH799tmN6Xf3U/1q7V9dOEr9sn7zv7NDDqUOpHfKPn3Lh
sv7aN41JE2YEWI/lTC2ZYrnv1eKV5aM3Jn/851WLLtX/e2Hbf1d1vtLYH+4a
iWgfjNyyZ3B0jOwl/24/GL80vfy5rLovJ1V9+njZx44WEZZ/dltqRk6+bhDi
HmCQIg1PVECXX9bXUbu0KlnLi5cseYgSeQuSwSZY3jY69LGsbRzIk6MeOrd2
aHqSeKX9ogDyt51cua1w1VjLGlW3XfuJN5ZLk9+5MpYrw1wRHrnhfG/i/haR
TkH1mz9nLFeJv/3K8tZWe/aep7K2PSB4emqQuOHaCveW7Q/a87fcuHFDleVb
dh82w3J6INR3ltNrpqiObK5Y3tPamv7pDyc98VBl+Ou1G9efeTe32C3zy58Z
d7amPv2hk48+kL5qeZuz/Ta/IG62Y+zCZzdw3bUWCSpZPpsVUnVNbfTHye6d
+WnJjbenjljODQu+j/x7dshyut8Sfz2jg2kk59KzteNyeciotWRPk+X0EhbT
cbnv1eKV5SMTN3YfTfxR5M4fnsn/eWHr/1R3v9zY/5fWwVd2HensGyBRe/fA
4B827Vmc0/jNtJp/Pl3x9LHiT+7P/WrM8R+9HNF6vVP7eyQ0b2lixWeWm1ts
+MQTTwggJy+AMdj0b8UU9LY/nbXjoaRI4SE0Yb0bAfkDJ1f+MmtZv+O10Vp1
t+WGeWc5s0KqXe+7LXS4Erz5F/x4u/F17EbH2Kmza81+zRXL+4aG7eVpl/b8
w4XYv1K6KPk/+nq7bvGjykpt3nVIZxk5U1HaLDc5xs7kLBtj16jGWfXRpP9t
zr96/nMfP/ued5598l3CYrdTjz5w9qlHTj5yf8pLP6qqqBwfnzD3fLnG2jfj
112rX1aOsfuw5E2yo6FFxvKfUCy/T2L5qT5LvOsbmcyi9zPnL3odY/eUTaNJ
a7FcsTTV9Bi7b9ViJC7v7usvLLO9dTr5J9G7f1bS/uuKTkL0/8qreXXfyQ0H
jr+yJ+EHZwueu1D7leSqz54sfyF8264jJ85fvFzX3KKyAk423cCOQhkaY6c/
pk/9xreffZYYDIN9YTk3sDnY83zO3qfOrXrozGrC8odPh3/03BsbrvxptOo3
Om7L/p4RlkuDbMwSMypwD/d0N0vCw5d4bv9fXBP+osaXYRZr3+TbvRbMN5YX
q7Oc9D6dvX32ipyct76RueORjNi/ztj8fy5sfZDQvfzy+q6uLtWnrQTF7Dig
93SZbLCX6bhVeiVja9+oHorJR68ajVSOTh9NcN5YUpLx/e+c/fCTp9/zjlOP
vu0UgfonP3ghfIW9umZkZFT14XLTz6TJ41T9685UaXa4zpgHtXrDJMDsdU30
x7mkxfLE3sdOXy/pZ9rJqZRMA2vfmGUB1DdijZcxdll7MMdy36tFv51IoXnf
0FCxrepHUXFhuU0/trb+tMj5UpHz3wvbfpjfQiLy5zJrv5ZS/YVTtk8dK3np
lYj65tauvn5hBN7oUoApw2vfFD+qoaxkGJ5DlhN3d1fnX/3j5Wu/ysr/Vb71
P5uLfjFc+rORMj23Zj1jjOUqP7bgmUxil33Kb+e1Fn/O6pk0zzwiu0JVo2A+
sbywpFSrDyKdCOlNGpqbKssvlVpPFl19KyfzcHODrbe3d2pqSue5oze37fPy
sLjqlCRffnKTpNxl6Jm0ZuphNgbnetXotXJcHV7irS5yx3M179qJ4yc2bCo4
fbrSaiXd+sSNG1ogN/dbMQqcGLjunnlfegjEXe3MUYZ+v0hlLYWjnvk403fq
xm5fG7h1oLgpr3+G+FzntOXM4HtT++xD8uVvJ5MzvLBcLCqDc/YmW2e+XGpd
4XnU99Qgy32tFq/tRLr5i40/981Txc9eqP3epfrvZzd+/3LjC9kNz2fVfSvd
8dXkKg7kCaUfO1r8ha0pF/IKhJ9nNzRX5XXpgOKZNK0lC3g8HZ4nlvd1V18v
+O1I4YvG3Zr+JXWWw97+tgph1cDIKAnQ2zs6m53tpeWV4+PjMzMz+s8QR2/d
6+vv6gTWGlqDfzODhFz9w6PWsurxiQlyl6M6Rz6Hv+E67y6vrlX9XKUV1dI6
9vemD+f1qaxjP3EuPSS/REbaCWkk312x5fOnbF86V/m1FPu3Ewu/mVTxjdRq
QvH/e6bycyfL/+nINWHJ28cOWX+7+aAmy2E4RFie/+vhq9817pbUz4LlWm5s
9f73y0mfQjw8PlHX2HzXgNbF7gkNlpPK6Rsw+nepbQ5DlRMCLG9sbT9xNu3I
8XN7Dp/YtufIhm371m7cuSIqNnrzLuEDjkzfKR5UX0uRcDYtJL9ERtoJ+Qa9
se3APx3M/aeE0m9HH3kleltYxK7PJZZ95kT5p+NLvrNi+3+ujPn81hTC8n88
cHXFzqOj+OO5cEiz3O68+C9DV54x7pbkT5Vl/ak49SXUuNJdff2T0zNGUg5P
3HDUG8JVVOzu0GA5qZyhsXGD9bNwWK7la0VlXj97/JnzIfklMtJORiZu1Da1
rN995I9bDp5Ny7Q7al/ddviTh60fP1r8iYP5UfsSissrYvcc+UnMoWU7jpCU
YDkcyiwfHu27vP+D6bF/ZdwkPTkK1a0VK0zcnDaW8ka1o+HW7du3bgl2a0bw
jMfrtuwJmcoZuzFpsH4qapvvGNPQyFhIsjzXWnxbtXlQbSMhFFlusJ2MTU4N
jIw6Ozpb2ts7enr7BoeOJmd+Yn8uCcSf3nv5fHZuz8Bge2dnfXNLS7urf3gE
Y+xwKLMc9q9N9Rd1zW3cH7+qqC4oq8ovtuUWlF6+VnwpryDzyrWM7Ktpl3JT
s66kZF2pb3EuwMrpGx519fQbcUfvYEh2001trvLqupKKGqF5XCkozc4vysq1
Xricn3Yp7/xFrnk4O7sX8pdImKsiJjE3cXl17a9iDnxr5c6Xd7zV0NpOtpDY
fXBklET5ADkMlsMwDAfBHQAJ05vbnPbaWhKIa/4lFRgGy2EYhgPWwq+39g+P
qD9HDsNBzXLyLwzDMAzDQWHE5TAMwzCMMXY4FD3w5AdRCTDaBgyD5eafqPnD
r4lxVQKhsxaMqoDRNmA4dFje5nJdyc1NTk5O0hDZRRKQZD4XxbYxsubv7ycm
L3BhAqGzRpcNo23AcOiwnBA6LS2to6ND549VkF0kAUnmG86L16/q/sg7ix+z
EHMv1q/CtQmEzhpdNoy2AcMhwnIScHfwf4PIq0gyktg8yFdXPvU31x6zHPnZ
YmLygrwlG81/nt4jv8tZerqX3thyOv/+39lbcLHNj6Civ4bRNmA4dFienJys
E5HLonOS2FQJHEf3t3/oHYTfb379451dvUNDo7HPPk3eko1kF1h+rwIvTI7C
Om0DoTkMBx/Lk5KS7hoWSWz89CUxa6vf90DJ4/cd/uF3XB3dw8OjxAODI6d+
sYRs5HbFrPUvy3Nic+5/VnD+kXYxWWEp97bQvpTfxeXQTr2mTyFuv//Z0pxQ
Bzm6bFhnyRsaBgyD5Zwbk07VvO+B/McsW575RKvz+uDQiOS+/qG45z5LdpEE
JJm/WM69FhlMv+ZZ7sYzv51+TTO7ea14B8DdE8Q2hzbIgXNYvw2gYcDwQmd5
5b6dTR94e+nj98U9+5nWtva+gSGZu7t7Dy36IklAkpHExlkuht2U3SwnJM5Z
W6gGfiEub7+p91oW4nO7Qic01++U0WUD5LjPg2GwXO7aUwnlT/1NwWOW/c9/
ob65radvSNVd3f3HlnyVJCOJySGzjcu54XFqXJ2OrY2xnBqfD6lhdq89Mrps
sBz3eTAMljNuykh1fuShksfvi/rEIw2NLV09Azq+fr07+unHSGJyCDnw3rM8
5JbRGeQ0cI6gHA0DhsFycdX68aOlT/1N6eP3Hf7O52obWjq6+gQ3VhRUXDgq
mbyVdhGfWPRlcgg5kBw+i/lyf4yxh+6SNz+mhHGfB8NwiMflZ9/7YMQHHriY
V+jq7JVcmnYoPWqpZPKW3puVW0AOifvKg3O59s07y4W7AWnJW05s0HPdbC+M
Lhuj62gYMAyWc+5sbU44nVJQZnde75FcmHzgbPgSyeQtvTczt2DXwWPkwLl9
Js07y0Wc86ZC/AUBckRgCMrRMGA4iFnu99+KOZ991Vpa1ebqlnz1zJ7jf/me
ZPKW3nvhinVfwhlcrXsLcvTaaB5oGDAcxCz3+2+4CixvcXZJzkncdejP35FM
3tJ7L+SA5fds+BQDqmgeaBgwHAos9/vfViEsv1bCsPzS8R17/vebkslbem8G
WB4YQTkiMDQPNAwYDlaWj/v7b54qWZ4Vv33nb74umbwFywMT5Oi1EZSjYcBw
sLLcv1ayPJNleSZYHpA9NQZU0TzQMGAYLPew/NDJ5K0H4o0bLA+coBwRGJoH
GgYMg+U3G1pdJVUOU7Y5GnC1Ao2+6LURlCM0h+GFy3I4qHtq4By3emgVMAyW
w/e4p55NV6uVD3ptgBw4h2GwHL4HIJ/N74EA57jVm6P7RRiG/cNy8i8cSvbK
4HnOBw6i5jGfmcAw7JvvfVxOzhgCp1ggM+iItBbgsA0GbOB7a3TgwTHGDpaD
5XCwtw00DBgdOFgOloPlMFgOw+jAwXI0BbAcBsthsBwsB8vRX6PLhsFyGCwH
y8FysBwGy2EYHThYHuCnKNz5lecW/ek0/efgymKfWxRbCJbDYDkaBjzXHbjr
zO8XkU6Y8++TXfTGLWVgOVh+01paRWyQ5V957uUz7WA5DJajYcDzyoiSLRSz
C3eyYRXi8gXP8ub2zjUbdhOTFwZY/vKffk/fErIsd8Oe4T3X/n6fXHL6ZWm7
i3pN58+l5A+fnzYKlsNgORw8LNcKnKi4vD35T8+JgbvbO0uow1U7XrA8RFi+
+/ApgeXkhQGW7yzhgS02Kap5kWYk3jPS949uQnNvxQEi6TU1LsQB3tMc56O1
geUwWA4HWVyu0jdqjbFznbMYF3FphI6a62k9wRhYHiosP5eeI4BcMHnrneXu
JiXc7qnfKtLNRYjL9V+z+XDNbh5Cc7AcBsvh4Jwvl6JtLZbzG6UOVuy65zNY
Asvn7xTW0ioa5IL1Js49DYIfruFaD8NycfCcWZphiOWKoSGwHAbLYVilAxd7
SzrmlrGc74o9vGd65lAcZl/QLJemyZXWnDinb+7cI+0elvPNxd1ETMflfOuc
5zV0YDkMlsPBx3LPeLvQGytYzkyDqqAda99CiuXSNLnSmhPnNMuppWp8o2EG
xn0bY5/nZZlgOQyWw0HKcmqoXEcagPIAABjSSURBVMZytb6UX7YcekvewHL5
NLnS6hPnLMuloR7hBtDDZmG7OZYzYT3JIRZj7DBYDoPlzDp2NpRy9580y9lp
co3pc9fpnRhjD8F17CYsY7k4DUOvaXevy6BwbpDlzKTOvCyzBMthsBwOpric
Xlek+lsxnqeClVPjnt+ZCb0H08Fy9NdgOQyWw0E4xg6D5TBYDoPlMFgOloPl
YDmMtoGGAYPlYDlYDpbDYDkMowMHy9EU/NJfmzWqDiwHy2GwHCwHy8FyGCyH
YXTgYDmaAlgOB1jbQL3B6MADiOXkXxiGF7J9YznqDYbvidVZfheCoAUvH1iO
SoOgeyKwHIIgCILA8jlWb19/jaOu2lFrr3FU2mtslXZbZVWprbK4zFZUUp6V
nWOvdkxOTeFaQhAEQWB5gLLc2e7S2VvtqBseHiGAn56exuWEIAiCwHKjLO/u
7s7Ly0tOTk7SENlFEpBksy9im7NdeDEwMp5WWN3RPyxjuRC7V1RVTyE6hyAI
gsByIywnhE5LS+vo6Lhz545WGrKLJCDJZo/z+ubWyZtczJ1V4vivTQkp16rI
a7Jl5tYtieVE/QMDV62Fk5PAOQRBEASWe2M5CbgJp43kTpKRxLMs4objWeEH
U89drVi2L5mwfO3R9LyqptWHzu9PuzZz67bEcnL30NffX1RSdvMm1u5BEARB
YLmukpOTdSJyWXROEs+yiHtT8wnCld52JkdgeWNTi+R8a1FqRmaAVrarMMyS
aCFeWtjF7ulKSLFYsq1ojxAEQdD8sDwpKcn4CUwlVlVOmUOA9//Enlh5IFVi
eXNHH9k7MzMzNjZOe1vcPlP58xxNpDxHTO2JX5poiXAIUI8vuGuN8EAdLIcg
CDITGsWHWUQtjWejo674pVq7SGcrHRcW79Lhgid7LmlCF1juO8v7hsePZha+
vuccIfcrcWev9w2RjeUNLoHlmxOz4y8Vt3T2y47avD3OPMsljjoiLSJx/Swu
57CEHgNlgCAIgryAPLJAeGONJLCNsDIgF99aI1icF0RKVOZ2WSKteiwX9xZw
ZxBPB5abV3Vr529ijwvk3puaL2ycvnVLCs3/e/PxHFu97KiYLTtnwXIVtBMA
czE0H7XTMHanLMhmhs2Ft7LE0ug6tV03LueDeD5xgDcgCIKgeRYDWhmwOe7S
hOZIr96LMjcE+qdg7g/ActOavDld6+zaevoyF5fvOtM7NEY22hrbBZAfuVBY
3dI5oSht9Kats2E5jVh3mC4BlUeydO3FwfkUz0ANnYB/TbFfHpfrsJzbJYwN
cHcGKfEufHkhCII84TUVKDOglWFeD8NcJprD7Gw+3A1BgA+zB8F8edJVmwDv
P2w/FXPi4u+2nBDeDoyMq6aPion1meUCnqk7NRmAqTlvd2IGtOx9AJ+AvS0w
xHLmjkFvZB6CIGhhRubCjDjpJ2Xw5t/ShNZiuRc809nKB+rBch80c+v2K3Fn
VdexH80qVF1NHxG90TzLxQFw+QpzE8G0CneZqNpoVuwtAnP3AEEQBN2VEGtR
znkz0+fuJWwUicWjvMTZzNq3iCBYyxToLL99586205eOZhbWObvXHE4jCN+f
dq2kzrn2aPrp3PLbt1Vgvipqg49xOT/VzU6fyAGsgO4csTyRMVgOQRAkyTPV
zZNbDmb3RmE7B28VGAuRvaG1bwL+sfZttmPstkq78OJsXsV/bz5e1eLlZ2rC
1673keUCX3XxPA9xOX8U1rRDEARpR+QUnhXj6nJma4TgesPsesvrQobl8/xb
MWW2SuHF8PiNmtbOqekZ/fTLV0f5zHL3gnNPHOx1vpyB7hzMl0MQBEGKkJoO
tXVWpHO7tIJvEyxXLKkLCZbP82+4lpTZTKVfFh7pO8vly9+EdexibM2uKld5
KNyP69hptGOMHYIgiI6SqUCcDZoJoTXXn5OUEvJlmeiyXPYIe6iwfJ7/tkpx
abmp9K+vjPCSgg2+NZ7tFrYIAC6MtKg87a3+Ay+qz5ebZLmIc/UffIUgCALO
LZq/++aZL1cG69SBJn73LTTXsd+d3795Wlhcair9q8tX+69+8EgYBEEQFOjy
keXzqbZ2V9y+Q9t379+6c+/m7XEbtuyI3rQ1KiZ2bfSmNetiwiPfXLFm/Rur
o/4Svva1FWsIyNMzL4LlEARBEFgOgeUQBEEQWA5BEARBEFgOQRAEQRBYDkEQ
BEFgOQRBEARBYDkEQRAEQWA5BEEQBEFgOQRBEASB5RAEQRAEgeUQBEEQBC0Y
lvf29dc46qodtfYaR6W9xlZpt1VWldoqi8tsRSXlWdk59mrH5NQUriUEQRAE
lgcoy53tLp291Y664eERAvjp6WlcTgiCIAgsD8S/k9bmbNdnuRC7V1RVTyE6
hyAIgsDyAPz75S2tTq8sJ+ofGLhqLZycBM4hCIIgsNwby0nATThtJHeSjCSe
ZRGbW9uMsJzcPfT19xeVlN28ibV7EARBEFiuq+TkZJ2IXBadk8SzLGJTS6s+
yxubWiTnW4tSMzL9XEmuwjBLYmSB+501ItGytLArUK9oV0KKxZJtDdpiBHj1
QhAEhQjLk5KSjJ/AVGJVEULr7J2ZmRkbG6e9LW4fWA6WQxAUurJGWgSFxbt0
ExaICSmFJah1MK74MH6v1NVLnVn8UupEYjK3lsYHSGcVBCyvb2wylX7z9rgQ
YLnPLDR2YE/80kRLhMN07gXZFkuKly8PWA5B0DyI43SYke7I0CESpCOsatsZ
los44G8pIqyBUBlBwPLa+gZT6WO27DSQioeZhTeNNI5V/EYaJ6ZZ7oi0JIYl
9PA88+TGHag8I1US5hQWySI7+WIIG0nmCnAKu7LjZRCVPpEnf654nvzFz0Kd
VJ3B1Fno3KiapKrFzXJPmak82frUr171UrkPET5IirkvMwRBC5TlXIStHpSL
kA5bypHaykQXwkZVlpOOTp4eLNcGY229qfTRm7YaA7mbDYQ6bjRyhBDAwIat
PrHcwzbZ/QH/lslNOBEb8iriWkekeFJ+FwUw+kA3O8UDyVvxU/BQlDKUx+Uk
T4b0WiG7Ii63RjCnlm4y3OCnbxSkSjPMcs1SibcIsnsaCILAcr30OkPiAqQT
GFTzkXeklT4Ry3JC+gAZZg8Clksr1Q0qKibW2xVVHyiWUYQhog8s96RReUvd
PTDhMstCjTFq5igVKqsfyHxqvTF2vQ+oN8bO5NmlMjzAUN9gXK5eKva+AYIg
sNybuPFwxVy4guUF/Oy4OGzOhd3ktRbLfRjkX8gsr6o2N60bEb3RyyWNUB1G
ZiM/bfYYH2P3+paNsHVZqMlyLje6gQYoy+ky+4nlel9LCILA8rt0xxjmJYAW
Ic0PmwvZEq7zLxQsFxUQo+tBw/JKe42p9KuiNnhnuQot2FlkxUT1nLGcPakO
y6mZb4sWFxUHsqfQYTk17W0xxXK23jTLrzUaoV+9GqUCyyEIMs5yl2zkXD8N
F8Fz0+rSmLxeXI61b4Zlq7SbSh++dr23G7QUNVZx2FAftp0zlvNo1FloJl/C
psJCXZYLIBebnU5cziwgMBOX09MHfo/LtUsFlkMQZJjlhma1KUgL6eMj1MbS
ZbcFATPMHgS/FVNmqzSVfvnqKG8twPt8+TyxXJtJMhZqk0yWOX0gC2wdlhv/
gLKq087T+Hw5c8NBn12nVGA5BEEGOWokKFcNuKUhdB2WG8w8MFk+z7/hWlJm
M5V+WXiktyRMwEcwQ5NV4qJshfacsFw2eU8tO1dZ7k4TWs5CJhBnUrqLKn98
S35eZqhA+wPKFp1RhXSfmmG59FY2DEAtTSCVH5Edps1y9VKB5RAEGWO50cfG
FA+Oe55e01/HHrzPpM3z31YpLi03lf71lRFGkqk/7U09w212cZZvLGdKwmYr
bhcA7JmWjiyQ45z+LMoV+GImMpwzD84xj8LT4BRykK+T95xdcWp3YmEiw8o+
Yc/G92Ll6z6TplMqsByCwHI5y4W1adQcttHHxnQibM21b5bgXsd+d37/5mlh
camp9K8uXz1fLSlbvnINP1gGQRAEBQvL51Nt7a64fYe2796/defezdvjNmzZ
Eb1pa1RM7NroTWvWxYRHvrlizfo3Vkf9JXztayvWEJCnZ17EdYUgCILAcgiC
IAiCwHIIgiAIgsByCIIgCILAcgiCIAgCyyEIgiAIAsshCIIgCALLIQiCIAgC
yyEIgiAILIcgCIIgCCyHIAiCIGjBsLy3r7/GUVftqLXXOCrtNbZKu62yqtRW
WVxmKyopz8rOsVc7JqemcC0hCIIgsDxAWe5s1/szNNWOuuHhEQL46elpXE4I
giAILA/Ev5PW5mzXZ7kQu1dUVU8hOocgCILA8gD8++UtrU6vLCfqHxi4ai2c
nATOIQiCILDcG8tJwE04bSR3kowknmURm1vbjLCc3D309fcXlZTdvIm1exAE
QRBYrqvk5GSdiFwWnZPEsyxiU0urPssbm1ok51uLUjMycV211RO/NNES4bjn
5bBGJFqWFnaZPs4RaUkMS+jBhYQgCJoVy5OSkoyfwFRiVRFC6+ydmZkZGxun
vS1uH64rWA5BUOjKGmkRFBbv8pbWFR9mEVMndHlNFlkg29EVv5Q6EZUbp6Xx
XYFRI0HA8vrGJlPpN2+PA5Jne2BBtsWS4v1LolBXQorFkm0FyyEImmsVRHpn
OY9eEc/cHYAmziVIR1jVtjMspzOUpwfLtVRb32AqfcyWnd4TuQrDLIkWzinx
CTJu8bTj99I3aCKlOJa4D3TJGaO2XfPsJHOOZ8JRHFw952VZpcxZ2sJbJKIn
NzlNPTlbIgpZllO7xHz4T+rJX6wElZQqePYcKJaWuy1QqU83y6W9VJ4yzLNv
ZSxXL5X7YgmZ+3LHAEFQKLDcGsGyljsk0qrN8rClYbIEJAd+oyrLSVcTppkh
WC4PxGrrTaWP3rTVSGwn8kyAgQfAHDmEXWxsKhJOwCR/lAcS3FsBMIYCU+lO
gj+Rh52et9J5tXKWh9dkr9i86E8nZ6H7XOJea4R4Ir5IHkYq4nLNlPpxOUns
ORdzk+EGP30V6JTGWK5VKrFKfRlagCAoVFjOD4/TLGdJrBLBJ8gSkMg70kqf
iM2Bu1cIjGH2IGC5tFLdoKJiYvUTyHlDc0uMmJXYYBHLZqKZgy7LJdyqvXXn
oJmz3lA5Az95ebQOZLfrjbHrnVrvVobNUwZs+kAzcbl6qWQXC4KghctymrVe
WV7A4J8Lu8lrLZYbGeQHyyVVVZubFY6I3qifQD5TSzFGGROzePBQSs4ezy4D
E7oyvmq/1c7ZMMu5T0fDNUBZTu/1E8uzrejsIGhBs1xII8FbWMWmMWUuQpof
NheyJen5FwqWi4oMnE4mCFheaa8xlX5V1IbZsTyRsVGWM/Yry1VzVgKVmjmW
T37rsJydetdjuUZKbyxnq1Sb5bJPbZTl6qUCyyEILKdxLi5651ar6cbldz1L
5MiBQkyvF5dj7Zth2SrtptKHr12vn0BvjF0exWoeRb/lXptaYGWY5do596gt
YVMZo9ZlOQ9Cd0r9uFw7pfdaUpsv8E9crlkqsByCwHINYGscQkFamAWPj1Ab
S5eN0gfMMHsQ/FZMma3SVPrlq6NMoNS9DM37bLcOy00/wWWY5do5s0Blc1CM
sStW6ass7tNluU5KvVrSy9PwfDkNbJbl2qUCyyEILFfroMI0w2hlwC0Noeuw
XGcCPvBZPs+/4VpSZjOVfll4pIFGQD0KpWSMhAFqGbYey2XL2guy/TXGrpOz
vJwM4eRr7NlF+zTLqQEAxfo7dlm7Rkq1ilUrvDAYLqtnJsJWW2ZIcsiOjNBh
uXqpwHIIAsvdaSR466fXeRJdfx178D6TNs9/W6W4tNxU+tdXRphsEPJxdc8s
tUbgeFdr7Fp3LtknluvkLE4V84X0TEtLD23LQlv3E96Kp7/EnNVw6HkoXCsl
M4rO5MnDmH6O38XgXHj2j33CXvmRucS6z6TplAosh6CFx3JhbZr8mXIDS9V0
ImzNtW+W4F7Hfnd+/+ZpYXGpqfSvLl9tKr3p2W5TOWuspIMgCIKge8zy+VRb
uytu36Htu/dv3bl38/a4DVt2RG/aGhUTuzZ605p1MeGRb65Ys/6N1VF/CV/7
2oo1BOTpmRe9ZUliO+Y31PCboBAEQRBYHmzyjP0C5BAEQRBYDkEQBEEQWA5B
EARBEFgOQRAEQWA5WA5BEARBYDkEQRAEQWA5BEEQBEFgOQRBEASB5RAEQRAE
geUQBEEQBC14lvf29dc46qodtfYaR6W9xlZpt1VWldoqi8tsRSXlWdk59mrH
5NQUriUEQRAElgcoy53ten+GptpRNzw8QgA/PT2NywlBEASB5YH4d9LanO36
LBdi94qq6ilE5xAEQRBYHoB/v7yl1emV5UT9AwNXrYWTk8A5BEEQBJZ7YzkJ
uAmnjeROkpHEsyxic2ubEZaTu4e+/v6ikrKbN7F2D4IgCALLdZWcnKwTkcui
c5J4lkVsamnVZ3ljU4vkfGtRakYmruuCka9/fp7/o7eRBahACIIWKsuTkpKM
n8BUYlURQuvsnZmZGRsbp70tbl/IXJ6uhBSLJdu6AFkLlkMQpPn9jQ+zMJJ9
l60R4o6l8V1Mjyo7TuVYpfjcIq3eCqPIpyt+KdkcFu9SOXVYQteCY3l9Y5Op
9Ju3x6GpB/uthuGcwXIIWqAs1/r+cugVEU6/VutnwvQgLaggUiC1V5ZbIqxq
22mWi5nwefq3/wkCltfWN5hKH7Nlp8F4zsI5Jd4ltQDmrTtNhMOzd2lhV0G2
RTiQvJaDQchTyqEnfqlwChkzpFOzrJJyFs+ohjTpWLYAItL49PJM9CnoOYTP
kK8ERQ78BxS2S+D0pJQ+tUo9UKzlPyDNXT4HFWCr5EzXD1Of7vylQ+T5U7XE
vJWz3HOx6EPc9S+cmqlwCIICkuXcrjBPN65HfWukd6ByacKWhnlluTINuY3g
N6qxXAjZI/wZBwUByx219abSR2/a6i0J13ELnT4LS75DF7tsruuneOamhXsL
n1LaK6JODhIhAUcCFvDuAx2RqmgpyJbyUSkeXQAPXdyMp6nprY2KtwVCJrJ7
FCYHTzn58nhud+S3Gir1wMTNzOHcKTQjb5Wc6Zsqzy73p3AXlSW0cZZbI8RS
8dvZAsvu8CAIClyWK0JtjpqqA9pcSm805cL6CKuX8F0oTIKsSOQmINLKxd+q
LOdvEfw6zB4ELJdWqhtUVEyslxQ0XGXxmdSVK0gjAwMDGxYAimxpnmmMCTO8
10CaLA13CuktM4RgbOSZP0R2N8C+VcmBrS5VlrNHyfKRbkG47Tp3G3pj7Ew9
qOUvG0oxFJffVc1Bdu8CQVDgsFx1RlwxqK4RAcvC97t6twWGWF7AnMh9o6DB
cv2R/5BleVW1w1T6iOiN3m615IEdjR9pzFmxmIIdZdW5IZAzQAEYlQFbNrBW
Q5q8AHq3CEZZrn2I7yxXTigo73K8zgL4ynKmlvzE8qBeewhBoS0uwKW5qMSk
KjiFgNvb7YJqSK05SMAnEw4hXOdfyFkuKsL/nUoQsLzSXmMq/aqoDQZYzlgG
LQU11TbSMFBnOXsW5XC9WggrG9ENFJZTc9WWWbJcexBCn+VslWqyXKfGdFlO
LUSwgOUQFGQxOjVfZiAupxCr1QPRI/MGWe4ZOSf5C2XQicsX4No3W6XdVPrw
tev1E3AdtPY6JokZXgCvG5frTwZ74j8l0oSwlZmfvtcsp9es+TEu111KJstZ
uCjSegJ/x+X0/ALicggKvtBcm7vK+XID687ca9flUp/hpm4mhDuJ+AiR05rz
5f4fZg+C34ops1WaSr98dZSXFDpxoTQJrUjjdb6cYZiRR5400tA5m5ovnyOW
64Sz5lkurTr0UkLtRX8+z5ezCwLYxYZaEyJgOQQFRVzuoazXdezeg3Jld2Qw
Llc8wKbNckOPw801y+f5N1xLymym0i8Lj/SWhF0E7lk37lnfrh7VacXBalRm
ZuU9y7DJgWpsJq/VCGRqHfscstxdBveDW55PKlsw743lzIiI/kg7kzP9YWUP
/rFv2TypVQskh+zICB2Wu1+7R2XAcggKZBVEivAWfo9F/jAY83w5G4L7EBCb
YLlsgbruOvZ7/0zaPP9tleLSclPpX18ZYSAV9TyxOh3lkZyA2Hjtx6+VEbZn
Yl6ZrXJ5nWdOWja2bOL58rlgOVtgOc7Fz0g/X65+FuWacPqBfmYUXSNn8a38
+XXxiXblikXpKnOJdZ9J81xWGudgOQQFprR+2U2+V85L7QF2YW282i4zLJff
c2j97ltArGO/O79/87SwuNRU+leXr56bphOcvxbCLlvDb55AEASFnnxk+Xyq
rd0Vt+/Q9t37t+7cu3l73IYtO6I3bY2KiV0bvWnNupjwyDdXrFn/xuqov4Sv
fW3FGgLy9MyLYDkEQRAElkNgOQRBEASWQxAEQRAElkMQBEEQBJZDEARBEFgO
QRAEQRBYDkEQBEEQWA5BEARBEFgOQRAEQWA5BEEQBEGBo/8P/KALiXwNCTgA
AAAASUVORK5CYII=
"" alt="Screenshot of Files tab in RStudio, highlighting 'Upload' and 'Rename' buttons and listing three uploaded and renamed files: 'cell_metadata', 'gene_metadata', 'expression_matrix'." width="665" height="228" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 1</strong>:</span> The view of the Files tab with uploaded files and highlighted relevant buttons.</figcaption></figure>
<p>If you are using RStudio Galaxy tool, you can get data directly from your history by running:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"># get the files from Galaxy history
file.copy(gx_get(2), "cell_metadata")
file.copy(gx_get(3), "gene_metadata")
file.copy(gx_get(4), "expression_matrix")
</code></pre></div>  </div>
<p>The number in the brackets corresponds to the dataset number in your history, so make sure you put the right number for the corresponding file. We can specify the name of the fetched files in the quotation marks, as shown above. All three files should appear in the Files tab window.</p>
<p>If you are using RStudio locally, then you don’t have to bother about uploading the files – just run <code style="color: inherit">file.choose()</code> and choose the corresponding file which you want to get path to:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"># get the file path
cells_path &lt;- file.choose()
genes_path &lt;- file.choose()
expression_path &lt;- file.choose()
</code></pre></div>  </div>
<p>You should now see the new variables in the Environment tab window.</p>
</blockquote>
<p>According to the <a href="https://cole-trapnell-lab.github.io/monocle3/docs/starting/">Monocle3 documentation</a>, the <code style="color: inherit">expression_matrix</code> should have genes as rows and cells as columns. Let’s check if that’s the case here.</p>


In [ ]:
head(expression_matrix,c(5, 5))           # preview the content of the file by calling its the first 5 rows by 5 columns

<p>We can see that in our matrix rows are cells and genes are columns, so we have to transpose the matrix simply using function <code style="color: inherit">t()</code>. But before doing so, we will change its type from dataframe to matrix - this is Monocle’s requirement to generate cell_data_set afterwards.</p>


In [ ]:
expression_matrix <- as.matrix(expression_matrix)   # change the type to matrix
expression_matrix <- t(expression_matrix)           # transpose the matrix

<p>Another condition we have to satisfy is that one of the columns of the <code style="color: inherit">gene_metadata</code> should be named “gene_short_name”, which represents the gene symbol for each gene. Some functions won’t work without that. Do we have such a column? Let’s check.</p>


In [ ]:
head(gene_metadata)             # preview the top ten rows of the file to check the name of the column containing gene symbols

<p>The second column indeed contains gene symbols, but is called “Symbol” instead of “gene_short_name”. That can be easily changed by a simple assignment, as long as we know the number of the column that we want to modify. In our case the gene symbols are stored in column 2. We can access the column names by <code style="color: inherit">colnames()</code>.</p>


In [ ]:
colnames(gene_metadata)[2] <- 'gene_short_name'     # change the column name
colnames(gene_metadata)                             # see the changes

<h2 id="generating-cds-object">Generating CDS object</h2>
<p>Now let’s store our files in one object – the <code style="color: inherit">cell_data_set</code>. This is the main class used by Monocle to hold single cell expression data. The class is derived from the Bioconductor SingleCellExperiment class. Similar to Python’s AnnData, the cell_data_set stores a data matrix together with annotations of observations and variables. There are three ways of creating CDS object in monocle:</p>
<ul>
<li>Using <code class="language-plaintext highlighter-rouge">new_cell_data_set()</code> function with three data frames as arguments (not their paths!): expression matrix (can also be a sparseMatrix), cell metadata and gene metadata</li>
<li>Using <code class="language-plaintext highlighter-rouge">load_cellranger_data()</code> function providing the path to the folder containing 10X Genomics Cell Ranger output files. This function takes an argument <code style="color: inherit">umi_cutoff</code> that determines how many reads a cell must have to be included</li>
<li>Using <code class="language-plaintext highlighter-rouge">load_mm_data()</code> function providing the paths to matrix file and two metadata files (features and cell information).</li>
</ul>
<p>In this tutorial we will use the first option:</p>


In [ ]:
# create CDS object
cds <- new_cell_data_set(expression_matrix, cell_metadata, gene_metadata)

<p>We are now ready to process our data!</p>
<blockquote class="details" style="border: 2px solid #ddd; margin: 1em 0.2em">
<div style="font-weight:900;font-size: 125%">💬 </button></div>
<p>Since Monocle’s CDS object is analogous to Python’s AnnData, why don’t we use some kind of conversion between those two formats? There is indeed a package called <code style="color: inherit">sceasy</code> that helps easy conversion of different single-cell data formats to each other. However, when we tested this conversion on our dataset and then used Monocle to plot the expression of genes, the plots were not correct – the expression was shown to be identical throughout the sample. For comparison, Seurat did well when plotting gene expression of the same converted object! Although conversion functions are very handy, you have to be aware that their output might be interpreted differently by certain packages. Therefore, to make sure that the analysis is reliable, we decided to generate CDS object directly using Monocle’s function.</p>
<figure id="figure-2"><img src="...
"" alt="Comparison between plots of gene expression generated by Monocle and Seurat using the CDS object that was converted from AnnData using SCEasy tool. Gene expression in Monocle plots is identical throughout the sample, while the expression of the same genes plotted by Seurat is noticeable only in specific clusters." width="866" height="839" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 2</strong>:</span> Comparison between plots of gene expression generated by Monocle and Seurat using the CDS object that was converted from AnnData using SCEasy tool.</figcaption></figure>
</blockquote>
<h1 id="additional-step-adding-genes-symbols-based-on-their-ids">Additional step: adding genes symbols based on their IDs</h1>
<blockquote class="warning" style="border: 2px solid #de8875; margin: 1em 0.2em">
<div class="box-title warning-title" id="warning-additional-step"><i class="fas fa-exclamation-triangle" aria-hidden="true"></i> Warning: Additional step</div>
<p>This step is not necessary for the dataset we are working on but some users might find it helpful when analysing their own data.</p>
</blockquote>
<p>If you remember the very first tutorial in the case study, we started with gene IDs and added gene symbols based on the Ensembl GTF file.
But what if we didn’t have the genes symbols in our CDS object and wanted to add them now? Of course - it’s possible! We will also base this annotation on Ensembl - the genome database – with the use of the library BioMart. We will use the same archive as in the Alevin tutorial (Genome assembly GRCm38) to get the gene names. Please note that the updated version (GRCm39) is available, but some of the gene IDs are not in that EnsEMBL database. The code below is written in a way that it will work for the updated dataset too, but will produce ‘NA’ where the corresponding gene name couldn’t be found.</p>


In [ ]:
cds_extra <- cds                                        # assign our CDS to a new object for the demonstration purpose
head(rownames(fData(cds_extra)))                        # preview of the gene IDs as rownames

In [ ]:
# get relevant gene names
library("biomaRt")                                      # load the BioMart library
ensembl.ids <- rownames(fData(cds_extra))               # fData() allows to access cds rowData table
mart <- useEnsembl(biomart = "ENSEMBL_MART_ENSEMBL")    # connect to a specified BioMart database and dataset hosted by Ensembl
ensembl_m = useMart("ensembl", dataset="mmusculus_gene_ensembl", host='https://nov2020.archive.ensembl.org')

# The line above connects to a specified BioMart database and dataset within this database.
# In our case we choose the mus musculus database and to get the desired Genome assembly GRCm38,
# we specify the host with this archive. If you want to use the most recent version of the dataset, just run:
# ensembl_m = useMart("ensembl", dataset="mmusculus_gene_ensembl")

In [ ]:
genes <- getBM(attributes=c('ensembl_gene_id','external_gene_name'),
               filters = 'ensembl_gene_id',
               values = ensembl.ids,
               mart = ensembl_m)

# The line above retrieves the specified attributes from the connected BioMart database;
# 'ensembl_gene_id' are genes IDs,
# 'external_gene_name' are the genes symbols that we want to get for our values stored in ‘ensembl.ids’.

In [ ]:
# see the resulting data
head(genes)

In [ ]:
# replace IDs for gene names
gene_names <- ensembl.ids
count = 1
for (geneID in gene_names)
{
 index <- which(genes==geneID)    # finds an index of geneID in the genes object created by getBM()
 if (length(index)==0)            # condition in case if there is no corresponding gene name in the chosen dataset
  {
    gene_names[count] <- 'NA'
  }
  else
  {
    gene_names[count] <- genes$external_gene_name[index] 	# replaces gene ID by the corresponding gene name based on the found geneID’s index
  }
 count = count + 1                # increased count so that every element in gene_names is replaced
}

In [ ]:
# store the gene names in our CDS object in a new column gene_short_name_extra
fData(cds_extra)$gene_short_name_extra <- gene_names

In [ ]:
# see the changes
fData(cds_extra)

<p>If you are working on your own data and it’s not mouse data, you can check available datasets for other species and just use relevant dataset in <code style="color: inherit">useMart()</code> function.</p>


In [ ]:
listDatasets(mart)                # available datasets

<blockquote class="warning" style="border: 2px solid #de8875; margin: 1em 0.2em">
<div class="box-title warning-title" id="warning-ensembl-connection-problems"><i class="fas fa-exclamation-triangle" aria-hidden="true"></i> Warning: Ensembl connection problems</div>
<p>Sometimes you may encounter some connection issues with Ensembl. To improve performance Ensembl provides several mirrors of their site distributed around the globe. When you use the default settings for useEnsembl() your queries will be directed to your closest mirror geographically. In theory this should give you the best performance, however this is not always the case in practice. For example, if the nearest mirror is experiencing many queries from other users it may perform poorly for you. In such cases, the other mirrors should be chosen automatically.</p>
</blockquote>
<h1 id="monocle-workflow">Monocle workflow</h1>
<p>Do you remember the Monocle workflow introduced in the previous tutorial? Here is a recap:</p>
<figure id="figure-3"><img src="...
AAAAAElFTkSuQmCC
"" alt="Monocle workflow: scRNA-seq dataset, pre-process data (normalise, remove batch effects), non-linear dimensionality reduction (t-SNE, UMAP), cluster cells, compare clusters (identify top markers, targeted contrasts), trajectory analysis. " width="8000" height="1374" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 3</strong>:</span> Workflow provided by Monocle3 documentation</figcaption></figure>
<h2 id="pre-processing">Pre-processing</h2>
<p>Let’s start with normalisation and pre-processing that can be performed using the function <code style="color: inherit">preprocess_cds()</code>. The argument <code style="color: inherit">num_dim</code> is the number of principal components that will be used. You can check that you’re using enough PCs to capture most of the variation in gene expression across all the cells in the data set. Note that “PCA” is the default method of pre-processing in Monocle3, so although we can specify this in our function, we don’t have to.</p>
<p>Note that this step can take awhile - around 5 or so minutes due to the high number of PCs calculated. Feel free to use a <code style="color: inherit">num_dim</code> value around 100 to accelerate preprocessing and compare the results.</p>


In [ ]:
# PCA pre-processing with 210 principal components
cds_preprocessing <- preprocess_cds(cds, method = "PCA", num_dim = 210)

In [ ]:
 # plot the variation in gene expression vs PCA components
plot_pc_variance_explained(cds_preprocessing)

<figure id="figure-4"><img src="...
"" alt="Plot of variation in gene expression vs PCA components, decreasing exponentially." width="859" height="543" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 4</strong>:</span>  Plot of variation in gene expression vs PCA components.</figcaption></figure>
<p>The plot shows that actually using more than ~100 PCs captures only a small amount of additional variation. However, if we look at how the cells are plotted on 2D graph when using different values of PCs, it is easier to visualise how the <code style="color: inherit">num_dim</code> actually affects the output. We will use the value of 210, which, compared to the results from the previous tutorial, makes the most sense for our dataset.</p>
<figure id="figure-5"><img src="...
"" alt="Six plots showing only the shaded shape of how the cells are clustered depending on the num_dim argument. The general trend is maintained though." width="1768" height="1784" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 5</strong>:</span> The "shape" of the plot showing how the cells are clustered depending on the 'num_dim' argument.</figcaption></figure>
<h2 id="batch-correction-and-dimensionality-reduction">Batch correction and Dimensionality reduction</h2>
<p>Our dataset actually comprises data from 7 samples, so there is a risk that batch effects will impact analysis. Batch effects are systematic differences in the transcriptome of cells measured in different experimental batches. However, we can use Monocle to deal with that!
First, let’s check how our dataset looks in terms of batch effects. We can do that by colouring the cells by batch.  This information is stored in our CDS object from the <code style="color: inherit">cell_metadata</code> file. Before asking Monocle to plot anything, let’s check the exact column name of the batch information column.</p>


In [ ]:
colnames(colData(cds_preprocessing)) 	          # check column names

<p>In our case it’s indeed ‘batch’, but your data might have another name (eg. “plate”, etc.), so make sure you put the correct argument value.</p>
<p>In order to plot our cells on a 2D graph, we need to reduce the numbers of dimensions. The <a href="/training-material/topics/single-cell/tutorials/scrna-case_monocle3-trajectories/tutorial.html">previous tutorial</a> introduced the methods of dimensionality reduction in Monocle. You can replicate what we did in Galaxy to compare the output of dimensionality reduction using different methods, simply by changing the <code style="color: inherit">reduction_method</code> argument. Options currently supported by Monocle are “UMAP”, “tSNE”, “PCA”, “LSI”, and “Aligned”. However, as for now, let’s just recall that UMAP gave the best results, so we will use UMAP here as well.</p>


In [ ]:
# reduce dimension first
cds_preprocessing_UMAP <- reduce_dimension(cds_preprocessing, reduction_method = "UMAP", preprocess_method = "PCA")

In [ ]:
# then produce a plot to check for batch effects
plot_cells(cds_preprocessing_UMAP, color_cells_by="batch", label_cell_groups=FALSE)

<p>We can see that upper and lower right branches mostly consist of N705 and N706, so indeed batch correction might be helpful. Let’s run this.</p>


In [ ]:
# perform batch correction
cds_batch <- align_cds(cds_preprocessing_UMAP, preprocess_method = "PCA", alignment_group = "batch")

<p>To see the changes, we have to run UMAP again, but this time on the aligned dataset. We will specify that <code style="color: inherit">preprocess_method </code> as “Aligned” and not “PCA”. Monocle would use the “Aligned” argument automatically if no <code style="color: inherit">preprocess_method</code> was specified.</p>


In [ ]:
# dimensionality reduction after alignment
cds_red_dim <- reduce_dimension(cds_batch, preprocess_method = "Aligned", reduction_method = "UMAP")

In [ ]:
# see the batch correction effect on a plot
plot_cells(cds_red_dim, color_cells_by="batch", label_cell_groups=FALSE)

<figure id="figure-6"><img src="...
h1UAAAAASUVORK5CYII=
"" alt="Left image showing dataset before batch correction: upper and lower right branches mostly consist of N705 and N706. Right image showing the dataset after batch correction: the cells from all the samples are evenly spread throughout the whole dataset." width="1383" height="590" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 6</strong>:</span> Comparison of the dataset before and after batch correction.</figcaption></figure>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-1"><i class="far fa-question-circle" aria-hidden="true"></i> Question</div>
<p>Does your plot look the same as the one in the Figure?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-1"></div>
<p>Your plot might be slightly different to the one shown in the Figure but this is fine, as long as you see analogical patterns. Some libraries that you’re using might have been updated, giving non-identical output. However, the principle behind the analysis is still the same, so you can peacefully follow the tutorial. Just keep your eyes open and… THINK!</p>
</details>
</blockquote>
<p>Do you see this? It’s amazing! Batch correction did a great job here! Now the dataset is nicely aligned, and the cells from all the samples are evenly spread throughout the whole dataset. It is worth mentioning that removing batch effects was done using mutual nearest neighbor alignment, a technique introduced by John Marioni’s lab (<span class="citation"><a href="#Haghverdi_2018">Haghverdi <i>et al.</i> 2018</a></span>) and supported by Aaron Lun’s package <a href="https://bioconductor.org/packages/release/bioc/html/batchelor.html">batchelor</a>. Also, due to the machine learning elements of the code of this technique - as well as the fact that packages are updated regularly - your plots may not look identical to the ones pictured here. Nevertheless, the interpretation should be the same - the batch corrected plot should show better batch distribution than the uncorrected one.</p>
<p>Now we can move to the next step and perform dimensionality reduction.</p>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div style="font-weight:900;font-size: 125%">💡 </button></div>
<p>When creating graphs, we sometimes use labels and sometimes just a legend. You can choose whichever you think makes the data clear and readable.</p>
<p>If you want to use a legend, then specify an argument <code style="color: inherit">label_cell_groups=FALSE</code> in the function <code style="color: inherit">plot_cells()</code>.</p>
<p>The labels are set automatically, but if you want to change their size (default labels are tiny), use the argument <code style="color: inherit">group_label_size</code>.</p>
</blockquote>
<h2 id="clustering-clusters">Clustering: clusters</h2>
<p>We want to get some information about cell types, don’t we? In order to do so, we have to cluster our cells first.
Monocle uses a technique called “community detection” (<span class="citation"><a href="#Traag_2019">Traag <i>et al.</i> 2019</a></span>) to group cells. This approach was introduced by <span class="citation"><a href="#Levine_2015">Levine <i>et al.</i> 2015</a></span> as part of the phenoGraph algorithm.
Monocle also divides the cells into larger, more well separated groups called partitions, using a statistical test from <span class="citation"><a href="#Wolf_2019">Wolf <i>et al.</i> 2019</a></span>, introduced as part of their <a href="https://github.com/theislab/paga">PAGA</a> algorithm.</p>
<blockquote class="details" style="border: 2px solid #ddd; margin: 1em 0.2em">
<div style="font-weight:900;font-size: 125%">💬 </button></div>
<p>Clusters are particularly useful while trying to assign cells to a certain type, because they are based on the similarity in gene expression. The relationships between different clusters are analysed to identify possible trajectories.</p>
<p>Partitions, meanwhile, are larger groups of cells that usually contain several clusters. Trajectory inference is performed only within one partition, so it is essential that all the cells that we want to analyse in pseudotime belong to the same partition.</p>
</blockquote>
<p>Therefore, let’s perform clustering and visualise the resulting clusters.</p>


In [ ]:
# clustering
cds_auto_cluster <- cluster_cells(cds_red_dim, reduction_method = "UMAP")

In [ ]:
# see the clusters
plot_cells(cds_auto_cluster, reduction_method = "UMAP", color_cells_by = 'cluster', group_label_size = 5)

<p>When using standard igraph louvain clustering, the value of resolution parameter is by default set to NULL, which means that it is determined automatically. Although the resulting clusters are OK, it would be nice to get some more granularity to identify cell types more specifically. The higher the resolution value, the more clusters we get. We will set the resolution value to 0.0002, but you are very welcome to try different values to see the changes.</p>


In [ ]:
# clustering with changed resolution value
cds_clustered <- cluster_cells(cds_red_dim, reduction_method = "UMAP", resolution = 0.0002)

In [ ]:
# see the new clusters
plot_cells(cds_clustered, reduction_method = "UMAP", color_cells_by = 'cluster', group_label_size = 5)

<figure id="figure-7"><img src="...
"" alt="Left image showing 6 clusters formed using automatic standard igraph louvain clustering. Right image showing the dataset with clusters formed using resolution argument set to 0.0002: now there are 7 clusters, as one automatically formed cluster could be divided into two smaller distinct clusters." width="1383" height="590" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 7</strong>:</span> Comparison of the clusters formed using standard igraph louvain clustering and using resolution argument set to 0.0002.</figcaption></figure>
<h2 id="clustering-partitions">Clustering: partitions</h2>
<p>OK, what about partitions? They were also created during the clustering step and it’s important to check them before calculating the trajectory because it is performed only within one partition. It is essential that all the cells that we want to analyse in pseudotime belong to the same partition.</p>


In [ ]:
# see the partitions
plot_cells(cds_clustered, reduction_method = "UMAP", color_cells_by = 'partition', label_cell_groups=FALSE)

<p>While your plot might be slightly different due to package updates, we can see that there are 3 partitions identified in <code style="color: inherit">cds_clustered</code> object. Ideally, we would like to combine partitions 1 and 2 to draw a trajectory through all those cells (we can ignore cells in outlier partition). Sometimes using the default values might result in multiple partitions while you only need one. Then you would have to change the q-value cutoff in <code style="color: inherit">partition_qval</code>. The default is 0.05 and by increasing this value you can increase the span of partitions, meaning that you would get fewer partitions. When trying different values of q-value, you also have to check if the clusters didn’t change. It’s all about finding a balance between the value of <code style="color: inherit">resolution</code> and <code style="color: inherit">partition_qval</code> so that both clusters and partitions are satisfactory enough for downstream analysis. Let’s try that on our dataset.</p>


In [ ]:
# changing the partition q-value
cds_clustered <- cluster_cells(cds_red_dim, reduction_method = "UMAP", resolution = 0.0002, partition_qval = 1)

In [ ]:
# see the partitions with the changed q-value
plot_cells(cds_clustered, reduction_method = "UMAP", color_cells_by = 'partition', label_cell_groups=FALSE)

In [ ]:
# check if clusters didn't change
plot_cells(cds_clustered, reduction_method = "UMAP", color_cells_by = 'cluster', label_cell_groups=FALSE)

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-2"><i class="far fa-question-circle" aria-hidden="true"></i> Question</div>
<p>Have the clusters change after changing the partition q-value?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-2"></div>
<p>It might be the case that after changing partition q-value, you will notice that additional clusters appeared. In that situation, you might either play around the <code style="color: inherit">resolution</code> and <code style="color: inherit">partition_qval</code> values, go forward with the current clustering (adjusting the parameters accordingly), or check the other method of assigning cells to one partition given below.</p>
</details>
</blockquote>
<p>Now we have cells of interest in one partition, we still have reasonable clusters, so now we can learn the trajectory. However, in some cases even this method might not be enough. Then, there is a last resort… assigning cells to a partition manually.</p>
<h2 id="additional-step-assigning-cells-to-one-partition">Additional step: assigning cells to one partition</h2>
<blockquote class="warning" style="border: 2px solid #de8875; margin: 1em 0.2em">
<div class="box-title warning-title" id="warning-additional-step-1"><i class="fas fa-exclamation-triangle" aria-hidden="true"></i> Warning: Additional step</div>
<p>This step is not necessary for the dataset we are working on but some users might find it helpful when analysing their own data.</p>
</blockquote>
<p>Let’s assume we have 4 partitions that cannot be extended to one big partition using <code style="color: inherit">partition_qval</code> (it might sound unreasonable but it does happen!) and we are desperate to have all the cells in one partition to draw a trajectory through all of them. We can simulate the initial dataset by setting <code style="color: inherit">partition_qval</code> value to 0.0001.</p>


In [ ]:
# simulate the dataset
cds_partitions_extra <- cluster_cells(cds_red_dim, reduction_method = "UMAP", partition_qval = 0.0001)

In [ ]:
# see the simulated partitions
plot_cells(cds_partitions_extra, reduction_method = "UMAP", color_cells_by = 'partition', label_cell_groups=FALSE)

In [ ]:
# store ‘1’ the number of times equal to the number of cells
big_partition <- c(rep(1,length(cds_partitions_extra@colData@rownames)))

In [ ]:
# take the barcode of each cell and assign ‘1’ to each of them (now the ‘ones’ are named)
names(big_partition) <- cds_partitions_extra@colData@rownames

In [ ]:
# convert from numeric to factor
big_partition <- as.factor(big_partition)

In [ ]:
# assign the created barcodes-partition list to the location where information about partitions is stored in CDS object
cds_partitions_extra@clusters$UMAP$partitions <- big_partition

In [ ]:
# see the new partition assignment
plot_cells(cds_partitions_extra, reduction_method = "UMAP", color_cells_by = 'partition', label_cell_groups=FALSE)

<figure id="figure-8"><img src="...
iP8PVVXusmTJLisAAAAASUVORK5CYII=
"" alt="Left image showing the dataset divided into 4 partitions. Right image showing all cells assigned to one partition." width="1383" height="590" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 8</strong>:</span> The result of the manual assignment of all the cells to one partition.</figcaption></figure>
<blockquote class="details" style="border: 2px solid #ddd; margin: 1em 0.2em">
<div style="font-weight:900;font-size: 125%">💬 </button></div>
<p>As you saw above, we used <code style="color: inherit">@</code> and <code style="color: inherit">&#36;</code> operators to navigate in CDS object. What is the difference between them? <code style="color: inherit">&#36;</code> operator is used to access one variable/column, while <code style="color: inherit">@</code> extracts the contents of a slot in an object with a formal (S4) class structure. If you use <code style="color: inherit">View()</code> function on our CDS object, you will see the elements of type “S4” that you can access using <code style="color: inherit">@</code> operator. The suggestion list of elements that can be accessed should also pop up as you type the name of the object followed by the corresponding operator.</p>
</blockquote>
<h1 id="assigning-cell-types">Assigning cell types</h1>
<p>There are two main approaches to assigning cell types to clusters that we’ve just identified – supervised and unsupervised, both based on gene expression in each cluster.</p>
<h2 id="supervised-approach">Supervised approach</h2>
<p>The supervised approach relies on prior knowledge of which cell types to expect. We can simply check the expression of marker genes specific to the expected cell types. Let’s then check the markers mentioned in the original paper <span class="citation"><a href="#Bacon2018">Bacon <i>et al.</i> 2018</a></span>.</p>
<table>
<thead>
<tr>
<th>Marker</th>
<th>Cell type</th>
</tr>
</thead>
<tbody>
<tr>
<td>Il2ra</td>
<td>Double negative (early T-cell)</td>
</tr>
<tr>
<td>Cd8b1, Cd8a, Cd4</td>
<td>Double positive (middle T-cell)</td>
</tr>
<tr>
<td>Cd8b1, Cd8a, Cd4 - high</td>
<td>Double positive (late middle T-cell)</td>
</tr>
<tr>
<td>Itm2a</td>
<td>Mature T-cell</td>
</tr>
<tr>
<td>Aif1</td>
<td>Macrophages</td>
</tr>
<tr>
<td>Hba-a1</td>
<td>RBC</td>
</tr>
</tbody>
</table>
<p>To plot the expression of all those genes across our dataset in one go, we will pass a vector with the names of those markers into a parameter <code style="color: inherit">genes</code> in the <code style="color: inherit">plot_cells</code> function.</p>


In [ ]:
# expression of marker genes across the sample
plot_cells(cds_clustered, genes=c('Il2ra','Cd8b1','Cd8a','Cd4','Itm2a','Aif1','Hba-a1'), reduction_method = "UMAP")

<figure id="figure-9"><img src="...
"" alt="7 graphs, each showing expression of one of the genes in the corresponding clusters: 'Il2ra' – cluster 4; 'Cd8b1' and 'Cd8a' – clusters 1,6,2; 'Cd4' – high expression in cluster 5; 'Itm2a' – cluster 3; 'Aif1' – barely anything, no specific cluster assigned; 'Hba-a1' - throughout the entire sample in low numbers." width="700" height="540" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 9</strong>:</span> Expression of the reference marker genes across analysed sample.</figcaption></figure>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-genes-cell-types-and-clusters"><i class="far fa-question-circle" aria-hidden="true"></i> Question: Genes, cell types and clusters</div>
<p>Based on the gene expression graph that we just generated, the table above and your knowledge from the previous tutorials, how would you assign the clusters?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-3"></div>
<p>Keep in mind that these results refer to our numbered clusters, while yours might be slightly different.</p>
<ul>
<li><code style="color: inherit">Il2ra</code> (DN): mostly expressed in cluster 4</li>
<li><code style="color: inherit">Cd8b1, Cd8a</code> (DP middle): expressed in clusters 1, 6, and highly in cluster 2</li>
<li><code style="color: inherit">Cd4</code> (DP late): average expression in clusters 1, 6, 2 and high expression in cluster 5</li>
<li><code style="color: inherit">Itm2a</code> (T-mat): expressed in cluster 3</li>
<li><code style="color: inherit">Aif1</code> (macrophages): barely anything here, minimal expression spread across the sample with some more cells in cluster 4 and 3 – not enough to form a distinct cluster though). In theory, we shouldn’t have any macrophages in our sample. If you remember from the previous tutorials, we actually filtered out macrophages from the sample during the processing step, because we worked on annotated data. When analysing unannotated data, we could only assign macrophages and then filter them out, provided that Monocle clusters them into a separate group. As you can see, it’s not the case here, so we will just carry on with the analysis, interpreting this as a contamination.</li>
<li><code style="color: inherit">Hba-a1</code> (RBC): appears throughout the entire sample in low numbers suggesting some background contamination of red blood cell debris in the cell samples during library generation, but also shows higher expression in a distinct tiny bit of cluster 3, at the border between clusters 1 and 5. However, it’s too small to be clustered into a separate group and filtered out in this case.
If you remember, this gene was found to be expressed in the previous Scanpy tutorial also in low numbers across the sample, and in the other Monocle tutorial (using Galaxy tools and annotated data) algorithms allowed us to gather the cells expressing that gene into a distinct group. Our result now sits somewhere in between.</li>
</ul>
<figure id="figure-10"><img src="...
AADI8f9/g0EhUFHPHwAAAABJRU5ErkJggg==
"" alt="In Scanpy graph the marker gene appears throughout the entire sample in low numbers, in Monocle in Galaxy cells expressing hemoglobin gene were grouped into a small branch of DP-M4, allowing to group those cells. Monocle in RStudio graph is somewhere in between showing mostly low expression across the sample, but also having a tiny bit of grouped cells, less distinct than in Galaxy though." width="1548" height="591" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 10</strong>:</span> Hemoglobin across clusters - comparison between Scanpy, Monocle using Galaxy tools and Monocle run in RStudio.</figcaption></figure>
</details>
</blockquote>
<p>Having identified which cluster corresponds to a specific cell type, we can finally run some code to add the annotation to our CDS object. First, we will create a new column called <code style="color: inherit">cell_type</code>  in <code style="color: inherit">colData()</code> - this is where the information about the cells is stored (eg. batch, genotype, sex, etc) - and initialize it with the values of clusters. Then, we will get the <code style="color: inherit">dplyr</code> package which will be used for cluster annotation.</p>


In [ ]:
# just to keep the objects tidy and not overwrite them so that you can come back to any point of the analysis
cds_annotated <- cds_clustered

In [ ]:
# create a new column ‘cell_type’ and initialise it with clusters values
colData(cds_annotated)$cell_type <- as.character(clusters(cds_annotated))

In [ ]:
# annotate clusters
colData(cds_annotated)$cell_type <- dplyr::recode(colData(cds_annotated)$cell_type,
                                                       '1'='DP-M1',   # double positive – middle T-cell (1st cluster)
                                                       '2'='DP-M2',	  # double positive – middle T-cell (2nd cluster)
                                                       '3'='T-mat',	  # mature T-cell
                                                       '4'='DN',		  # double negative – early T-cell
                                                       '5'='DP-L',	  # double positive – late middle T-cell
                                                       '6'='DP-M3',	  # double positive – middle T-cell (3rd cluster)
                                                       '7'='Unknown') # no info for now, so call it ‘Unknown’
                                                       '8'='Unknown') # no info for now, so call it ‘Unknown’

In [ ]:
# check the annotation
plot_cells(cds_annotated, color_cells_by="cell_type", label_cell_groups=FALSE)

<figure id="figure-11"><img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAlgAAAIcCAMAAAAdcSCUAAABGlBMVEUAAAAA
ADoAAGYAOmYAOpAAZrYAtusAwJQzMzM6AAA6ADo6AGY6OgA6OpA6ZpA6ZrY6
kJA6kNtNTU1NTW5NTY5NbqtNjshTtABmAABmADpmAGZmOgBmOjpmOpBmZjpm
ZmZmZrZmkJBmtrZmtttmtv9uTU1uTW5uTY5ubqtuq+SOTU2OTW6OTY6OyP+Q
OgCQOjqQOmaQZgCQZpCQkGaQtpCQ29uQ2/+liv+rbk2rbm6rbo6r5P+2ZgC2
Zjq2Zma2tma225C22/+2/7a2/9u2///EmgDIjk3I///bkDrbkGbbtmbb25Db
/7bb/9vb///kq27k///4dm37Ydf/tmb/trb/yI7/25D/27b/5Kv//7b//8j/
/9v//+T///9KmeYmAAAACXBIWXMAAA7DAAAOwwHHb6hkAAAgAElEQVR4nO2d
DX8cx5GfIR3BFaU4F0KWVlbi+C7hWZZkQpcoIWQTjn13hIjkLpQgnHAkwfr+
X+N2XrrrX9U9rzs907Nbz0/G7s4bFvTzq6qu7tk9IcNIwMnSb8A4TEwsIwkm
lpEEE8tIgollJMHEMpIwTizT0ejAxDKSYGIZSTCxjCSYWEYSTCwjCSaWkQQT
y0iCiWUkwcQykmBiGUkwsYwkmFhGEkwsIwkmlpEEE8tIgollJMHEMpJgYhlJ
MLGMJJhYRhIOQKybHbsfS78NQ7B+sW5qsUytrDgQsW5qscyuXFi/WKxW+Wzp
N2NUHIBYHK8sYOXD+sW6EbnQyISDE8v0yoPVi3VjYmXJ4YhVvVj67Rg1hyNW
3W3gZqlZtiQrF0skQn6kqvew9Ls7ZlYtlqqvuJtlUi3OAYlVBy0ia5VmwKrF
4p574JOJtTCHIZYfE+o2qem1FAcjFtbveMByb+64ORSxBEu/LWPtYtWr/MhH
K7LFM5mwbrGwMwobvFhm2GIchlg3Sqz6qYm1HGsWS7SwVDY0sRbmEMRCt6I2
mV/zs2axxH0UUiveVL1a9n0eI+sXS2VBLLAaQ5iRnAMQi2SNpYaE1oJYhjWL
xVM4gVh+2Yx4YnrNx7rFgtbCDUmxbtg7UYkt/J6PhjWLRVon7xoOF0Esi1rz
cQBioTF+nZ+utiwdzsxaxcLSigOT74rydu47gGlGctYs1g2C0zmcEPlA2Lr0
ez8K1iqWtEXkQjSND/UxywLXLKxTLNZGFO4+QUK5pQPbjYk1C+sVC4t0fA13
F96EYqmy3kjFOsVyQIUuC3hMkVopE2sOVixW0MHy21vFMqtmYfVi+VEeaAQv
sTuq3DLBUrJWsUSnnd0RS2XUCNHEmpN1ixVWTuFw0E9EWz6ck9WJJYyQ4Ypv
s4csyGNEMrFmZNVihflNL224gdpLHr/YH3AkrE4sAreaPNGyuTjWfIYxNWsW
C2orrsZ12XWjZDKx5mGNYtXIEioA+hEQssi8moeVi9WAC2Zcc0UaFEZKVixW
QZD4IoELUieZWHOxcrHCO3XYGmVQcNhib/koWK1Y4Vor/8rvxnwYaWYt9M6P
g1WK5TtWYkrHv3ZP+NB6s8qYy/0BR8BaxWooxVvFCoovIx2rFIt0wKq3sD5s
FsQ3nxnNrPSsVawCXatHxNKJEk9b8q0fPgchFmGA8jvVshl1jomVltWKpdro
WM/DjyDzZSfWTz/9tPRbSEKHIj+cnZ398mXx7O1XZ5/82POsOdDzMzDPTFiz
EwXhLCuzjlSs75/WT95995R++LTnWekR/QVyT27ArgZ7oALLQqxD9apDkXd/
eF4/e/v7l/T6i5e9zpoBbDhUP0OxRK/dnSar+cX56UjF2uW/s7MyaL3+3Y/0
9utCsw92LC0WluPkkx67BJuhP4q+ZRKxDtarDrFe//Z5HbV+/sSJ1X1WeiAs
yawoJp0JdZPrlnOIWYVTB+tVH0XKOosjVs+zEhMppMJa/YZVcmIFvfrlOFyp
CvqKlVmNVQOScGAiXHvljoLVDZRFvDp02hUpEuC7PxY6vfvuy5xGhRXYshJi
wRyOfyVKexMrNd19rI+fl6Eqoz4WTOGQsktMPqtTRObMaGR4mKyx8x5O/GGH
PWqMCFnCSyMNaxQLcR4JpTADikMbtDOmZ6ViBZlOVPHQiVBzP6q2n/U9Hxdr
Fis+sawzopxUxBNNrISsUyyhDClnoG8lTwm2GelYs1j+qRMLpnVkCx5Wjy7w
bo+StYmFZsT0Cop3CGHw0/xKzSrFivbXxaSgOAFyo/DPSMjaxCpRxRM302/w
BXmXCJrwxjysUizBjQxiNzLhqexoes3F+sXi7CgGiDccswirLlNrHg5ArAKo
nrCGgmZWvBtvpGL1YnlxYFqH5cFue8OA0kjCSsRqFkH2qpxZUFP5A0ysOVmv
WLqvTjIXmjnLshKxIkTECmYNySfJJd7hUbNesWLIWUITa0EOQSxZq6NVZtRi
HIxYmAx5NGhiLcU6xWqo5bnVrroOsVPMuaQcjFjVZuGTibUg6xSrCb3iSi3I
MubjsMRqabCbWPNy2GIZi3EwYsUnmc2ypTgQsZpWL5hYS3EwYk1zjDEVKxNr
u6VBhtiKhqVYu1gdtphMS7EysQLam6DGYqxdLIGJlQ8HJZaRDyaWkYQlxSor
ceMwMbGMJFgqNJJgYhlJsFRoJGEJsZxQ+4tlambLkmLtD1zJmqN5sd4aS+lp
YuXFcmLtG7csDWbNesUysmYxsfp4Ze6tFxPLSMJ6i3cja7KosbYWmg6ODMUy
yw6BpVNhbRHK1FMs8y9rlherFGSEJSZW1swuVuHDdgvThSIlDryQkS2zilVL
JcVS+4ONcYO2VvDnzYxibSsZqoc2LTrF6jjfyIBZxaI6WnW71XklI3PmFgtq
9Va32t0xs7Jn3uJ9u5W10TZSKlUbTKyVM/eokGOWz4rxCt5YN/OIBcM9LxMy
6k0YOTOnWNsYvVKfsTqSi+WNiTrlA9bW/Dos5hArkvvCqGViHRZzpEJ2JzCK
fLyKNt2N1TLnqLAlZNV1vRTLRosrZr4ay4slBMNWlol1QMwgFojTKNZ0N0cb
eZBMLN9hIF9LYWHVVMBvTa3DIKVYLFVDX5S3Edjmz++DeZgpicQCVfzr5jgl
lPPn9nonJlamJBRLWIKpUIYuFI7AMriSsT4mFksuXWhrMGib3NlbaD7oKxrr
IaFYBOuQG+xSCdD75HbHr9r6K408SNtu2MqyXNdVald9Cj5Q+CL8HSP+ACM1
ScTy0QY1UlGKfJQK45gxO3cfPtv951/+2z/se8FkYmH1DaV8w+hQ5Et5qVFv
0BiIFAsdG8m0YsGMn/fIb+Z6i0SBRfy/YFbHxJqJvMVSRRRHLgxKwfShLuF5
66j3ZvTi/vzk5LR+fP+FFOvu0cnJ48ti7/Xp3Yffnpw8eOUP7MuUYqkqKkhx
Pn7J9aQkxYOr8IXJPJuW+/NTevPZk/KRrh+8CiPW7c6i+/Mnd4/ef1Ec5A7s
+wumFYuae+xCLD0gFDlSNbJIiGV6TYOzqNCnMCwUq9Du7qMXd4+elK/dgX1/
wcQRi0AN4Q0s5gt6CWiVaGbpY7dwDWMvbuusdn1S8jhSY12fFv+VT3dCuQP7
/oJEYoFZfieGI9GtckeFwUrV8ubVZHix6uQWEevuo386f8Ji9c+CJdOJJWsk
CrvqkbyI2RHX1nAdNurtGZ34VPjeM/dai3V//rcf7ar6MhV+9MId2JdpxUJr
VNTBakrKRpzjlHdGMlw9fn++i0Q7aaRYVS11XQwbuXivDuz7C6YfFRIJeeod
pEIakXyKV4ioZapNDbYbdrqozvtl5VRZt38rDuxLgnYDhyBVrUdLsCBrbsUL
PG3UWzVGs0uA43ulk6dC2X2XTQMlDsalQKx6IxfsJtbcXBdDwHzEIqmH7Ibq
YwnUEVmU5P/MqjkoOu4FZca7e1SOA7MRS0cjbHwq64RIcBXSKm31UUb2TNxu
kNnLpz8cGnJDQaQ7F7RAuzDEmVqrIUnEchu45clioYF8TLNY/hQTa1VMOCqU
YnFwgqynWw4Q5Kpz4ER5hg0MV0aiPhYU3aKWl7rpylxs5Q3ombESkvSx5HhP
WKUlUnW6inhE8qf1HFZDSrF8vpO1E5jDDoqruN2BRzpmmWfZ0iHW6787O3ta
Pvvh7Ozsly9bz1LFEHYZGiZ0Yv6obKl+g9xiYmVLu1hvv35Or3/7vHj6/dOO
s3wxBEKIENZQd8Egks8QAsrfMuKPNOanXayfP6VaqXd/eN5xFugAVhDbIXIg
ZEPYHYlW/kLGquiusYqotfv51VmdFD/Y0avzHu0tcFNL2LRF3/zl8OJ7/p1G
Azc7Uly3U6x3331ZPBQJkaNW5KxovJLCCLGCw9k+3qjT5AR/sIHc1Ex/5S6x
3n71Jb/wdVabWLo8wgaDOy4S3vgS3KiAhhgRi2lMxc1NMrM6R4VYs7eIpaMQ
b99C2NGpURwvh4fgo/5FxlQ0iVUtc3hSr/WrFmYNpF0s9urnT36kd39sbjdA
AaWKI/dS5T4RriAW8TFEJHcEbSxjP25uGswqTSrv/3pU3HQxvVhF82pXs7/+
4mXx9GM/MIymQv8Emgu+ciJpUTQVurNlSsSOqTzW2JN2sejuF6/uPvrzaQqx
Bp6F7VHdEpXZTgcsp58KW7SNnTHqLRshHWK9+dWz3ZPLJzmIBf/XQw8rzJMy
KHmxxG4o4uEgYzo6xLr/phDrza9fLS2WjCkcpzAr8gFKI19S+W3+CrKqN6aj
sXiHiEXXj/MRS0UcNEkf5G2CuAUXJLKBYDraxbp9UMaq+2/+17Jihc5Amc7z
NpFjYZNoUJBJlZiGNhaPCss7wB4N+Pgix2RiYaLjioqinVM/2OPYpvoN/qWJ
lZhod5T7WGWsul5WLCKVzuBnEJ18W0JspzDCjXp/xuKk+KjIIEb5B9ntQuXk
ONEnRCuy1srUYsnGOqE8JDcHQkGihP5VvaE3JmEeJBwV+oTWFMIw52F7QrcX
BrliYuVBIrEgl3EXlAt6PFgVYf5aeFm9ycie6WssEvPNpOKTyHZQffGjuBKJ
gBbTzsiTiT/cNpbyYgtk4lkStmHzIay6TKzsSVxjxcKNXmODx8okCo0Lk2ll
TD4qDKNQ2MQS6VAdHOTTLchmrIYpPxSEYo0sajRGqBU9zP/PxFodk4sV1Oqq
M1ofCwdS7CgQycRaI1N/+xfWSTIZ+v3EEqJYW38q9OXrc8RLY0qudqS4boKv
lYu0pVzi0/0p7RafDv1STojG1FzVTH/lycWCYMS9AiJUhw8VzYXw6LCbZUzK
1VUysxKseScuiSDWiNgEx1NUQ+kYmVhpaBILbv/iL9AZto40jVgoCRbqRCwd
F+tcYHnxIG3qKs2YjKurBrPkQr83v3rmN/YmQfHODzJWRYoplE4eFruINUkn
pl2s6vavF8V3UjzhjX1JMiqM9a3cBvmKtjJOxQoxf5EusUy7gXSIVd9M4T7q
fVmxSEUoPdgLelqQ6LYoncyHvTCxBtIhVnH7lyu2aHGxsOHJpuj8RvIFFFmR
ZkWDMebR3jQW7yJiUfGlTSeni4rlM5ao12VZD0O8sCG69fv5R7xyt2p+f9rF
qm//khv7MrlY1U9RjNc7GppSOBLEcaNoTUTFCl6bakNpaGPJUSFu7E0SsUiK
FavIXf3EBpHveWGl1f89mVijiHZH1e1fuPG053XTiCUKd4xcsB/CFIao8Gy8
rvhVplHGJKixIHvJOEQoFocYjmx8FaI+Ypla+QKK7Cr/8o7X2+77XvuJJdoM
MleBWCJlij5DdABJHduMPGBFLosvky7y6j5iIRhxfMrz+9wmrregIqMw0zWE
p2gXw1ger0j1xeVvPjudTKwgfkEMEwfIzAfVWGfIkhM+RkYosej+/HQPsZQJ
EIVAMq2LOoSrLVl5tanTW6uHD3seaOyJTIVUCHYygVi+eIJhX2MZDvWYuKkV
fPT+dfzeLkysucDivfyO6V3MGikWpiPd19xGn8GZ0BmV3SwOYhGx5DjByIiJ
P3iNn5OyCUaGW7kNOxHbwDb/EP+V+GBkxMQ3rPJTEmpgIsRxIhwvx4awOSjj
R2OJcD6m/qhI8Rplab9BXpkUDA2VsiMxseYjrViwi5Qq0QvAk6ZAFWxqyZQB
ZlZAqk7NtGI1ngD9LGkBFlIqdcZWBEZehWI1v4+HD00tAZYp05JgBalA1O+R
7VGxeEQIBzeJ1fI7Q0wswXabzKzIXOGwswThGxSjvHC7CGD8QvW8tlt5ds+K
yyTqokms+O1fj4olM5elI91rs2SDtO/nLvcTa+uXwqgEyH+K+KswVLnu1Vaf
IX/NULHMNWS7bTArfvvXh//xFb35L8WW2+4Q5BUp7/Gpb/TppK9YbEnwB7nj
ITZtt/4/nuQRlwp/x0CsyELaxQpv//qvz+juN7stl+91f1eFnCu8P3/c6x31
TYVsh8qDmOP8iXpUGK7Riv2ODrRIJhbTIVZw+9efn9D/rZQakAonEUu9beJQ
01Rg8QYIblhs+SMaRgFdmEjNdIgV3P71l7++//u/ZCIW96Q6DsC/MVLxi+TZ
UyxzqovG4j1++9c//c9/+Zu75cVisKZq2AlSbeEMqRls7vNrQazdU9MspF2s
8PavP337eIRYJ47JFvpxPY6DQLmboKTCmstZJlwaUcCXSlX0Pud4iHvVePvX
7XvPBos1iAFisR/+dbxvIP48Lrj4AqPeaS2W5cUmov+y8du/PnpR/Udjxfo/
00UsrJ6ql43hJkiXoBT89UNHh5VUDy0XzkygyF4rSAUqqcE2VYLLEsz3U13t
FR1RDhfLcuGsKEWu/eByyFlxQCy51RfpUHnhPl1ShpX/0LzoxDK3ZgMVKev3
XrdQD6nMsH73vXW/PXK0Tp+4C58OquBNrJlhRXbB6r1ndJlGLJ/YmnyQ1my9
je0X7k2dCU2s2dC3f00sFoaeoAclh4E+CkGswgi3FybW3LAit2V5lUYsfi53
6iNJKOgC3B7FlceK95lBRe7PE9VY0edqEyRL4i31EUOHgiF+WGh6zYNS5HbC
UaGg0QguvSBRYocCaq7RPKz772RizUWgyC5sTTilQ3qZQlQPFbU4g0LdNZVY
xjzEFPnXycUKOqSwM5L8/HPdytoHE2tWUt9M4XrnWMGLndCTCu0RYo2eLiS3
usHMCtnsSHHd5GJBw8H1EqJHqYkdCtPfnmKR1Vchm5rprwxr3lMsmxE2tZgR
hDSssibJg5YLI2w2ycwSfax+d1LQuFEhBqUgHAmb9HyhLL0G/2qnlIkV0CRW
x+1fxdYuV4Qil33dGimWXJsQ1k1ykhDi1d5i4YPh2GwazGq//at4Xd9g0YxS
pEiIU/exwuCEgUsP+ziwxVqmk+RFo6ZdrMbbv27LyNWhSajI6A9ea0LmQClW
vL+F5ZZu3I8Uy6JVhA6xWm7/clGsmTBivddxRuSsdlpaBbGRIE4wqv7X3sNC
Q9AhVsvtX9133ai5wj5W0ZRiySdcrKuSbG+xjBiNxXvH7V9vPuu8mwtHhT2t
ohHF+zZIa00HbVkv2CEfx+AGhuOvcIC0i9V4+9fdo+46PHkfq6avWIT/8Y79
I5ZNF8ZoaGO13/7Vx6sZpnQqOp1wCROschPS0D4delXAVmRFiXZH22//uj45
6V4EM5NY3Y0oaFlxRqx2NHo1WKwB79jYj/BO6D4fkTVOrH7pzLdIvU+NXg3C
xJqVUJHbHjX8uBqrnx++dIcTphkOmlozElGkx7L3oWLhTE10t2hIcO8h8nQc
9Y32dgvYbEQUme7bvzyqM6Ud0Z0ubopyNpxELFuUNRuziCVL8YbMJjOfap3q
TuoYzKg5mVusWLMTJp2JTXIFfHipkdhivzmZp8YiTHV6GlDMGOJIMCbWHtQ3
6ZhYszDXqBAmdbYolF7UEGZDYeDw34tYhTUfYR9r8tUNHj1Ps93K/BifIVSt
+D2ndcyrmZhrSqdApjacaubnaBo6tG/1Dh+TNep8YyDzihUmPzFF6M1Ti2Ym
qLRMrAZS/YvMKVbYjhIVuqrCRPzaRqPXUEwsTbrPOpxVrDDybGVs8m0H/j4L
N7djYiXg4cNkZs0nVqR5LqYF62Mi04STtRzss0gVTWJ13P5122OpwnzrsXzo
kfX7NrCLgqy4N/zRydbJAh4+bDCr/favYuN1V7NzzoV+2602i5RDbB/hxika
WDZXGNIuVvO3f1GPD3qfLRVCryHoJXipSKnHik0SuSwNSjrEarv9a9+I9far
s09+VM+6z4qhm5uYEHEtX1jHy5Hj8N9c4JKgqYV0iNV8+9fdo842ersi7757
Sj98Kp91n9WIjlLVTymWMxCnDKWEY3jIq7FMLKaxeO/69q/BN6wq3v7+Jb3+
4qV41n1WI3GxeByIg0J5xjQdUle5m1017WK1fPvXmFvskde/+5Hefv1cPPtg
xx6dd36Bm3FvUN/j0aMqef53e+i/DMwW0BQ0NGDab//aCbdvxPr5E6cTP+s+
q4lQLN0ddWkRDmm7Rl+EWA9NLCRadXZ8+9d1j6UKwyNW91lNxAMR1Odu1hD2
yifxqwyBxbJKPilz1ljNmRCbVT4f6lg2wZQ0Bn4TKyldo8Iv/ajwy71HhXIY
2HDIFp7oVvwEjVJRU5hX6ejVxypC1b59LMLEF2tc1S/EVlHP62PH4HIgvzLS
MPPqBj+q46FfXBZUSi2v2QcLWXMxt1gEJkEU8vWVbJQS75cfC9gbpU4glpmV
iLnXY4V5D3/EmwmyuB8Gdhqqn1iym1jJmHcFaTj13Fht8SEuDYpR43Dcl1PY
WHAWZharJMiFqg3BeVGX76MDV4UTS24yUrCAWLqMcuWTz4m+WQqRzDvYfuXO
X64jlomViEXEUjGJKAhFol/au4M1QiwjEUukQp3s6i2q146HyubDeGxFVsDF
jhTXXUasEmGJqswjZVhwyghMK8VFzfRXzkss30EVY0BokvYotFqwxX6Ki4tk
Zi0oVqQm4taCmqXGJtdeYlUPJlZFg1j1R7M/eBU9qfM+ipJFxWKw6CJlnOpj
uQOG62WLsBQXF40hq8WelYjF0zwoi5hyljvdrjFi2c1fgj5i3X/z7cnJ49vy
rtXyewrffJbmg7X3OEsiqnK/oX4id6oANrr7biFL0kus89PyHui7+nsKqzWk
3SwilpwjDFKcaEJgjeVSZXxKsQcWsgT9Itaz8n/1Ivc3n2cvlnwFxmzFRpwr
3O4pllXtiuZRYWlPcdOXEOuy+NCGjMVyYLcqmDUkH7+YPSYKLQ1G6BCrAMSq
b9tZl1jVz3CZg+w0xGahe0YvkypGYxsrJlb5jXIfPstcrHjRDhEKjoyJBRbu
140/chq6o9Ea6/rk5K/+85M+3+2cl1jVCy7VcRN8BlIwnDQyZPE+VoXsUcEX
2fOcjiy1BjQdLAkuQUZiYTDisaGfO/R+kWhGdGNiLcGiK0jlRg5DweAQP5lt
gq/xNdKTkVgUy2+yl6pHiWZXtmSRCqHNoFURXXe14MHEypgs+lgglC6hZMEe
qmVkSm5iwXYsqXhUSLArMkV9cKRZOJyexVMh1FUsEWGoko1T3kcmVsbkIBaU
4qKzDk0tEpoFo8bp3k52mFhjCBcriJBEMh+G2/GcwyQy4bIO1XIQSz/H6CSy
IcQ2WcofLuFU3sR3Pvy0Y8rrORZPhR4Y+slYJEssJ1yCd5AnyqNpb6n5qWa6
KzryEksmvmorq8TBa9icziGRxqsEZmUglkxukcJcTEqjTccn1sT3ADaJVX+X
jl94VaybGciSYnFTSoagoCgH546ltmoglVfKrMMQS4QsIjlhE5Tu8SlFos1m
kveUOzOLdfeL/1F83vtOrPvzB/9cvaDyBrDiK8GKb2+69ZslOYhVv9j6CCam
nrkLKt1TFztAsWKdhrnFevS4+OaTnViXj6l+QZflw/Vjut75dP3YbZZkUGNV
oFjYGsWVNC5cHclMYbyFNaFZPcQqlrl/9OL+m//wuNpa3F74+YtyBfwvXv3p
2+ImnnqzunaGYkW6DG4v97GmfwMrYd7i3Yl1/p9+/cqJVTzsfHrz+T9+/peP
/vHzF1mLJZsMqtMu6nblFLw8wGTIeJ0mbY82iVXdnVp/w2oZsZ7tMp+MWHT5
t399/823p5S5WLL1LseHW0yG4Yk1xyHWtDS1sS5Pi5vrvUtF8c45r6qx6Pak
qrJWIRYWUgSbVKiKpcLNTqyDdisNDd3Ry+rL61ms4gtWa4PqjwUpx4Tvv8hb
LNly0BLpTU1ibUysbMhMLDEVuOWN6tB48X4sWq1ieUMWYsmFDf4Be1jRoyVH
Ea+qhsMKzMplaTJhkMLee0fFDhyDV1W0MrE6aBQLDlAN+Zb+6FGItRoySoWy
ROcJHdEjjWfHikPPhWsIVJ4sxCrgORuYKSTMjLrfEJkuNLGyISOx/CNOQZNU
DI8+/HmdVN8aMQdZiCVmbvw2iosV8elQI5WJtRfQFdU5T8xGE9RaMmBVYh2i
XkKsVVmWhVgiAfIKBlesyy0sorrOikusq6v4dvlBjibWcESFBRpxgoyEMsV6
xbrqJ9aqyEQshNvufvI5zH49WYVrjV6FS0bXI1kGYkGFTuiSf7XHitG1iNU3
YplYA4iIRbJuV82t/qzCqyFi6d2p3tL+ZCBWSUQZF7g6xHLyxEqsFYh11awV
02CQidUXP6sDhTvP4sg5HxbKPQYWrcCrSixpFk40Vw85G9RARmI5jbiowvzI
rtVoa9YZsEo6xVohGYgl1zjAihk4AoeIDTRZtIJ1pW3JcKXdhpzEcmkvNjdI
YFd4hVZzsl2yDDYps+Q4cC0rsCQZiFXjnXGPbAP22d0TdCXujd+arVisU7NY
TVtyJyex6odSLHHLTSwHdt+TI8TK0azGmZwYa+vAZyCWDFS1Q96F4sF3tnb/
wWCwry0ZitXaYggMKpwysQYjxn7k5al/6ITo9m3EfYQN4rhlD9mZBT2GULGo
WGsjA7EqeBqntoB/4CGwQqaXWO4imYlFgVgdIcyptRrFshNru/WhSoaZzaYS
K/CtDcinGbGTqFWsyOfZolirSIw5iCUKcymWGPqJYDWIzO6+v+J2OwwNWa3I
JyW76r2ePHQvZnvLg8lOLJ+5WKww7AytmXIUyz31DxCzIsbw94JfEPzMVq0c
xCrBuZq6MK912PgaCYaKOXkyAj0/2LYiyz3GyTVu5SpW9QSjFxE/9E+GuQrY
Y0lDxQW33iFOgVcmVgdyglkuXRAF16D6PT+xXG3Vdoya0/GhKRTrItdmRF5i
bblRtXFZcMO46EWUozE96ROqYGkD1+0E1Ts/NbG6cWKhR0KscKhYslrH2nEl
OrxS9XvGXmUlFvnZZ5H4XAXvNsMgcX3Bq/f0oLcKOlcRsbIdGGYlVg1r5QIW
xC6su7qVGtX1GsSQiWR3eHfl7vUhKRdpm7JVK1uxNhynqm0iR9aH8RmNl0uI
NKEAABAcSURBVKLEUzqiA9UpjO+Kth98AWLhbI6MWXyo258ReYglbpfnNtZG
iUU+ctWHwkkdv2GGqFVO1PQ6MnwucblOLVIO8mG1mYv4/f6KaclVLG69Y8rz
qxW4weXOmfgdjUCLEojTFac8KAreYB/xKpYesyAPsQQbZRRELZn8QKyU72cY
kBa1SFGx4joEYvnxnxMoGsem/EP2JF+xQp+8bgTZsNupWbTTPsXEikarNhvE
rLNoacFuMrH6IdpYFCzW22ygHhfDxOYLJny3jlCstoP808Z4pTqixHLhiBAD
V15e5SmWD04yDWJ4crvr50sRStSz/dAoFnYW6gJKrfQTYvltmXmVp1iigBf9
UYo0RZessJRGA5taATwa9CphawEOEw8ZZsIMxSLQaKPF4pUOeDQtX79jg2o8
3KNSwUiELSWRidWBan5yoYWTO1jVu63wELtqivcaECnVKfI6Ag7yvB5BMIJi
q1GsSf6QqchQrCD1+Yik4xhRd1s9pVhCm6tqJXvLEXECsYKmqDtODA/hIx6y
NCsXsSDHOZGwqvLBi4eEYqHy5O9H0FSiB6tAYx71yo4+A0a6C3p5A2HhhYNF
EyuC7qJzc10uIRXBjM9NS1+xoiVWf7FEhQUKKbN0kZVhuKJ8xPJwAvSBCYqr
sO8wb93enupG1u5qdllkQN1okImSWL7c3MpDrNAN0Eh2IMKuQzBMTIiKVPpT
09rFav5ASO2GGBTCYWJZH+vn0uigPyUxuYilZwFBKFG5+8YotrMaxErmG9+7
1euDHvVpMUR1FWx2zwmKKl9XaeHyoEOR1393dva0fPbD2dnZL1/2O2sEkell
yHnYaWCruq41pVgRKWqp9m2KVjSIVe+SdRTaBDcZ5rVwpl2Rt18/p9e/fV48
/f5p77P2Qvay+JkvudQkIp6ZaoonPt7r8aEL8vDIkTgGhMq83seLGmAv+Z2i
wvIbM6FdkZ8/pVqpd3943vus8QTjP8iCfABMJ6pQl6gJ3+COj1jNHxwTHK24
uABtLrxD9T7RqxKTzrLRlVt9VdCtSBG1dj+/OquT4gc7EokFnU/sObBZvtO1
gU3hVeTjOHoEIt9td271mJKOLseCMeGF2kWEFbyq5P36h6xiVUWnIu+++7J4
KBIiR630YtWvYQe02L1YrW33/VpdXbnvypdYtVhRE5vFEmFJllE6r3GKxEqe
4Pyhf9wMNCvy/dnZp0Wk+hI2uTorWY3lUx0r5X6KKUMKZOu87hDqEKRsESv4
rjhe9V9zDKjqGwslLxbELDhWXybDPEg9RoVYs6cVS3amZNKLlFLE+XJCxH0R
cmaZzYGtLR/M1wEWVuiW6KnLn2Fj4WKlYrFXP3/yI737Y7p2A4melCi1uMew
gXBFWNNPR2yWRqVAIrAuErB6KoajQCkWlFR+UEjqUBLzzxP84VPTrkjRvNrV
7K+/eFk8/dgPDNMv9FOV/IbFIj3tE73GVJEstl6Bc6F7KU/oOznIiQycEkv7
QCx3Eh4N27Ijj847A1M2kdkcFaqwC6GukqTvzkUWgVL44Xy9G/FQUpEXDGKR
SIgXKFZYamVpVo5igRfCKikatYiVAsh5MKUDDfiBEzwQfUhnuWq/PBIOgSEl
DCPzIjexRI8KR4k4q+Nilwhi/vxEbwyqdO/QFdLvXmi4jwKbmzGxREjjNMhX
wmorNzITSwYokEgMFkPL+og17dShilO649CyMgvEQl9U9eSfYVwjJVG2pXuu
Yvnnssbym7A9Gs77NF57v/cGi2UILfGRK3qwPMWPJEXBDn4IsXDkx20H+dzE
6sdG+SHqKu0a8UpArOhTvC+f5ZxGMD4Mls/oSot3cpf+QoiFxTxML4Nicl7Q
nZNlFizJXSy/EfaKEBa5AiWwy7vB8zeYArngwgZErHnqzhJpsNjhIhEMEzEc
ef/IN0vRuuzITSzyM83+Fc7mxGowUkfj4yTI2R0ZpQKaVjvIigxaB6Jsgqk/
dg4UFKHMaqyeKCtcEeXznch1XGSppTPh9fZFN9VlUcXRingnHs9zPz6ycWlO
OujgHtykcmPs1KzIUyxZl/sAFW9wCYNkcJvwzUlHomFKLHbQJ+H84xX5AR9W
7q7zyf346lfD8JCEZrlKVZCTWI64WBjCmpdkqWJsmjckQpIsqa7kCyjs4WCc
Z/TNBqofMXSJPrw4xh/a0NLKjhzFKpG2BMNBzI6RRhYfMwURsUTHHWerpVhX
IJpodUERDm0F9wu1MiiWblBkSp5iyWgTq9hFGV8d5I8WL2maWgvaDTJOYVnu
DyWZNWG7aESIzgIX7t4ZTn4U1FMm1hhkFpOpUSrWUUw1pcreoBU6A3K0Iqir
QKn6ZWSxYBNilHgBsck/6INzJU+xSnzowfzHQUqLFe1+RTYOexORFpQ0i8ur
xlN116ELGPGBWG4XNt4pX7vyFUvGJ7W0wUuFx4cXmGpY6DpX/kkIEa+ngfBF
OmTF8HKo2UFoxYsDUCkTawAi3clXYb3VEIRS9BrEsFAPDN0RKlH2CVJKrOrZ
BTzzu7Fqd92JPMlZLNH4jAQrqVhwDfmImxsaqm3IilzUXCS7oqIa65X6kCAM
iT4XDyFh6nDYL5iLLMWKxCJOiNoxGcDghOhjVKweemEXQc7O4FhRHu2O7Q+0
SgkmDKM1FXRScyRPsSh0JZ4HN6KdhTJqx+LlGIvltjWYEHQNMBnK1X9w+Lh4
xa90y0rNPOfczcpXLNU9F05x5IJGPJyoL9bye/SzqAuy3ymClQ5asLxmuFgM
jAh9v0GuJs28S5qvWPwpDXITsVh8hGiPBs14vG7Dr2t/OygWoVZ+L0jWPQqM
EOm1uyTIGVG2IHLWKlux1BIHEb0gdhHHNmx7NS/KGikWkYxDYpqZY9PIBFiA
E85+k2+543ShDlnDf9cs5CoWC+LFwg78BgRC0WKZUF93HL6SCuZpXJV1BXlR
ntcbYYoOTjozmlhjwFxXvQ6qdheUZKsLrxC/7ji7QCweA8r2lVqMxef1JtCk
0SzQa/jfMge5ilUANVYwBsQQxfvxXFGXy7ZC7yaDIz6xIxf+8chw3F8bpy1s
ZR2ychYrEqZwuOhCFcQseSo8i6m06WmWanPK3qco2mWXlMZVWxTrj+r8l7lV
lL1YhHW7TISR3AgBSb5quXw3anUCF1N+ngebWuKc4WLpieXGKJW1VZStWNi4
hMAEcUoHLljEJat4Hab2e2PYxFIz0tBEHU/YdmjKhbGjMyJjsVRgIrVF1OHy
wR8bXjWytQXRVpBzzO61nHTeW61IMEK1VDfexBqDTHMUrPSDIR4kyw1sDK9I
g0orIZZaMOoOwZrranRVJagN4pduI2fB7DNhxmLJ9OerLbcL0mQY3DAuiSZE
XLgo2PLk6BRM66hyazJwGSkFRTxlHa0KshZL9ELrjeQ3ybQI+kXEao5i7YBO
uEAmEKs6dtpOQ4Wv1PFusel/zeTkK1aBjz/1KxgjRuuujT7GX4ekVIMEwyHe
lcaL5Quvvf/qELCKso9VFdmKBX0qFkv2FtAtSJoyS/qriSccvXooFvYb4gQt
r8lZR7AqyVgsKMfBn87RIm8WF/PPYKPa0ITIcVeROsuFrrRWFZhY+yOSGokR
X2iQ2BfUWFDAj+tjhTPLcbEMT75ikQs0MbEIrfIH+0lrvy8wKp4gBxHU71xl
jbncgZKxWCoFytwIYgUhLSypxAvZAhuOidWHzMXa6CzXXF+FvYmm6ZyhfQdV
vFM0FY79Kw+VjMWS889yUFjtxWNQML9XHcuXHfQ2QrFgFbKJFSdzsQjEEkGL
9QnDGKS5ScSKcAU9URMrRtZilQhNdPMT9HEblV7cB9OF/T5A3ErfvVoluYsF
BRZvkKWUM0/0ImSNzreMja3YY/hAZVqF5C8WQZfU96R4gzePj8fsiflz6vfm
fTKxQnIXC/ukMvZIX1AbzH91ReU2DLDLbNmLNYmlu1jcPlWZ0TlYvxZ1fe8G
vIm1F9mLVcCxiQus0DQ4mCCYSRFV5jRSsQqxfCZr6l2p+8R4fodPxp7FzO/+
KFmTWD4uhVZhqe5LehZpg1tMrDlYi1hicEjCKT4GsqEosSg82kjMOsQSwUc2
rIIgRlCTcZ40rWZmJWJB5IFMB9M7bWJZvFqA9YklQhUbA7bJk0QHy9yai9WI
VRIJWpvgKYe2sBlhYs3FusQqgcAk6nJ0auMPCNwyuWZhZWLJ7rmKSyTWOaiI
xfnQxJqDVYkFYSecqvGP0XLMSqyZWZlY4gUEJQrE0nM5fmmEMQurEsuhJglF
qsM+RJgHTay5WKFYmPwwXrWz5Ds+RlYpFvHQsIdSVrAvwQrF4naWiZUvaxWL
m+o9vDKt5meVYpFcGQMOhU+rgxd+t0fIusUKm6DBGhmTahnWKhapRlYkA5pY
S7JqseqHptLKWJAVi1Wjs6BbDmgsyurFcsM+OQg0sZbmAMTCp/zK1FqW1Ysl
sbWiuXCwYhnLcmBiGblgYhlJOBixLAHmhYllJOFgxDLywsQykmBiGUkwsYwk
mFhGEkwsIwkmlpEEE8tIgollJMHEMpJgYhlJMLGMJJhYRhJMLCMJJpaRBBPL
SEKHIj+cnZ398mXx7O1XZ5/82PMsw+hQ5Pun9ZN33z2lHz7teZZhtCvy7g/P
62dvf/+SXn/xstdZhtGhyC7/nZ2VQev1736kt18Xmn2ww8QyOmhX5PVvn9dR
6+dPnFjdZxlGiyLfn51VNVVZZ3HEKs9q4YO2nXNib6QkvUNRevzeUixZY7Xx
wb5vaSrsjSxJu1hFAnz3x0Knd999CaPCNrL5Z7Q3siTdfayPn5ehSvSx2sjm
n9HeyJJYGW4kwcQykmBiGUkwsYwkmFhGEiYXi9dDLErvQWxqMvn3mJ3JxfLr
IRZFLsZYkjz+PeZnarF4PcSi9J8oSEwm/x7zM7VYvB5iUeTU5oJk8u8xP1OL
xeshFkUuxliQTP495mdKseR6iEXJJmKVLP/vMT9J2g3L/0NmU2OVLP/vMT9T
i8XrIRal/2KMxGTy7zE/KfpYH2eQgnLqY+Xw7zE71nk3kmBiGUkwsYwkmFhG
EkwsIwkmlpGE4xPrzWdPiofb91/cPXr/Rbnhwavdw/XJ493P+/PyZrz3nvnD
f/Ws4UJGG8ctVunP3aNCrPvz3xSa3Z8XetF1qVzB5Xsm1hiOW6zfFBZd/6YQ
6/b9vzx64sWqD9o9nphYozhusf7br1/R/TffFmJdPvj/56eBWLendx+aWGM4
brH++9+/oLtf/L+dWG8+e1zmv1qsS58KTaxxHLdYT66f0O3j251Yu5e7YuuJ
K969VybWSI5crNtTunxSiHVZpsFTF7EYE2scxyfW/Xkp1vWDVzux3vz6Xz5/
sRNrV6RXgcrEmojjE4suqyrqtEx9f/r2lHZiVe2F3QYTayKOUKzbkyelQ+X/
irbobTUgLHOhiTURRyjWTqiqtV7KtfPm9sE/P6q6C9fv/W8TaxqOUSxjBkws
IwkmlpEEE8tIgollJMHEMpJgYhlJMLGMJJhYRhJMLCMJ/w4WjGumsl2LhgAA
AABJRU5ErkJggg==
"" alt="A plot showing the identified clusters, now each coloured by the assigned cell type." width="600" height="540" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 11</strong>:</span> Our annotated dataset.</figcaption></figure>
<h2 id="unsupervised-approach">Unsupervised approach</h2>
<p>But what if we don’t have any reference that we can use to assign our clusters? In that case, we will turn to the mentioned unsupervised approach - we will check what are the specifically expressed genes for each cluster. Then we can identify the cell types by looking up what cell types contain those genes. That’s a more tedious process, but sometimes can lead to exciting and unexpected results.
We will use Monocle’s function <code style="color: inherit">top_markers()</code> and store the information about specifically expressed genes for each cluster in the data frame <code style="color: inherit">marker_test</code>.</p>


In [ ]:
# find top marker genes in each cluster
marker_test <- top_markers(cds_clustered, group_cells_by="cluster", reduction_method = "UMAP", reference_cells=1000, cores=8)

<p>You can group the cells by any categorical variable in <code style="color: inherit">colData(cds_clustered)</code>. The parameter <code style="color: inherit">reference_cells</code> is used to accelerate the marker significance test at some cost in sensitivity. It works by randomly selecting a specified number of cells and performing the marker significance test against a chosen set of cells. If your dataset is not massively big, you might skip this parameter as it wouldn’t help much.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-3"><i class="far fa-question-circle" aria-hidden="true"></i> Question</div>
<p>What are the variables stored in <code style="color: inherit">marker_test</code> data frame and what do they mean?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-4"></div>
<p>Those are 10 variables that you can check easily either using colnames(marker_test) or <code style="color: inherit">View(marker_test)</code> which will also display all the corresponding values.</p>
<ul>
<li><code style="color: inherit">gene_id</code> - Ensembl gene ID</li>
<li><code style="color: inherit">gene_short_name</code> - short name of the gene corresponding to its ID</li>
<li><code style="color: inherit">cell_group</code> - a group to which the cell belongs, specified in the <code style="color: inherit">group_cells_by</code> argument</li>
<li><code style="color: inherit">marker_score</code> - numeric vector of marker scores as the fraction expressing scaled by the specificity. The value ranges from 0 to 1</li>
<li><code style="color: inherit">mean_expression</code> -  numeric vector of mean normalized expression of the gene in the cell group</li>
<li><code style="color: inherit">fraction_expressing</code> - numeric vector of fraction of cells expressing the gene within the cell group</li>
<li><code style="color: inherit">specificity</code> - numeric vector of a measure of how specific the gene’s expression is to the cell group based on the Jensen-Shannon divergence. The value ranges from 0 to 1.</li>
<li><code style="color: inherit">pseudo_R2</code> - numeric vector of pseudo R-squared values, a measure of how well the gene expression model fits the categorical data relative to the null model. The value ranges from 0 to 1.</li>
<li><code style="color: inherit">marker_test_p_value</code> - numeric vector of likelihood ratio p-values; p-value is an area in the tail of a distribution that tells you the odds of a result happening by chance</li>
<li><code style="color: inherit">marker_test_q_value</code> -  numeric vector of likelihood ratio q-values; q-value is a p-value that has been adjusted for the False Discovery Rate (FDR)</li>
</ul>
</details>
</blockquote>
<p>We can now use data in <code style="color: inherit">marker_test</code> to rank the cells based on one of the specificity metrics and take the top gene(s) for each cluster. We will filter the expressing cells that constitute more than 10% of the cell group and we will take one gene in each cluster with the highest <code style="color: inherit">pseudo_R2</code> value (you can of course modify this value and choose more genes to be selected).</p>


In [ ]:
# filter the ‘marker_test’ data frame
top_specific_markers <- marker_test %>%
                            dplyr::filter(fraction_expressing >= 0.10) %>%       # set the fraction of expressing cells
                            dplyr::group_by(cell_group) %>%                      # set a group to which the cells belong
                            dplyr::top_n(1, pseudo_R2)                           # set the number of top genes and the variable from 'marker_test' to rank by

In [ ]:
# store the names of the marker genes
# you can also use IDs, the conversion to gene names should happen automatically when plotting
top_marker_names <- unique(top_specific_markers %>% dplyr::pull(gene_short_name))

<p>Now we have all elements to plot the expression and fraction of cells that express found markers in each group.</p>


In [ ]:
plot_genes_by_group(cds_clustered,                    # our CDS object
                    top_marker_names,                 # a list of gene names to show in the plot
                    group_cells_by="cluster",         # how to group cells when labeling
                    ordering_type="maximal_on_diag")  # how to order the genes / groups on the dot plot

<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div style="font-weight:900;font-size: 125%">💡 </button></div>
<p>If you notice that on your dot plot any cluster has more genes than you specified (here we set one gene per cluster <code style="color: inherit">top_n(1, pseudo_R2)</code>), go back to the code and create a new cell, right after assigning <code style="color: inherit">top_specific_markers</code> object and just before <code style="color: inherit">top_marker_names</code>. In the new cell, paste and execute the following:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">top_specific_markers &lt;- top_specific_markers %&gt;%
                           dplyr::distinct(pseudo_R2, .keep_all=TRUE)     # select only one row if there are multiple rows with the same value in 'pseudo_R2' column
</code></pre></div>  </div>
<p>Then you can execute <code style="color: inherit">top_marker_names</code> again and see how the plot looks like now. Better, right?
This problem may arise when several genes in one cluster have the same values of specific variable from ‘marker_test’ (in our case we chose <code style="color: inherit">pseudo_R2</code>). It might likely happen in small and quite unsignificant clusters.</p>
</blockquote>
<figure id="figure-12"><img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAArwAAAImCAIAAAA+Jc4xAAAACXBIWXMAAA7D
AAAOwwHHb6hkAAAgAElEQVR42u3da6x060HQ8V28kBQvgSiKKKVwToEDYuEt
RV6gpXLOe47oJohtgEILFNiUUAbk0mDFQmlpT3t6xo2JRhKCiolRJxgSyY4h
mqAGM6Yh3qKiW1FhC37yk1/8QMaZPZf1rOteM2suz/PM758n58xec9l71p49
6/c+a83MxUySJKlHF1aBJEmCBkmSBA2SJAkaJEkSNEiSJGiQJEnQkP9auLAe
JEmCBmiQJAkaoEGSJGiABkmSoAEaJEmCBmiQJAkaoEGSJGiwFqBBkmaz5z/x
2+vDahE0QIMkdVmBHgQN0CBJu4iBGwQN0CAJF7YeVh00QIMkEQM3CBqgQZL2
JwZugAZokCRogAZBAzRI0l7FwA3QAA2SRAzcIGiABkmCBkEDNEgSNCgLNNxN
RpelRpO7nlcbT4v/D/8hOm4FGiQRAzcoEjSE2+vpuBcb9omGpVugQRI0QIPS
QkP49RwQldmH9ZLReBygYby+XHBD4SVrVw9dMr+Fza3FhIZnX/Hm1B8987vg
XrgX7kJC9wIalBoaFl8ut+jBGevZh2IW4n7rv0ZDsHCNgfIlW29wSYbJXcN8
xaOg46Nh+cziWd69cC/8IqBB0FBBQ6nVxrthOz7f1G+WNO2eKE5XLnkPhKYd
GV3nmWmQdGZBg1Kbaahtz9suuXJBIxpK1w1gUL/BYr7BMQ2SoAEalC4aDjzT
UJvjaD0CExokQQM0KGI0hGesT7Yc01BHw0PHNHQdfQkNkriBGJQYGmZNL3ZY
zw6UXz0xbbip9XXHk0n3qyegQZKgQWmg4ZQo6b0WoEESNECDMkVDeLhCv3eY
hAZJ2rcbrExoOMu1AA2SuIEYBA3QIEnQIGiABkk6jRusQGiABkniBmIQNECD
JA2mg9UFDYIGSdxADIIGaJCkYXqwWgQN0CBJEjRAgyRJ0AANkiRBAzRIkgQN
0CBJEjRAgyRJ0AANknReecmloAEaJGlrK9CDoAEaJGkXMXCDoAEaJOGCD6wS
NECDJB1ADNwADdBgPUgiBm4QNECDJEGDoAEaJOkkYuAGaIAGSSIGbhA0QIMk
QYOgARokCRqUPhruJqPLUqPJXc+rjafF//f0/VtvChokEQM3KAY0hJvq6bgX
G/aGhvn363N9aJAEDdCg2NAQfj3foFdmH9ZLRuNxgIbxuD5REF5yffVgVmF9
yfmiXlMbx0fDs69483yk/uhxL9wLdyGtewENSgoNiy+XW/HgjPXsQzELcU+C
NRqChWsAlC+5PBmcXdz2xiUNOyceBUGDe+FeuAvQAA2KAQ2lVlvuhv0O4Z6E
pt0TJQqEl6zPJawvUBjlgVkHuycknUPQoKRmGsLllS14+ZKrzX4jGkrXLX0R
TCw0fNOO4xugQRI0QIMiRsOeZxoWYGifxoAGSYIGJYqG8Iz1yZZjGupoaDqm
oXrJylESy6VecimJG4hB6aFh1vTqifUREOVXT0wbbmp93fFks3uiOHxic/XS
MRVdL6OABknQAA06NRoO3eA3coAGSdAADcoYDeFLMvq9wyQ0SNIB3GBlQsNZ
rgVokMQNxCBogAZJggZBAzRI0mncYAVCAzRIEjcQg6ABGiRpMB2sLmgQNEji
BmIQNECDJA3Tg9UiaIAGSZKgARokSYIGaJAkCRqgQZIkaIAGSZKgARokSYIG
aJCkvr3wB99ZH1aLoAEaJKnLCvQgaIAGSdpFDNwgaIAGSbiw9bDqBA3QIIkY
uEHQAA2StD8xcIOgARokQQM0CBqgQZL2KgZuEDRAgyRi4Aatu725uYUGaJAE
DdCgB8Rw/fji8TU0QIMkaMgeDc++4i3z4TcODadDw3R8uWk87X+9u8nosnbN
/guDpV3fFRrOuTyeHz3LJyeGON2wfCCFI8mt9fXVxbLShntx3rqrm2LZ1fXq
jOXC4GLrS3Ve/fH1TfkawQVX3z+8aumHuil+0MWNbL5f0/c6IzQsxLDZZi82
46PJXf+r1rf2vRcW37j7ux4fDc++4s3LkfiGKoO7kOyTY773AhqgYRgaypv/
1TY3PB1MBCwvX2yXw4sttujhNVqvvlZAcaHSTEN4O+ENLZavL7TUQ/3Hjm7O
4ggby8X2urQ5ny9Ybb/vT01WkxCLZdPgdPmijdd/YGFp2fymW9VwZDRsxJD0
RjeLu5D482O+9wIaInkspfeIKm1wg01uacMdbMjbLl9p26uHt3N7e1ufC6n9
SMUttX2vc0FDzQyV84KpgPD08lS4V2NzK70XdqLhURA0QAM0QAM05IKGcKu/
2uKW9w8Euwkql2/ZQPe9eiMaZsFUQvtVlztKbjq+1xmhofVf+OF54en1Br60
U2F9gf4LK7snzDQc5o7MUs7uCWiAhqwM2oGGxg1vfzT0uXozGpZeCPdhPISG
iI+hPO5MQ3FY4nL7/RAayu1yfMP6O44m0w68OKbhnHMgZGybqzh/PGhIBw2V
Sf/NnH/Tprhtq19p26uHy+fXrTKmaY/J5su273UuaGjYP7ExweHR8MAFT4YG
SWl1JmhI3qDdB0KW/vHfuB+h7djJflcvvgxEECJg9eMVl2k9ELL+vc4GDcu9
BAECii8fQkMJD/ML3G/1+y8MlnYdWQENkk7hBqv0MGgIXnLZ+prJ6s6C5ott
e/Xgy9UxDJu5g+KK4QxD8JLL69prLKI7nuFoaJhV3kUhfPnlAzMNwfVKKOi5
sDg+svNVntAgCRryQUOK76pU2YsRazaW0CAJGqDhFEooftTKsRjQAA2SuIEY
oKHyw0b6xo/QAA2SonGD1ahUN5dWATRIggYJGqBBUnRusAIFDdAgiRuIQdAA
DZI0mA5Wl6ABGiRxAzEIGqBBkobpwWoRNECDJEnQAA2SJAkaoEGSJGiABkmS
oAEaJEmCBmiQJAkaoEHSefXCJ7+zPqwWQQM0SFKXFehBs9nN1fIzr5s/qbvx
3O6rQAM0SDoDMXDD+XV7/fji6qZ86oFzu68CDdAg6Ty4gA7xbwL2vRW4udps
9hsI0Hhu91WgARoknZkYuOFc0DDf6he7GOYaKO9vaDy3+yrQAA2SzlAM3HAW
lTb6NQE0ntt9FWiABknQoDwz0wANkrQXMXDDeUw1OKYBGiQRAzeo11TDaq6g
7dUTtXO7rwIN0CAJGpTzZEP5TRdKFvA+DdAgCRqyQMOzr3jLfPiN6yhomI4v
N42nW1/9bjIqXW19c6PJXcO3KC65uF79GxdLu34UaDjnnv24tyxH2vfCs3xq
YojTDcsHUjj86nVINCw255vN82KLHW7re4OjdAvLL5pOlYAxv2rNBcVP0/2j
HB8Nz77izcuR+IYq/buwFkPSbsjjKT7yuwAN0t7RUJ0lWCxYbarvT02CWYPa
DML9gtFkGt5GIIHabdcoUUNBadn8plrVcGQ0bMSQ9EY3h7tQFkOibsjjWT7+
u3AOaKg/lrhBh0RD43Y9OC/4V/9lfdqg4UYe2uoHlw13igQTEe1XfxQEDdAA
DdAADdCgo6Oh9R/z4Xnh6RoFKrsc2rb6q4MV1ktKux82t1/ePWGm4TB3JOGf
Pws0zLLYPQENDKrznmkojkBcbqp3QsPD+xeadzoUezXWP8b9bo9Y0DDL5ZiG
DMrjmIZZFgdCOqYBGnRmaGjY2bDZqO+EhgePaWg+/rHtoMjWF1B49QQ3pP7q
CUEDgyo5NCx3CAQIKL7cDQ3FXodiceM+i9LC+WWrF+063AIaJJ3CDVapzh4N
s8obJoSb/x3QMGt8n4bgO5SkUF9YHB/Z+dJPaJAEDdIp0JDiWoAGSdBwrs//
YVYINECDpOjcYGXGxgV0gAZokBSjG6zGmMXADdAADZKgQX3FwA3QAA2SonCD
FXgeaOj8nOvFx2Sv2nxadrAsXAwN0CDpXN1g1aUihmFuWABgudkvTpU8sabE
4uzV6fnSSKUADdAg6ch0sLrOCQ0BAOpqKOtg/dX8co2TEtAADZLOzA1W1Hmh
oQSAYF6hmRfLc9e7M2LeOQEN0CDpcHqwWs4UDSUndKAh2DkRnIx51sHGEhok
CRqOP9OwmFpoo0G0xzfYWEKDJHHDfl890XlMw2phFwugARokSWeChmJnQ4MZ
GsVQmo+YX8JLLqFBkpSIGwZ/h9r7NKz5UH4/huKwx2B5vC+jsLGEBkniBu8F
CQ3QIEnahg5WCDRAgyRJ0AANkiRBAzRIZ9Jr3jeuDOtEggZokPQAF9BBggZo
kLQFF9BBggZokLSdGLhBggZoyKFnH78/HCnehee+6H2V4dcaoRi44ci98fkX
w2GFCBqgYZ9cSJQOdTFwQ7Ri4IaTcOEc6OCdnaABGk4mhlTc0MYFdICGgb3w
1A9tRk5iyNINh38baWiAhj31/Cd9RzhyEkMSboCGFMUQuRtCLqRIhwfFkJkb
Dv+BVdAADYcRQ0Ju6CmGyN3QRwwpuuHJK982H9AQlRhScUNPMWTjhottsn1M
Fg3T8eWm0eSu57XuJqPVdcbTCNDQKIZU3JABGvqLISE3LLkQjlzFEK0boCFX
MXBDqmi43/QXW/2FH3q5YXHB5dUWt9B+leM8LDrEEL8bthJDtG6ABmg4shji
d8NWYkjdDRe7xgppoWG+7a9s8BcGWCNiMwVxf5n5OaPxeHQ/tbA4vble/Uag
ARqSR0NdDNG6ARqgARqg4URmaNTD8mLhpEQnGh4FQQM0QAM0QEPeaLgY1vbf
8OZqec3H17e1826vHwe3fXXT4yrQsA0a2g5ICGccGpaUd0+YaYAGuyegARqg
4RhoWKhgaYHiVBkUtWUPXAUaeleaMHjorAoj1gdCjibT9luBBmiABmg4lBsi
f/xAw2HQEKiggQDzRfXJhO6rQMN2Uw0tNHhgpqHnfIVXT+zfDdHeCy+5TNoN
cd4vr56AhoaJhkIFcw1UiLDeDRHunHjgKtCw5VxDy6snAiMsT1Z3T9Qvd0o0
zLxPQyJumAka9uGGJH4v0LB/NJQ2+jUBLCYSNkvWWOi+CjTsMt3Q/D4NtVdP
hDoonxkDGmbeERIauMHbSKfphqQfezHNNFSBsZhsMNOQ1oPJStiXG1K5I7gA
DdrKDRk8z0dzTEPjRR3TAA3n54a07ggxJOQG6+20bsjmqf5Yr7cs9kA0AKA0
jzA/v3jNROtVoAEaMqODFaLDucEaOyEdMnuqP+KbO9XedCGwQPA+DeFuCO/T
AA2ShrnButJp3WClQQM0SAnQwfrRyd1gdUEDNEix08E6UQxusKKgARokSTNi
gAZokCQNpYPVAg3QIEkSNECDJEnQAA2SJEEDNEiSBA3QIEkSNECDJCmnJ38v
nYAGaJAkbWEFeoAGaJAk7SIGboAGaJAkXPCBVdAADUfvc/7SuD6sFu3Wq37q
I/VhtSgGMXADNEDD/rmADtojF9DhhL32u8aVAQ02ENAADQfhAjpoj1xIjg5f
+rUvVUbqXMiJDheD8zcLDSdDw7Nf8v7NyFIM3KA9iiF+N9S5kBYdurmQAR0u
9pS/XGg4KhpCK9RHZmJIwg1PvvC9leFvIU4xxOyGbjHE74b+YkjXDdAADemh
oVsMMbthZzFE7oa6GLghZjHE6YY+YojZDduKIVE3QAM0JIaGPmKI1g35oaGN
C+gADQcSQ7RuOAc0XOy1bb7zzdXyOo+vb8tn3F4/rtzs1U19+XIZNJwfGvqL
IUI3DBRDnG6AhkTFEJUbthVDhG7YTQzJueFEaFgAoKBABwDmtFirYn4yUilA
AzScLRr6iCFFNzz5hLfPBzRAAzTEgYYAAF1qCMiwuFxtUgIaWpuOLzeNJncN
l7ibjC7H083JZasFyxsovmi/KjQcWQxRuaG/GBJyw5IL4chYDJG4YTcxROWG
IWJIyw2nQUMJAKEMWqYjVheLfudEJGi4N0Bp89/khs2Wf37+6uzFovUlW9Gw
4shp0LCtGKJyAzRAAzRAAzTsgoaSE9rQUF6+EEQCsw4xoKFAQNPMwHoKYjQe
LxfOzywuXlBhcWo8rsw/3F95NJmebKYBGqDhyGKI1g3QAA1mGjrMUD8zzsmG
CNBQN0Pjmffb/+6ZhtXp+s6Ipt0Tj4KgARqgARqgwWENe33pxAPHNHSaARq6
0dA6CxCe1zj9UGgjvGj1Ju/MNEBDnmiY2T0BDdAQIxqKnQ0tZqjtgCghYn6u
l1y2VdrdUDurfqhjdXphs3uiaafFqdEwcyAkNEADNEDD2aFh1vA+DSU+NEw0
BO/TEO/LKCI/pqFppqF08dLui/VFa0aABmjY3g2zpIr/JZc5oWHmJZep5e0g
c0JD16snGo5paJ1pWJ/sd0wDNBzNDdFtX725U8puiOS+QMNZucEfb1xo2Jig
6X0a1u/JsHn1ROnC5V0Sq1dP1PZ2nBQNs5TfRhoaBA37ckOEv5HzEQM0ZIeG
Uz+YDv0tfGBVKnTw5xCzG2K7Oz6w6hzc4M8WGk6AhlmyH42dKxpmPhobGo7l
hph/I+cjhh3o4A8WGk6JhkY9JLF+shSDknNDtPcoXTFs64acnvOJARqSQUOK
EYNO64b471SKXOhPh4yf/FkBGqDh9G6wurRHN1hdJ6SDdSJogIYD0sEq0h7p
YBVJ0AANedLBatEe6WC1SNAADZIkQQM0SJIEDdAgSZJsLKFBkiRogAZJkqAB
GiRJw5/8vb8TNECDJGk7K9ADNECDJGkXMXADNECDJOGCT8eGBmiQJB1ADNwA
DdAgScTADdAADTqz3vj8i5VhnUjQAA3QID3ABXSQDiQGboAGaFC2YuAGae9i
4AZogAZlKwZukKABGqBBxPDiVsMaEzRAAzQkjIbnP/Hbw2H963Bi4AYRwynd
cHv9+OLqpuGMm6vlbT2+vn1gITScNxoqYuCGk/fklW+bD2jQfv/A/VFAw9oA
dTQUlAhQ0bgQGvp0NxldlhpN7tbLx9Mdb6rrikdDQ6MYuOG0z4zhyFIM3ODf
Axn/UUSLhnswPL6+adz+z89cLyyA0LgQGvpu6cNt/HS8ZMPWaFhfcXnV1cmT
oaFDDNzg+REanvudX78c0OCPIpOZhrbN/3xhsQNiboX7LxoXQsNOaFh/HS4P
ZiOCi86VEMxNzC9TQGF+Vis4oGHnnv24t2QjhvifIjNGw0YMibohj7/rFP8o
kkRDiQTrLxoXQsMuaNjMEhTLiymEYGFwtdX5XTMNj4KgYWcxLAc0QAM0QAM0
mGko3/H1ER2L0we9O7VjGlYUaN49sZ5CaD03PCzCTIOZhq6nyMh/YGiABn8U
OaAh+2MaFgd0LJSw+P/9/bgnxOHuUduxC+Xlm10Rlxs0VGQQTi8sLm33hKAh
ATck+nByTMOZuGG7b9+8+S/+6V159URtYZJo2Fhhc6J08hRoCAiwWVi/VrAT
o/LFSdAw8+qJiJ8lU/lpvXQCGvxRpIqGkgVyfp+G+NBQPSJyqYFg6fJkZDMN
+T2zCBrU9gduVUBDnuuzL5VKuyfW+ytONtMQHPYwGo+LS5dfPREu6DyowTtC
KmM3WGPiBmI4KhrCWZPo5052fEh5KChLN1hX0r7cYDXOvI00NChjN1hLEjRA
AzRID7vB+pH26AYrcBs0bN6loVSsrwiBBp03HawTab9usOq2QsOB35UBGiRJ
UdLB6toBDQd9eSU0SJJidIMVtftMAzRIks5BD1bLQDTkP9fgUSJJ0l7QUH6L
BgdCSpIEDee7FqBBkiRogAZJko6Khs07NVzdBJ/fCQ2SJEFD2PrzqTbHQ+b2
zg3QIEnSXtBw9I/GhgZJkqABGiRJyhgNm4MYSnrI6agGaJAkaT9omM2qb9aQ
13GQ0CBJ0h7RkPdagAZJkqABGpR3j97xcn1YLZKgARqkB7iADpJOiIbiZRKl
oxry+vgqaFB+YuAGScdFw/17OG2Oecz2BZfQoBy5gA6SjoqGCg3CLzN7J2lo
EDRI0hA0VKcTSl97cycpETFwg6QToKH/mdDQ3fO/91s2w0NQRxADN0g6NBq6
dkHYPbEXMXCDoCGznv/971gOq0LnhoaOz7I8xTzD3WR0OZ6GJ1anV60XRY2G
uhiSdsNzv/sb5iP1v4EnH//W+cheDPG74ckr37YcqXMhHP4odEZoWOmgMqVw
T4kT7JpoQMN0fDma3G3OXZ+OFQ1tYkjUDUsxpO6G5ZNjEk+ReaNhI4Z03ZAN
GhL6o1B0aCiUcPLPnaijYW6GYHph9dX83NF4PFpPPfSbioCGIWJI1w3hk2P8
T5EZo6EihhTd0CiGFN2Q1h+FokRDJDXunpiFZrifabhXwvrsylRE6VqPgqAB
GqABGqABGnQmaAh2TjSTojYvYaZhr25I9w/A7glogIZ0/ygEDTugYa6B4HCG
GhoWZz+0g8KBkEPckPrfgAMhI3RDig8kB0IKGqJHQ2lnRB0NCzCsvmidgTgq
GmZeciloeMgNif6CckKDlCMa6mJoIEV4uvXlFd7cSefgBivwmHSwKgQNcaEh
eGFEsPuhPKNQXOb+JRVtUw3eRlrQIEkZoeGgawEalLsbrDpJ0AAN4gZikAQN
0CBBgyRogAbpOHSwoiRBAzRIM2KQBA3QIA2lg9UiCRqgQZIkaIAGSZKgARok
SYIGaJAkRdPt9eOLx9e3g65/dZPM3b25ulj1wJ2eXzC8ROVLaIAGSYKGrMkQ
3NfFyY6f/N4W5dWyKxtsLKFBkqBh0L+/T3FfFhIonFD+qmkyonJTOwLJxhIa
JClLNNz/+3tZafMYbEdvik1n7arLc4NNbvPtBUvbFl+EMwKlm+3abHejoXpu
ixrmi+dLG29qJyPZWEKDJOWHhnDGPjy92Liut5VLPSzPKG9Xl5v89dcrAJS+
LK612VYHN1HahBcXarjZdjZ0o6GKhNaphtab2mlWxsYSGiQpOzRUtqGbL8vL
g8126xktXzZ9l83Xt7e3jT9V2+1UL1yv5oGmb91KgGYfdDoDGqBBks4IDeFW
cr28uvUs9uzX0RBcruXLxu17cbnitQ3rxd03e+SZhp2mGmwsoeF8e9XPfLgy
rBMJGrZDQ/NWd+mFcD/EvtHQ85gGaIAG7d0K9CBliIZwg9iyeyL4chc0tO4T
qL8twt7R0PfVEx03ZfcENGgfYuAGKXk0bH0gZHlD3xMNy1venLO57fCbhEdR
boWGhwq+yQOHVDoQEhp0QC6gg5Q+Gma9XnJ5HfxjO1RDXzSUv0vz8QzzhZsD
JPeJhtJ36T66su8yaIAGDRADN0iZV5peiOzNnQ7PK2/uBA2ChvR67k/+eGVY
JzqgEiqTBJU3ZErokyf2piVogAbtSQzccHwxcIMO/k/s1rc/OBM2+MAqaNDB
xMANR+YCOkjxbi6tAmiABmiABknQAA2H7cknvH0+oEEHEkOKbnjye755PlL/
7bzwye9cDg9UJYWG6fhy02hy13CJu8nocjxdnypaListXS+KAg3P/a5vmI/U
uRCOvMUQvxuefPw3LkdmYkjIDUsuhCNpLoTDllIJoOF+c19s6Bd+aHBDgYb5
BWosmC9bX2dxwWZ3HB8NSzEk7QZoiFMMqbgBGqBB0LD3SYbKNr7wQTEFMRqP
Vwvn59ZMUHZEkypOgIZQDIm6oS6GaN1wDmioiCEJN+SHhroYUnRDoxi4QSmg
oW6GxjPv8XBPgXBPRiMOqrf4KAgaoAEaoAEaMkDDb//W08MHGaSJhtZjEMLz
1tMPpb0P9VmHzp0Tdk/sww1x/pzQAA0ndENyf9cZoOH//eZnDB9kkCAamvY2
VJzQoYvafokuMcwcCAkNjmmABmhod0NCP////c1PGz7IIEE0dB3T0DTT0IaG
yuGU0aAhj7zkMkI3pPLg8ZJLaDhE/+d//dHhw3N7imjoevVEwzENJWPMr7o+
OLKHGKAh77xPQ9JusJZOS4fkfvL/ffcpw4cHQJJo2Jig6X0a1m+/ULx6InxH
htVFy+/c0PVWDdAADdAADcqg37j7lOHDakwWDUdbC9DADcQQGR2sGe3Qf/uN
Pzx8tN/84iMyFzV/2NPmk7By/nxtG0to4AZiiM4N1ol261d//VOGj5bbLj4A
s/GjMOegWC3b/SMkoQEaBA2SjtW//fVPHT7apxnWUmhQQ3Bu3ptLDzJo4AZi
kPLrY//z03qOXjc3h0Ixf1CbTLg/9/rK7glo0LnSwbqSkuuX/8erh4/WiYZu
NFyspxpKvIAGaNAZuMFaklLsl/7708PHzjMNredCAzQoSz1YJ1LS/eKvffbw
MWudaug8pgEaoEGSlFC/8GufN3zMWqcaVhZ44NUTdk9AgyQp/v7hf33t8NF+
87X3aSjxYX1u1i+jsLGEBknKpL93+7rhw2qEBmiQpPz72f/yxcOH1QgN0CBJ
+ffT//nLhg+rERqgQZLy76//6huHD6sRGqBBkvLvJ//TVw4fViM0QIMk5d9L
//HJ8GE1QgM0qKGbX3umPqwWKd0++B++aviwGqEBGvSAFehByqAf+/dfPXxY
jdAADdpODNwgpdh7/t2fGz6sRmiABm3HBXSQUuyH/s2bhw+rERqggRieGTKs
wIP2Be8c14fVoh36vn/9dcOH1QgN0EAMz2Tshude/77NyIAL6KCd++5feevw
YTVCQ1xoeOGpHwqH9Q8Nw61QHxmIIS03PPnC91aGP73jd/Wxtw0fViM0xIKG
CheSpsMLT787HHmLIUI3dIshfjf0FEMqbqiLIV03pPKn3di3fuxbhg/bRGiI
Ag0dYkjLDZXnlJifX/Yohqjc0EcMMbthKzFE7oY2LqRIh4T+tNv6pum3DR+2
idBwejQ8KIZU3NDxtBLhk0uWaOgvhjjdsIMYYnZDNmhI60+7ra//l1fDh20i
NJwYDT3FEL8bHnxaie3JBRqg4eRiSMINyf1pt/Xnf/mdw4dt4snRcDcZXVYb
Te52vbnp+HI87fhW4Znzyz787aAhy2eWvYshEjckjYadxRChG/qLIX43ZIOG
r/4X3z18kEE0Mw3VLfq+0bASwubM4vt1f+eDomErMcTshp5PK/E8uWSJhm3F
EJsboMGf9kH7qn/2PcMHGUSLhp+ZOlYAABXQSURBVPnXo8kkmAuozwuslwQU
uEfDeFxZen/B0WQafotAF51qgAZogAZogIYM0PDkl753+CCDmNFwGUwFXNam
BYqtfnjVhQ+WrKhTIFxyb5JAH6VdFI+CoAEaoAEaoCEDNLzpn37/8EEGcc80
3NVPbzbw1S19bQKhtquigot2NJhpgAZogAZoyAwNb/gnPzB8kEG6aAiPoQy3
/yULtKKhc6YBGqDBgZDQAA2ZoeHxL757+CCDlNEQTi+sl3UdqnAX3TENM6+e
gAZoyBENMy+5jK/X/+MfHj7IIFk0hHgoo6H1RRGlJYsv2g5+gIZzeGbxPg3e
pyEGN8S/JcgGDa+7+YvDBxkkPNNQvHii+dUTtQmJKN+nYeYdIaHhRG6I8CEE
Df60D9drf+FHhg8yiAYN0a4Fnz2xpyeX2H7UXNEwS/wDq3L67IluOqT1TJjB
Z098/j/6keGj4xnl6uK+x9e3rZe5vX58cXUDDdBwYDckt9J8ymXkboj88ZPZ
p1zOMvpo7KQ/5fKZn//R4eNBDXS4YHHWBTRAwyHpYP0n4YaY72BCVtjBDR7A
6t9n/dz7ho/2aYY1BtrUMF/++OrKTAM0CBqsxpO4wSrSzr3m597Xc/S6uQUI
Nnsl5oCo76JYXsLuCWjQ2bvBCjw+HawW7dBT/+D9w0frREMnGtaogAZo0Hm7
waqTUukz//4Hho9dZhqKc6EBGnSudLC6pLR69d/9ieFj1jrV0HpMw/L4x7Cu
F1hAAzQoPzdYUVJyffrf+eDwMWudalhRoHMywUwDNOic9GC1SOn2qp/90PDR
8ZxRfZ+GBiFAAzRIkpJAw996cfiwGqEBGiTpDNDwMx8ePqxGaIAGScq/T/vp
Dw8fViM0QIMkncFMw099ZPiwGqEBGiTpDNDwN14aPqxGaIAGSToDNPy1l4YP
qxEaoGHP/dl//q7KsE4knbxP/6sfHT6sRmiAhkNZgR4kQQM0QMOh+twfHNdH
NmLgBp1tT31wXBnZbHQT+vlf/VdeHj48mKEhCjQ0ciEJOvTnAjoIFxKlQx7/
/n71+OXhw0MaGk6Mhge5EDMddhMDNwgXUqFDTlP3n/HSy8OHBzY0nBINW4kh
NjcMEUMSbnjDn/nwZqT7AH7jCx8Oh7/o2MQQsxsy2+X/mS+Ohw+PbWg4GRp2
EENUbsgVDaEV6iNdLqBDtGKI0w35HSr41IfGw4eHNzScBg07iyESNwwXQ5xu
6BZDKm7o4AI6xCmG2NyQ5UsMnv7AePjwCIcGaDiZGGJzQx8xxO+GnmJIyA3P
P/OecEADNOzWa358PHzYJkLDCdAwUAwnd0OWaOgvhsjdkBkaKmKI3A1DxBCP
G3J9P4PXvG88fNgmQgM0QEM+aNhKDPG7oVEMMbsBGmJGw2e9dzx82CYmhYa7
yejy8nI83SyYjjdfz88bTe4qF95cdHnNeavLbL7eFNzqYdGwFzGc0A37FUM8
boCGhMQQpxuGiyEGN+xFDHG64bP/8nj4IIPk0DAajTc2mI5Ho1EbGoLmtliZ
YH6qfqnGhdBwTmjYVgxxumEHMUTrhgfFEKEboCFyNHzOe8bDBxkkh4bxeLze
wt9NFl9U0XA/iTA/Wcw0FGZoEUU7GaABGqABGqAhCzQ888Pj4YMM0kPDdDpe
M2E8mU4qaCgIsEHD/TmTcWn3ROUmK9/nURA0QAM0QAM05ICGd4+HDzJIEA3z
/y428/f/uyuhYTQKUBCiYXPIQnUnxgPTDGYaoAEaoAEa8kDD5/3AePgggwTR
cD/FsJhTuD8ZoOFyMaGwMUB5puGuUQkPmwEazgINsywOhIQGaICGLjR8/3j4
IIMU0bA4AHLFg7vq7olCCOExDS1o6GEGaDi4GyJ5cHn1RHJuiPBnzuAllxmj
4Y9/33j4IIMk0XD/Usv7zX0NDcWJ4GiF4kjI0qxD1wsuDoeGmfdpgAZoGOyG
OH9g79MQ8/s0fP5oPHyQQZpoqJ8IBbA59KE4xHE6bng7hj4TDdBwLmiYeRvp
dNwQ8w+cwdtI54qGP/Gu8fBBBkmh4SRrwWdPHNINsf26M/jAKp89AQ0nd0Oc
v5rXftd4+Gi/+Zuri/seX9/Wz7y9fnyx6uoGGqDh/NAw89HY6bvBX3dUbojq
LmSJhi/4zvHw0XLbCxQsNVCcKnliTYnF2Y2sgAZoOJgbIlkzuaKhUQ8pPnRx
ISE3RHgXMhPDvC/8jvHw0T7NsJZCXQ3BmbWvoAEaDuyGqFZOxmLIJlyI3w3R
3oWcxFDp0Tte7jl63dwcCsX0QTCv0MwLMw3QcCw3RLh+iEHamQ7x//w5ieF1
3/ry8NFDAh0syHrnBDQcBw096RD5WsIFaSs6pPXz5zHB8EVvf3n4GDDTsDhS
MmMxQMNR0dBBh1RWFDFIeZf0/oh5r/+ml4ePWetUQ/sxDauFOb9uAhpOg4Y8
YgVJEfbFb/3o8DFrnWpYzSI0mOE8xAAN0CBJ+fQlX/fR4aP95mvv07DmQ/Ae
DZm/VYONJTRIUi5oeMtHhw+rERqgQZLy70u/9qXhw2qEBmiQpPz7sq95afiw
GqEBGiQp/778qz8yfFiN0AANkpR//T/GNvVPn4EGaJAkDWqrD4D1uS3QkAAa
3vUrb62P5NbYb//W0/XhgaRz7ts+9s2VkeK9ePu/ekdlpPXzf8WTF4cPD2Zo
OD0aGq2QnB4arUAPYoXukaIVEtXDm5790PDhUQ0NJ0ZDTzFE7oaeYkjIDU99
aFwZKT56n/7AuD78Uccjhvjd0EcMqbjhT73pg8OHBzY0nAwNW3EhWjpsxYUk
6FDnQop0aOQCOsTGhZjp0J8LqdDhK9/wE8OHhzc0nAYNO4shKjfsLIY43dDN
hVTo8CAX0CFCMcTmht3EELkbnv3SDwwfHuHQcAI0DBRDJG4YKIbY3NBfDDG7
YSsxpOKGL7/8SDhyFUM8bhgihpjd8OyXvH/4sE2EBmiAhq3FEKcbdhBD5G6o
cCF+OkBDzGh47vXvGz5sE6Hh2GjYixhO7oa9iCESN+wmhtjcsLMY4nRDBxei
pcNwMcTghuFiiNYNT173Y8OHbSI0HBUNexTDCd2wRzHE4AZoiA0NPcUQlRv2
JYbTumFfYojTDU++4L3Dh20iNEDDWaNhiBjiccNAMcTmBmiAhkP0/Of/yPBh
mwgN0AAN0BARGrYSQzxugIYE0PC57xk+bBPjQ8PdZHQ5nnYv2bLpuOn6i5td
1XHre0TD3sVwEjfsXQwndMNwMcTghr2IIR43pIiG/YrhVG7YrxgidMMLn/3D
wwcZnC0a5stGk7vN7a9PQwM0QENSYojBDdCQBhqefvfwQQZpoWF+ajSZjJfT
A4vt/DQ4XWhguWg8Hq0WL9AwHpcmFcqOaJ6KgAZogAZogIZs0PCZPzh8kEFy
aLgMTwanCwwUUqifbJ60CGYd7nsUBA3QAA3QAA05oOHT/8LwQQZxoqFeMNNw
t7nc5vRmqx9OGRQXCJfWJhU6d06YaYAGaIAGaMgDDX/se4cPMkhupqETDeGy
EhqKpbX9El1igAZogAZogIZM0PBH3jV8kEFeaHh4piG49WBfBzScJxpmWbzk
0qsnYvixveQy/pdcvvCHvmv4IIPM0NB+TEPl6Id+Yph5n4ajuOGEj3Lv0+B9
GqDhXNDwB75z+CCD3NBQvHpiPJk0vHpitaR+4MQRXj0BDd5GGhr27oZIfmZo
SODNnT7x24cPMogPDYfDx25rARqgwWdPQMMR3XDae5HxB1Y9//u+dfhov/mb
q4v7Hl/fbn0uNJxUCuu6D3A8DRpmPuUyyk/HzuCjsX3KZR5uiOFe5PrR2E9e
+bbho+W2b68fX1zdlE/1PRcaMlsLF/tfD6mLYY9uiOcXnbQYBroh2r++hLgA
DQmg4eO/cfhon2ZYW6DBBd3nQgM0QEOCaNjKDdE+XHMSQxsdIv9pUxfDcDck
8cT+3O/4up6j183NKVDsd5gTobwTovtcaICGI7ghkpWTkxh60iH+R2xmYkix
1MUwxA1n+isvSaDGgu5zoQEaDu2GqNZPZmLooENaj1tcSIgOMd8LXDDTAA2x
oGEHOkS7lnLiQk7hQvxuiP9eEEO/qQbHNEDDUdDQ3w2RryhikPrrIcV7wQqd
Uw2rCYS2V0+0nwsN0LBXPSS3xlhB0llONpTfiaEEBO/TAA2SJAkaoEGSJGiA
BkmSoAEaJEmCBmiQJAkaoEGSJGiABkmSoAEaDtGr/vaH6iO5Nfaqv/liZXgU
SRI0QMMBrZCcHupWoAdp2VMfHFdGivfi1T/50crwmxU0HBsNPcUQuRv6iIEb
hAuJ0qHOBXQQNJwADVuJIU439OcCOhy/L/ualyrDOomEC6nQoZsL6CBoOB4a
dhBDbG7YTQypuOHLLz8SjtS5gA5xiiFmN/QXAzcIGg6Lhp3FEJUbckVDhQvJ
0aFbDNwQmxjidMO2YuAGQcOh0DBQDJG4YYgYonVDBxdSoUMfMXBDbGKIzQ27
iYEboEH7R8NexHByNwwXQ4Ru6CmGmN3QXwzcAA3QIGiABmg4XzRsK4ZU3PDC
p35POHIVQzxuGCIGboAGaICGs0DDVmKI0w1ZoqEihsjdAA3QAA0RdzcZXZYa
Te7aLzmePnRDjVffLxr2KIYTumFfYojKDamjYTcxRO6GRjFE64bhYojBDcPF
wA3QEDMaQgpMx21s6EZD57nQcA5o2EEMsbkhPzR0iCFON0ADNEBDWmgIv54D
Iph9CM8J5ifuF5UvCQ3QAA3QAA3QoPzRsPhyueEPzljOPhQLgtmIzcLaTMOj
IGiABmiABmiABuWBhlKrLX99d0PzDog5IJrRYKYBGqABGqABGpT1TEO4vLKr
oXzJzQ6JS2iABmiABmiABp07GlpnGhZgqE5IQMPp3BDPo8lLLtNyQ5w/sJdc
EgM0JIiG8IzlyUYfbI6BOCIaZt6nARqgYbAbov1poQEaoCFFNMw6Xj1RHAgx
Go+XS6EBGrZ3Q5x/E1m+jXRC7wg58zbSxAAN574WoOFgbojtd50BGmY+sAoa
oEHQkBMaZrl8yuUsx4/GzuBTLmc+GjtNN8R2L4hB0BALGoa7IZKVkx8aHqRD
Ko9bYkjLDXHeC2IQNMSChiFuiGr95CeGNjqk+OjFhSTcEPO9IAZBQyxo2M0N
ca6lzLggHYcOqdwLXBA0RIGGbd0Q84oiBqk/HVK8F7ggaDg9Gvq7IZU1xgqS
BA3QcPjNbcpWkCRBAzRYD5IkQQM0SJIEDdAgSRI0QIMkSdAADZIkQQM0SJIE
DdAgSRI0WAtHRMMzP/+jlWH9S5KgARq6rEAP2kuv+qmP1IfVIgkaUkVDHzFw
w/H7nPeM6yNpK9DDCXv6J8aVYZ0IGqDhUFxAh5NzISE69BQDN5yKC+ggaICG
I4khCTe84U9/uDKy4UL8dNhKDNxwQi6gg6ABGo4hhpjdUOdCQnTYSgxxumEH
MSThhjd95Yc2Iz8xpOWGr3juxc2wdRA0QMOhxBC5G3YQQ2xu2FkM0bohtEJ9
5CSG+N0QWqE+bCYEDQdEw3AxROiGPmKI1g07iyEeNwwUQ4Ru6BZDzG7YTQwx
u6FbDNwgaDggGvYlhqjc0F8McbohdTTsRQxRuaGPGKJ1Q2Zo6CMGbhA0QMOh
xBCbGwaKIQY3ZIaG/mKI0A1DxBChG/qLgRsEDdAADdAADdAADUoRDXeT0eXl
eFpZUlow4Ja7bmePaNivGGJww25iiMcNexHDad2wRzFE4oZ00TBcDLG5ARqU
NhpKbNgLGqbjy8rNQgM0QEM6YojKDZmhYVsxcINiQ8N4Ot/Gb7bvBRoWSydL
VFyOJnebs0eTydIE90unwekCDKPJ1EwDNEADNEADNCg/NCyBMK2jYW2Bxcnl
wmB/xnKWojhdJkLTjMWjIGiABmiABmiABiWJhoULlj4oo2G92b+fYLgLT1RO
zy+8Od2KBjMN0AAN0AAN0KAM0LAmQAkNhQPWUxHQAA3Q4EBIaDiqG2w1oSFG
NKwQMG1EQ9QzDTMvuYzsJZfQAA2xuSGquwMNygMNlRdT3B/TUPUDNEDD8dxw
2rvgfRqgARoEDZ1oKB3ouHDAeNz06gloiNINUT3KocHbSMfjhgjvDjEoTTR0
VXPAfteCD6zaqxsifAD57IkIP7PKB1al5QbbS0HDodAw89HY0BClG2L7vfho
7FTcYGOptNBw4LUADXtyQ8y/5aTFsBc3xPzbScUKQ9yQyv1iBUHDCdAw0A2R
r7G0uDDEDbHdhSzFkG75iUGChpOhYWc3+I1E4oY47wIxpEUH60fQAA2HooPf
RSR0iPznJ4Yk6GCdCBqg4YBu8IuIgQ6p/PDEIAkaskVDhx6sf+1dD1aLJGjI
AQ2SJEEDNEiSBA3QIEmSzaVVAA2SJEEDNEiSBA3QIEkSNECDJEnQEC8ajt+j
R48u0s+9cC/cBfdiX9kYQYNam/9NuhfuhXvhF+FeCBrkmcW9cC/8ItwLQYP8
TboX7oW74F4IGiRJkqBBkiRBgyRJggZJkgQNkiQJGiQdubu7u/Tvw2Q0nvpV
RvGLuPSbEDTE03Sc/J/k4mllVap3pbgL6T8/Lu7LaHKX9J/E/S8iXftk8BdR
/CLmJf1wEjRkZ4b0n1nWzymp/qOkgNv8HiT/BHk3GY+SZcP95jZxuJUeRInK
p3heyuFPQtCQ2TxDsNlN/BkyzWmT4C7kMBc7HY8m03Tvx/oxtPpnbpqPp+VP
vZxwSPFvezop1AMNgoaYniCXzy4ps+Hubjqd3iX+/LL89+BmOjbp+djlgypd
/iz+GEbrH37xK0ntbixX/fqPIfW5E2gQNMT7PJP+nUj66aWYSU4acet/5yb7
T/Xy2k/xd3HvnvEkh4dTFkddCRqwIXIz3E0mudyVBH/2+dbqHgzj8TiDQ9iS
/F2UD0ZNW9NePiFoiFf0GUxjro6gGk9T3Bc9yeBwyKUXpg3bruTok/LvIlj1
yc/A2UEhaNCBNlYpvxJkOn9u9Aozv4u9/0lk8Hoccw2CBh3s34aSJGiQJEmC
BkmSBA2SJAkaJEkSNEiSJGiQJEnQIEmSoEHS0bq9fnxR9Pj6Njwj+HK7G725
ubVqJUGDlBkYKk64uLi6GYaGIdqQJGiQIuzm6qK+cS+2+NAgCRokbcywmlTo
3vZXEBB+Wd63sbi1YEnThdbfcLHs6np1RtdPIUnQIJ26B+cDHkRDw/KNGzbL
N0vr0xi0IAkapGTQ0LnR7oWGplsIL1+dzlh/XbKEJEGDlPdMw5IAF7U9DMEF
yrsvgt0WjnuQBA1SOrUd0zBfXrZCxzENwU0VdqigodEG0CAJGqTU5xqK/QZb
oKGskOruiaYLQ4MkaJCSY0OP92kIt/zLi9x/VZmqKF8+PPax+BabW4IGSdAg
JVdxXELHO0IGxyZc3az3X1SvXFx3tTjQR9PLMKFBEjRIkiRokCRJ0CBJkqBB
kiRBgyRJggZJkiRokCRJ0CBJkqBBkiRBgyRJirL/DxigiGEZ2JE7AAAAAElF
TkSuQmCC
"" alt="A dot plot showing the expression of genes and fraction of cells that express found markers in each group. On the diagonal there are two genes corresponding to each cluster with the highest pseudo_R2 score in their group." width="700" height="550" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 12</strong>:</span> A plot of the expression and fraction of cells that express found markers in each group. Here, the two top genes from each cluster were plotted on diagonal beased on the highest pseudo_R2 score. Shown is the result of changing the value from 1 to 2 in "top_n(1, pseudo_R2)" in the code above.</figcaption></figure>
<p>Look at this – we have identified some more marker genes specific to each cluster! However, sometimes it happens that the found genes are not as specific as one would expect, and they appear across the whole sample. Therefore, it is a good idea to plot all those marker genes and check how they appear in the bigger picture.</p>


In [ ]:
# plot the identified genes to check their expression
plot_cells(cds_clustered, genes=c('Pcdhgc4','Pcdhga5','Gm5559','Gm10359','Ccr9','Cd8b1','Plac8', 'Il2ra', 'Cd52', 'Tmsb10', 'Mki67', 'Hmgb2', 'Pclaf', 'Npm1'), reduction_method = "UMAP")

<figure id="figure-13"><img src="...
/h8OqErL5H8lMwAAAABJRU5ErkJggg==
"" alt="Plots showing the expression of fourteen found marker genes across the whole sample. Those genes are 'Pcdhgc4','Pcdhga5','Gm5559','Gm10359','Ccr9','Cd8b1','Plac8', 'Il2ra', 'Cd52', 'Tmsb10', 'Mki67', 'Hmgb2', 'Pclaf', 'Npm1'." width="900" height="600" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 13</strong>:</span> Plots of the expression of marker genes across the sample.</figcaption></figure>
<p>Further steps from now would include reviewing literature and checking what cell types correspond to the genes expressed in each cluster. Then you can annotate your clusters in the same way as shown above. Once you have your clusters annotated, Monocle can generate a file of marker genes for the identified cell types. This file can be then used with <a href="https://cole-trapnell-lab.github.io/garnett/">Garnett</a>, a software toolkit for automatically annotating cells. We will not go through this in the current tutorial, but we will generate the file of marker genes.</p>


In [ ]:
# use ‘top_markers()’ again, now grouping cells by the assigned cell type
assigned_marker_test <- top_markers(cds_annotated,
                                             group_cells_by="cell_type",
                                             reference_cells=1000,
                                             cores=8)

In [ ]:
# filter these markers according to how stringent you want to be
garnett_markers <- assigned_marker_test %>%
                        dplyr::filter(marker_test_q_value < 0.05 & specificity >= 0.25) %>%
                        dplyr::group_by(cell_group) %>%
                        dplyr::top_n(5, marker_score)

In [ ]:
# exclude genes that are good markers for more than one cell type:
garnett_markers_filtered <- garnett_markers %>%
                        dplyr::group_by(gene_short_name) %>%
                        dplyr::filter(dplyr::n() == 1)

In [ ]:
# generate a file of marker genes
generate_garnett_marker_file(garnett_markers_filtered, file="./marker_file.txt")

<p>A new file should appear in the ‘Files’ window. If you click on it, you will see the cell types and their corresponding marker genes, satisfying your chosen conditions.</p>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div style="font-weight:900;font-size: 125%">💡 </button></div>
<p>Note that you can use the above block of code to generate file with the marker genes for unannotated CDS object to help you identify and check specifically expressed genes – you’d only have to change <code style="color: inherit">group_cells_by</code> parameter from “cell_type” to “cluster”.</p>
</blockquote>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div style="font-weight:900;font-size: 125%">💡 </button></div>
<p>If you are working in RStudio locally, you might want to try a great function <code style="color: inherit">choose_cells()</code>, which allows you to make a subset of the CDS object containing only cells of interest to investigate certain clusters more in-depth. It only works in interactive mode, so can be only used locally - when you call it, then a pop-up window will appear where you can choose cells to subset.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"># create a CDS subset
cds_subset &lt;- choose_cells(cds_clustered)
</code></pre></div>  </div>
<p>Now the chosen cluster is stored as a separate CDS object and you can analyse it independently, using the methods described above.</p>
</blockquote>
<h1 id="trajectory-inference">Trajectory inference</h1>
<p>It’s time to perform trajectory analysis! First, let’s learn the trajectory graph. With an argument <code style="color: inherit">use_partition</code> we can specify if we want to learn a disjoint graph (value TRUE - default) in each partition, or to learn a single graph (value FALSE) across all partitions. The thing is, we can visualise the cells in pseudotime only if they belong to one partition. This is why it is important to make sure that all the cells that you want to analyse in pseudotime belong to one partition.</p>
<p>In one of the previous sections we’ve already prepared the partitions for trajectory inference - we assigned all the cells of interest into one partition by changing the q-value. As a result, we got two partitions – one containing only the cells that we labeled as ‘Unknown’ and another including all other cells. That is a perfect assignment – we’d like to focus only on maturation of T-cells, so we don’t want to connect those two partitions (therefore we specify <code style="color: inherit">use_partition=TRUE</code> to get a disjoint graph) and we will work downstream only on the graph going through annotated T-cells.</p>


In [ ]:
# learn trajectory graph
cds_trajectory <- learn_graph(cds_annotated, use_partition=TRUE)

In [ ]:
# visualise the learned trajectory
plot_cells(cds_trajectory,
           color_cells_by = "cell_type",
           label_cell_groups=FALSE,
           label_groups_by_cluster=FALSE,
           label_leaves=FALSE,
           label_branch_points=FALSE)

<figure id="figure-14"><img src="...
AAAAAElFTkSuQmCC
"" alt="Learned trajectory graph connecting all the annotated clusters except the ‘Unknown’ group of cells." width="650" height="540" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 14</strong>:</span> Learned trajectory graph.</figcaption></figure>
<p>We have to tell Monocle where to start ordering the cells, ie. when we expect the analysed biological process to begin. Thanks to our biological knowledge, we know that the beginning of the trajectory should be at DN cluster.
There are a couple of ways to specify the root cells:</p>
<h2 id="option-1-root-nodes">Option 1: Root nodes</h2>
<p>Here, you will use <code style="color: inherit">root_pr_nodes</code> argument in <code style="color: inherit">order_cells()</code> function.</p>
<p>To find the names of the principal points, you have to plot the learned trajectory again, specifying <code class="language-plaintext highlighter-rouge">label_principal_points = TRUE</code></p>


In [ ]:
# specifying root cells: `root_pr_nodes` argument - check the principal points
plot_cells(cds_trajectory,
          color_cells_by = "cell_type",
          label_cell_groups=FALSE,
          label_groups_by_cluster=FALSE,
          label_leaves=FALSE,
          label_branch_points=FALSE,
          label_principal_points = TRUE,       # set this to TRUE
          graph_label_size=3)

<p>You can see now the principal points and their labels in the form <code style="color: inherit">Y_number</code>. Pick the principal point in the cluster that you expect to be the beginning of the trajectory and type its name in the <code style="color: inherit">root_pr_nodes</code> argument when calling <code style="color: inherit">order_cells()</code> function.</p>


In [ ]:
# specifying root cells: `root_pr_nodes` argument - use the relevant principal point
cds_order_1 <- order_cells(cds_trajectory, root_pr_nodes='Y_14')

<p>There is also a helper function to identify the root principal points based on the annotated cell types. This function uses <code style="color: inherit">pr_graph_cell_proj_closest_vertex</code> which is just a matrix with a single column that stores for each cell, the ID of the principal graph node it’s closest to.</p>


In [ ]:
# a helper function to identify the root principal points
get_correct_root_state <- function(cds, cell_phenotype, root_type){
      cell_ids <- which(pData(cds)[, cell_phenotype] == root_type)

      closest_vertex <- cds@principal_graph_aux[["UMAP"]]$pr_graph_cell_proj_closest_vertex
      closest_vertex <- as.matrix(closest_vertex[colnames(cds), ])
      root_pr_nodes <- igraph::V(principal_graph(cds)[["UMAP"]])$name[as.numeric(names
                                                                    (which.max(table(closest_vertex[cell_ids,]))))]
      root_pr_nodes
      }

In [ ]:
# call the function to automatically find the node in the principal graph where our DN cells reside
DN_node_id = get_correct_root_state(cds_trajectory, cell_phenotype = 'cell_type', "DN")

DN_node_id      # check the node found

In [ ]:
# order cells using the helper function output
cds_order_1_helper <- order_cells(cds_trajectory, root_pr_nodes = DN_node_id)

<h2 id="option-2-root-cells">Option 2: Root cells</h2>
<p>Here, you will use <code style="color: inherit">root_cells</code> argument in <code style="color: inherit">order_cells()</code> function.</p>
<p>Specify a vector of starting cell IDs. You can provide only one cell as well as all cells of a given type.</p>


In [ ]:
# find the names of all cells belonging to a certain type, identified as a beginning of a trajectory
starting_cell_type <- 'DN'
index <- which(cds_trajectory@colData$cell_type == starting_cell_type)
DN_cells <- colnames(cds_trajectory)[index]

In [ ]:
# alternatively, if you work on unannotated data, you can use the number of the cluster that should be used as the beginning of the trajectory and pass it in the ‘root_cells’ argument
starting_cluster <- colnames(cds_trajectory[,clusters(cds_trajectory) == 4])

In [ ]:
# order cells
cds_order_2 <- order_cells(cds_trajectory, root_cells = DN_cells)

<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div style="font-weight:900;font-size: 125%">💡 </button></div>
<p>If you are working in RStudio locally, you can use <code style="color: inherit">order_cells()</code> function in the interactive mode. The pop-up window should appear and then you can simply select root nodes in the cluster with the cell type identified as the beginning of the trajectory.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"># specifying root cells: pop up window
cds_order_1 &lt;- order_cells(cds_trajectory)
</code></pre></div>  </div>
</blockquote>
<h1 id="plotting-in-pseudotime">Plotting in pseudotime</h1>
<p>You can use any <code style="color: inherit">cds_order</code> object for the downstream analysis. Let’s pick one and assign it to an object with a shorter and more general name.</p>


In [ ]:
cds_order <- cds_order_1_helper

<p>The function <code style="color: inherit">order_cells()</code> calculates pseudotime for each cell, so we can now visualise the T-cell maturation process in pseudotime! Additionally, we can access it and store in our CDS object for further analysis.</p>


In [ ]:
# plot cells in pseudotime
plot_cells(cds_order,
           color_cells_by = "pseudotime",
           label_cell_groups=FALSE,
           label_leaves=FALSE,
           label_branch_points=FALSE)

In [ ]:
# access pseudotime calculated for each cell and store it alongside cell metadata
pseudotime <- pseudotime(cds_order)
cds_order@colData$pseudotime <- pseudotime

<figure id="figure-15"><img src="...
"" alt="Plot of the cells ordered in pseudotime. The trajectory starts at double negative cells (ie. pseudotime = 0) which are coloured blue, then goes through double positive (middle T-cells), gradually changing colour from purple to pink, then double positive (late T-cells) are coloured orange and the trajectory ends at T-mature cells, which are coloured yellow." width="650" height="540" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 15</strong>:</span> Cells ordered in pseudotime.</figcaption></figure>
<p>We can now see how our hard work has come together to give a final pseudotime trajectory analysis, which starts at double negative cells, then gently switches to double positives: from middle to late T-cells, and ends up on mature T-cells.</p>
<h1 id="differential-expression-analysis">Differential expression analysis</h1>
<p>There are two approaches for more advanced differential analysis in Monocle:</p>
<ul>
<li>Regression analysis: using <code style="color: inherit">fit_models()</code>, you can evaluate whether each gene depends on variables such as time, treatments, etc.</li>
<li>Graph-autocorrelation analysis: using <code style="color: inherit">graph_test()</code>, you can find genes that vary over a trajectory or between clusters.</li>
</ul>
<p>In this section, for some examples, we will work on the subset of our CDS object because the computation time for the whole dataset would take quite some time. Let’s make the subset CDS containing the information about the genes listed in the table in the previous section, so instead having 15395 elements, it will have only 7.</p>


In [ ]:
# make the subset CDS
test_genes=c('Il2ra','Cd8b1','Cd8a','Cd4','Itm2a','Aif1','Hba-a1')
cds_subset <- cds_order[rowData(cds_order)$gene_short_name %in% test_genes,]

<h2 id="some-more-plotting">Some more plotting</h2>
<p>Monocle also provides some easy ways to plot the expression of a small set of genes grouped by the factors you use during differential analysis.
For example <code style="color: inherit">plot_genes_violin()</code> allows us to create violin plots which are quite common in the field. Therefore let’s visualise how the gene expression changes between the cell types.</p>


In [ ]:
# produce violin plots
plot_genes_violin(cds_subset, group_cells_by="cell_type", ncol=2)

<figure id="figure-16"><img src="...
4ojZYrHi6N8Bt4UyV90n9SkAAAAASUVORK5CYII=
"" alt=" Violin plots of the expression of the specified genes in each assigned cell type. The trends are consistent with the previous biological interpretation and assignment." width="859" height="543" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 16</strong>:</span>  Violin plots of the expression of the specified genes in each assigned cell type.</figcaption></figure>
<p>When analysing the above violin plots, we will realise that the results are consistent with the detailed analysis in the previous tutorials, as well as findings in current tutorial.</p>
<p>Another great function <code style="color: inherit">plot_genes_in_pseudotime()</code> takes a small set of genes and shows their dynamics as a function of pseudotime.</p>


In [ ]:
plot_genes_in_pseudotime(cds_subset, color_cells_by="cell_type", min_expr=0.5)

<figure id="figure-17"><img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAegAAAImCAYAAACYWhlVAAAACXBIWXMAAA7C
AAAOwgEVKEqAAAAgAElEQVR42u3dPW7jSuKucW9DvY1ODrwCJ45u9E8uMMCB
gRscXzg5G7i50ZmTAWYBzj2Ak0km8B4ajqa3MMDJeE1JlKlyVbFIkRQl/R6g
0BZZ/JDa3Y/e+uJVBQAAFseVjwAAAIIGAAAEDQAAQQMAAIIGAICgAQAAQQMA
QNAAAICgAQAAQQMAQNAAAICgAQAgaAAAQNAAABA0AAAgaAAAQNAAABA0AAAg
aAAACBoAABA0AAAEDQAACBo4bV6rh9VN9fTe2vT+VN2sHj72rF9UTzer6tu3
b9VNu9JeHQAgaGBkPz9UDw8P+/KtqryIP46phb0iaAAEDUxBnY7r9PyRom+e
qvcvUq7T9Wor403Kfn1YbX5+laABEDQwkZ8/JLsVcy3eh9dIak41ZWviBkDQ
wFR+vvls2n59qFaNoQkaAEEDR9PzbvBXU1YxKRM0AIIGZo3Pu+bthl0zN0ED
IGjgWH6++exz/jT0ppmboAEQNHAUPVdPNzG5budEvxI0AIIGAICgAQAAQQMA
QNA+AgAACBoAABA0AAAEffw3c+X7BgCAoM9W0H/9+YffDACL4L+//8eHQNAE
3ci5KQBwbDk3BQQ9PW+P1e3t7bY8Vm/pitVjU+/+ufolQQOQoEHQE/Hrubpv
SfnX8311+xhT9K/q+f62anal600jaAAALi9BZ4S9n55b25P1CBoAQNAT+DmR
jGsh7zVr18K+r56Ddu7v37/vFYIGcG5o4iboIyTnun/5q3Q3Pn4M+p3jgpag
AZy7nA0SI+gjkRBvYYImaAASNAh6KkU/fg4G2xe3PmgAAEHPZePCpuvNKO77
7Q6juAEABD0xa9nu5kG35PwlJR9vHjQAABcn6EnfzMiCtqIYTvrfw7/+vajz
zMU//3FeX9T1QRM0QSfkTNI4VTk3ZQnnmVPOTTkXORvFTdAELUFDgpagJWgQ
9GkIGgAAgiZoAAAIGgAAgiZoAAAIGgAAgiZoAAAImqABAAQ9JZ0PwLDUJwCA
oOelfmjGWr4pQW8eltE8H8PDMgAABD25m7cPyXjLJWiPmzzoL/X1t8Wf85Dz
TfH+lnjNSd5HYlWwY68Wdgqrf9Urek1xn10rhVlJjKDnJyfdel/BYym/f/++
Vwh6I5KmLPWch5xvive3xGtOJefYutrHXm/7FNbPbuQ89n12rbVtLW6CXp6g
i58bLUFL0BK0BC1Bg6AXl6AJGgBA0HMKWh80AICglyjozSju+21kNoobAEDQ
xxL0F2GbBw0AIOjzeDMEDQAgaIIGAICgCRoAQNBTUta3vB4Y1tTLLgtK0AAA
gj6Q8jW26yVBCwZuEzQAgKDHSc8l85trkXcvTDKmoP/684/JS/K+g6UkY6/r
49vbxyrh9Ur2xe576LnGfB+ln+ko91Uvjdkc3yyV2WzfrsrVXkIzXE4zta+9
/29//6zX/Fz/2d4+tKTuIXlvrffW5xrtpTFzy2S2l9HMlVTdrnPE9ofbappl
NevX7WU2U6V9zCF1SuqCoCcO0KUrhLWawbcllqbHWot7Lb+PYxVFUZZYSJqg
ZwjQhWtsr5N1e3tZopagJWgJWoKWoEHQkybomNv790kDAEDQZRF60BrbBA0A
IOhpI3TZGttfmsLr48pEDgAAQQ9O0ZF50EGa3p8H3X9ENwAABA0AAAgaAACC
BgAABA0AAAgaAACCBgAABA0AAEEDAACCBgCAoAEAAEEDAEDQPgIAAAgaOG1e
H6pv377tyurmqXpPVn6vnm5W1cNr6XYAIGigN+9PN9Vq9VC97vl6lZF0XMT1
eWq5EzQAggYOj87Vw+qmenrvknBdb7VN1w/VQyjo96fqJrYdAAgaGOLnh47m
7M9EfdNYfNsc/iniWua15DVxAyBoYDxBdxq1Ts/tJvB9EddN2xt5EzQAggbG
Yd003ZGg6zqRPuq1iPeOJ2gABA2MFaEL+qDTCboZGBaWm6d3Hy0AggYOC9Hd
o7jzfdAxqQMAQQMjBOmuedAb+SZHcRM0AIIGAICgAQAAQQMAQNAAAICgAQAA
QQMAQNATv5kr3zcAAARN0ACOyj//4d88CJqgASxOzk0BCHoEfj3fV7e3t63y
WL1Fa75Vj02d++fqF0HjUv+R/uvfs15jjusdkpRTPwMEfSBvj7fV41unxqvn
+896a6l3H0TQOEs5N2WOa8xxvUOSstQMgp4uP3+I97567ozDdXpuJetfz9V9
MmkTNCToQxP30hJ0mJL7pObcfmIHQWfFe7vXxB0NxrWQ95q16+O+iv379+97
haBx6Ql6iQl4zsScqy99g6CzAbpOwm3RJhL122PQ7xwXtAQNCXqcBL1ESQ9N
vRI0CHqsTB3rky5M0ASN0f9BvP6W3J7aN7U0w2P/9vfxhb80qeeauEuT8qH8
9/f/TPLe2ued6hog6OkErQ8aR5JzTMTt7UMkfUizc3hsLeemDJVzeC9LaxZP
NXE30s01f481kKwWZ1PGlnOsAMcXdN10/fv/bfVB/7/q8T4m3s0o7v9p6v3v
/20UNyRoCVqCBkFPxkcS/j+3/9MS9P+qbv/fWyQlbwT9O0EDAAj6CCSbrpfd
xP3Xn3/4zTnBZJxLwLnk3FWnK4V2TWFqNy+XJNm+U6JydXJN3nMm7FRCjqXl
PttTSTqXsFOpNtx+aPotPV7KJugj+TmxAMmCp1nVcm4KTkvOKUmX9j3nJJ/q
x+1aBKTvIiGH1C89V3jM1H3UJX3Mh5bc9UIZxvqFw+2H9h+XHq+fmqCPlJzr
5uvEyOyFT7MiZwlagpagJWicdYJOitc0KwAAQR9Z0aZZAQBwZEEXN11vRnHf
b3d4WAYAgKAnZv9xky05f0nJy3zcZHuAWPjzQX8RXf2bPfpP+75uby/pi53l
F3N7vZJBXWPMXQ7PtytN33BkkFh4/a7BWM3P4Z+5AVhhvWaRkj6Dukrq9Tk+
fF+xPvGhfdZd/czhttLBZbH9MWIDwdo/h9tyx8bOHf6ZW7DEYiY4q8g5taDb
I7hTZaiMuuRYOgK57+vYeXL75pJz6n5SI6sPWf0rd85G0kV1eozSLhmNHduf
k/MSS185z11yQiwtuWOHnLvkfCBogpagJWgJWoKWoEHQpyFoAAAImqABAARN
0AAAEDRBAwCwQEF3Lj6yzGlWAACcr6DrBUvW8k0JerNQyePuSZQWKgEAEPTE
bt4uUPKWS9DzL/WZmy5VMu95jPnQRX9hB0x3GjKV69ApW6lpSam5zX3qDp0f
3jWVLHeN1PSucD70elvBVKo+86CbKVZjzIWOXiOYLvalROZ7x6ZkDSE2var5
s2Q6VW577nrhz7G5y+Hr3MM0uuY1p+q1r52b2gWCnoecdGd+3GQf8fYpU8h5
6IIhQxZDGWvRk7lK3+tPds9D5Ngx97gt57Ek3UvAhe9tiQuUlF5vyAIlXXIe
u4Cgjy/oIzxuUoKWoCVoCVqCBkGPlKDHFDQAAATdJWiPmwQAEPQSBe1xkwAA
gl6GoE/kcZMAAJypoMvEu//c6NycaYIGABD0obG5eAGSer50Qas2QQMACHqc
9Fwy+KsWefeo7TEFfch850PnRZfMN66Pn2rucN99sfseeq6x50CXfKaj3Fds
WlJkmlWfZzGH+9vTqZqfx5xm1eveesyBbk+1ik2DitFnTvOYz4MOXzdTmkrn
RbePOaROSV0Q9MQBunT6VKsZfFtiaXrMhUrqYxVFUZZYSJqgZwjQhQuQrJN1
e3tZopagJWgJWoKWoEHQkybomNv790kDAEDQZRF60AIkBA0AIOhpI3TZAiRf
msLr48pEDgAAQQ9O0ZF50EGa3p8H3X9ENwAABA0AAAgaAACCBgAABA0AAAga
AACCBgAABA0AAEEDAACCBgCAoAEAAEEDAEDQPgIAAAgaOD1eH6pv377tyurm
qXpPVn6vnm5W1cNr8/KpulmtNsetHqpXnyYAggYO5/3p5otYXx9WGUm3Bf1a
Paxuqqf31rmycgcAggZKovOeYJMpeV1vm5JvHqqHZl+dnveEXNdryb6Vruvy
IF4DIGigxM8PRYm3TtQ3jcW3zeHdCTqQ/1rWmsABEDRQJujOWBuk4my6zsk+
PA8AggYQ50sTdaJOpI9618TdTsm18CN1d4PPCBoAQQNFEbqgDzqToL80kbfP
t0nW+6O9CRoAQQOFIbp7FHeyDzqXoEMhh3UBELSPAOgK0l3zoDep+cso7uDY
VSDgWv6f5wyOA0DQPgIAAAgaAAAQNAAABA0AAAgaAACCBgAAFyfoX8/V/e1j
9ba38a16vL2tbuty/1z9KnkzV75vAAAIehzeHjcS3hP0r+r5/rZ6fGv8fV/d
Pr4RNHAh/PMf/i0DR/1X8PZYi/m+en4LE3SdnluvowmboIFJ/lP417+PLuem
AAR9bEIB16/3mrVrYX+IPGjn/v79+14haBDo4eduigQNEPRXQdfN3gWClqBx
acmzVKCl9xKrd2w59xF2LGlL3yDoBSRogsaS5DxX8iyRc6nEl5CWS+ScE3F7
X2wbQNBjClofNJbwj+P1t8Ul6L/9fVOaa6UScLg9JeK+SXvK95gTammCJmcQ
9OSC3ozivt9GZqO4cQw5N2UptOUcllwq7lO3K2lPlbiHiDWXngGCnkzQTYo2
Dxqnk6C7EmbfBNtOrXVpBB2TdFfabddLbc/dV+rYqRJ06uf261x6PkVJ//f3
//hHhwUKeqw3Q9A4YoLOJcy+CTaWWlPpOZRvl7xzx5cm67nSdFcfc277KUm6
lnNTAIKW/M76vQ79DNpSbv8Zni8lplwajaXPkr7kUIY5UXd9GShJ0GOPFB+S
olMp+ZQTdJd8U/tJm6AJ+oKT37m916GfQfu4VMkJbGifcN+025Wkh5aSBD4l
qRRcmpSXnKCHJmTJmqAJ+sIT9BJkHruHrnSck3C4r1TYUSHHZF3LK3L+rpSc
2lba1J0bKJZK0KUyjl4z03KQSuWHCjqWkkuSckrauYQ+Z2Juv25+7krN5EzQ
BH3BCXoJiTt2DyXpuETQpfVz91OSrLtSdmla7UrauYFifa5RVDLvqaQFYMz0
fGiZO2Hnkm97X6xObP9ZSvrnj+r66rr68TP4uejYl+rl5wX8f77YO9s9RCN8
kAZBS9C/Fd33kASdunZpgk6lzTBxpiScS9A1jZxzCbakqXypCTqWdM8lQacS
8sUm6KGC7itzgh6ZYNqVedAAQNAEfQLCPrag//rzj8Un6LlT8SGpvU96jqbZ
xPGxZurc9bqatPf2tZNkplk7lVR7pdjCUdxdxw7tmy7t5x4rPacSdN+pVrFr
HLJud8ko61zijaXmrvrLb+L+Wf24vlr/37sudy/5/Vd31cshgl7X+zzf9ccB
L3dfr/vzx/VmW3PeH3ef93D9o/pZeo8EXeLneII+xtOsajk3ZclynlPSh/R7
9+l/zh2bOn7M+nvN22E/7QSjqg/tgz5GGSrnkpHaQ/ueS0XfJec+/cV96p1u
P/RWbDs5vlR3H/8Hf7oy3L8VZyPIsRL0y10g1c1115fdCb3ZH95Txz0SdFdy
3j4zumApMQlagpagJWgJ+ghN1DG+iDOQ+GhN3MEXg/X+7XW3gt4L2O3ju+6R
oEvwNCsAWBRRuVX7SfTqKlrqpukx+6B3TdrBz/Hzfgq48x4JulDRj7dV1zix
QwQdpuJcSo7tOyRVD11QY0hzcy6tdqXMoUm+a2Wu0qbsrnRbkqBT7693M3d4
7p4peGiKHat5O3qNHtOphjZ1lzyVauh0qb5N3WFSD+8nllqb7aXN0rGk3LW/
61wnJehcU/GYg8R2qbnVvF0q6IU0Z5+OoOspVjM+DzrsV871M8f2HdIvfWj/
bZ+m3yH9r2PIefI+4QOOP+R6c/c/z9IHPeL7SUm467nOxyzh/bTFGMo5Jd0p
y+k1cWf2jzqKeyvmH63m7WwT97ZO1z0SdJz1wLDdPOjp+6AlaAlagpagJehe
hu4YcLV93U6obSEOXqgk3ke8a65u7ygdJJa6R4Ie8c3ogwaA+SVdPM2qJb4D
VhJbT61KyDg6IKw9zarPPRI0QQMADv2uEDRvV9XJL2pC0MAZkhqZOmUBjuvn
668jrwl6YgpXESNoEO/xZEniGC8F536HIrJtjomNxiboCdk9MIOgQcLnIDkC
B85A0PW85/Xo7bd5EnTJtKrYdKx2vdR5SkZ4p0Y7d9WNbc+NWO4z6jn889BH
QpZu73uOvqO4R5lmFT7NqsdKXXv7EpI6yiju3AjyyPSrrvc4prwPGcXdUPK8
6Xq0dHuKVW4aVUPX6Oz2vti5wrpd1wNBL4dME/dYa3HH5JuTc2kJz9El5xL5
jDl3+JBjJ5tTfOD1j3IPpWJMCeiM1uIeKuncFKkuaY9Z5prPfBbzoEHQc/VB
S9AS9JgJuqsJt0tsl5igcw+3iCXlUnFL0CDoExc0cNA/JH2pi/x78HcBgiZo
EIEP5UT+nvxdgaAJGoQMf5cAQRM0/CcOf9cg6AX/QwP8J40xfxcAgj6yoA+d
UlU67Sp63wVPXqqPn2pqUt99sfuebLR0zylWJZ/pusT6JgeO4o6NdN7bPvBp
Vu3R2s3PR3uaVY/pZO33Fxs5HaPPnOahT7Pq2lbTjJhuj+IOf0/6PmqypE5J
XRD0RQp6Lb8jrF2sHLmMOf955OdDhxJuz3+e/XnQB7yv3MIhMTkfs7TlnJoX
fazfVZImaAlagr6cBH3o4iYS9MUk6EPTsQQNggYAgKABADgSiade3b18rbO3
bbc9eG40QQMAMJagQ8m+VHdtIe8kHtQjaAAAPil5JsFhgt5ub54L3dR5uauu
2jGaoAEA+JRz12DZUQS9TtHX1Y+f+3Ve7sJkTdAAAMyXoD+y84/rr4JeiztM
1gQNAMDI9EzQm0Ouq+t6B0EDADCzoOv+5mRS3qbrF4IGAGBGQcdGccdGcEdG
dhM0AABjCbpkHvRXEddN3QQNAAAIGgAAggYAAAQNAABBAwCAExJ0YtTc1QkN
WQcA4MwEXU/wvtpfcBw4V14fqm/fvu3K6uapek9Wfq+eblbVw2vz8qm6Wa02
x60eqte907bqJU9XH79/HACCzhBMAAfOlPenm6hY05JuC/q1eljdVE/vrXO1
jusU9PaLwYqgAfRN0ASNM4/Oe4JNpuR1vW1KvnmoHpp9dfrdE3ld71O2a0E/
fKbztqzXXwLqa79K0AB6CbrarHGqrxln7eeHjubsT5neNBbfpt7SBL17nWrK
1sQNoJ+gN03cVwaJ4dwF3dlJvJ+K8+l6X/ZhE3e0yZugAfRO0MC586WJOlEn
Nfhrva/VRF4Lf6+Je7/5nKABEDRQFqEL+qAzCfpLE/n++faFHCZvggZwgKA3
TwHZb95ePwAbOJsQ3T2KO9kH3ZmgV59N6PqgAYwl6PgjujZ90ySN8wrSXfOg
N+n3yyju4NjVKtKk/fAQ3UfQQK+0WPi4yavyUJl4POUJCDozD9robgDA7IKO
B8adp5J1rquoowkaAHCJ/PMfIw55Ssm03n79o/qZrJNZ0+N0Ba2JGwAwXM5N
mVTQ7YQcq7PednYJui1pg8QAAAtM0OuE3BZ0Rz/1OQkaAICjMzRBt2vetcRd
W5ugAQCYSND1mKhsH/QQ6S9S0O2BYaex1Gd9LwCASxR0ySjusxH06UHQwOUw
ap8mTlDQJfOg+wo6EkCP+BhHggZwcmLuGhVM3jgLp5VXTTV5j9Ak8Ou5ur99
rN72Nr5Vj7e31W1d7p+rXwQNkHMg55ikR5/SAyxd0OsRb9uo354Tvf75kCaA
t8eNhPcE/at6vr+tHt8af99Xt49vBA0M+Uf+r3+fhZi7fm6/PhU5//f3//gF
xaGCbqfnzUosOykfsJLY22Mt5vvq+S1M0HV6br2OJmyCBkrk3JRSefetf6zU
HKbkU0vOtZybAowk6GD1sDGW+gwFXL/ea9auhf0h8qCd+/v373uFoIGvEm1e
x7aHAo9ty22fKim3/ywppyxp4EBBb1LzWsprIX8uldZu+h5N0HWzd4GgJWhc
Cn/7+2GJuUS6peVYSblL0F1908CZCrraH4K+FfJazs2k8CMkaIJG8nfh9bfZ
jy+R1xDB1XJuSomscyLuStA5cU9J38TcN1mnrgech6Cn5Jc+aIwr56bMdXyJ
yA6RXVvObVnnrtEl2iUm57ElXXI9gKB7CXozivt+G5mN4p43PZ7D+wz3lXwm
7TqhpMN90WsUiKyv6FJi7dpfkqC7jg/7rkv7sMdK0DFZpwSeq7vUBN30P4d/
AgMEbR70JaTHc3yfJXXbdfqWlKwOfo8D+4hL76HPeboGk43x/sdu3l5yUm6P
4A4L0FvQk82DHvPNEPTFJOhUuh0jQXdKuZZQIPzd9kTK7ErUXem0tGm6K0H3
HbFdum2sLydjJujUCO8wsc+dmPsmaNIm6B7pebx50ASNQ9NzLB3HmqdHS9Bt
GcWE3mNqUqk4S+Q9d5lK0FMn6WMl7K6EnNovWRN0T0FPMA+aoEdLyTERnWKi
7ivU3P4SSafqHpKgu1JwX0m3B4oNEefQOc9Dv0gMkXKXpIck6HD/FAk6loJz
ibkt3VSCJmcsYx50jN0SoOEyoASdkk20b/QE+6S75JuSaJ9EPCg9J0Q9RapN
ybI9insJSXoMSc+dksdOzrn+5JiUS/qe9U2jh6CraedBhwSDxozilqD7fFHp
SsVdn2fqi86XtNwxijs1wKokQcfOm0rQ4bVL0vk5JejUMbHztuuOyRgJOid+
ZHyUfdzk1zq71t/UOYPAufZcrJV4hudHn4bRFjoP+q8///jyOtx2DGmPIeS5
pZ76UpHrXx7axN1Vv296Dpu4SwTWJcmcKPs0cfdpRu8j6KnnSU8xSCx1/phE
SxJz7HVsX07cKVFL0CWCDuXY7orN1flsAf56zuvq+q4dOj/qX19/Pc+6Jflq
8u7dkxB0KkEfcy3uRsaNkNuvjyHpEmkd2oQ+x70f0uTcV7YHybk1UOzLtgkH
YC29iXssOU85CCy1v0SGJU3VU5ZzbPUbT9Db7U2rbrTOprs22iu7rX931xL4
x7a7u7u982wS9Uedl4Ul6M30qs+mhPXrKZq495Lz9olXBROhJWgJWoKWoCXo
Zcp51C/9yeblVkKO1dmm5HSC/qj/kY4bgf/8cZcW8ZKauD/7mzNTribDWtwA
IEF3ybF2UlvQHf3UsXPWf64r1efavl62oL9Os7r7zPuzTLOqnx3dNU6MoAHg
AhiaoMPQ2R74vKv/Iea7bRjd207QjY1nf9xkrGm6vS32c7jtkD7oXg9m6FhB
K9UsHP4cO1947pJm85Im49w5UtftmjZ2yDzoPlOtovuDhUp2dQpX2hoyD7rd
B31of3bz72XKUhPr/41RMoo7tS93TK7/OXbd1BSp9r4UXU3duSlX7evk6qBD
jrWPsn3QZed8qfuhf9xtRnz/PIEm7s/+5syiJSOyHhi2mwc9bR90TKyhcIeU
PsIdsq70oYOdhh5T+oVg0jnII30OB91De5BYahDZ3KWvSJd0L1fTzoHuqnPI
YC2DxJYg6JJR3IXnbK/3cQqC/kzLhXPKjtHHIUFL0BeQoPtIt+81ctcurdN+
f30SdJ9ULkETdNk86IGCTv28aEEvHH3QOLff55SkfCY+E1zA73vhV4v03DGC
BkhnoZ+hzxEXIOigbX9OClcRI2icskzgCxAwUNAzLEoSY/fADIIGGWPavxvg
pBN0uhlp/I7yet7zevT22zwJOja4K7WMZ2rUd+o8xX8ZBQ+5KB0g1ufhEOHx
4XVKR0vn7iN3f3OMLh/1cZOxlcRyI5cHDhJrT6dqL/E5xjKf0WvnnszVY5T6
EPqsCharkzomvEb4c7tu6v+3rhHVuYdg9HmaVVddEPTyyDRxj7UWd0y6c06z
KhXKIVOLctcaIsQllj7vpeSLTVGZaOpSKONQ0JOtxR0T8ICpZEPlfKwpVqmp
Vn2lbZoVCFqClqCPlaATzaJ9piJJ0MtM0KWpt0TaEjRmF3T7YRmd65qemKCB
1O+Ufkr0/T3xu4JZBb2Rc3zllkkXKyFo+I8WZ/B75HcJEwk6M81q6odlEDQk
ZPjyB4ImaBAyQNg4IUFX+4uHB+JeynrcYwwSm7Ik77tg3ej6+Cke+pA7LrUv
dt9DzzX2CO7ovYX/EY5xXx2DqlLb+6xlPcbTrAY95WqE0eANsYFZMfo8zSrc
nzsmN4gsdp5mQFbpwzMaSgd+5ep2DT4DQWcTdNkTaaZ/NvTYgl7Lb+JH7ylH
LFN9MTjC06omn2Y1Yil5eEWXWA+ZMjWk9BX01IWkCfqsmoyWmp5TCboraZYm
aGXmx03m5g2PKONTFXRpEo4l6GPOhW7kPOQRlOZB40iC/ln9/HneH8Rczdkp
SZfcX0re7e25JvOhzdKxc5Zs67pOqRxL6pV2G5R0Mxy0kljwuMmSfeE52q9D
EXfJOXXOovvLzYOO1AmvFXvdbtrONXF3HZObw9y1PXXuQ5u4283buabrWBN3
qn7XedAoqfRxk1flj0hu6geDrV7ugpbh4LxTTjXu1cQdf2P1k66O16wNALhE
Qcen/e6EmawTjqVq17+uru/az5z4qH993TpPcPz2mKmGYfUcJHa1/8CMZtsV
QQMAMrIZuMpcuaC32xtHRetkHp28rX93ty/gu7u7z/PUzosk7KlSdM9O28hg
sTtqBgDk5XzIUrDFgm4n3FidXOJt6n9IuNHazx931Y+X1LU6Evn8gq722t+X
Mr0KACBBbxJyW9BX5ctTN+es/1xX2nbfZq81rQd7CHpzM7vUHGvyBgBgaoYm
6HbNu6AleFf/w3XrfuiPc+1tD68zfUjtOQ86vlCJPmgAwNEFXQfHbB902Tlf
6n7oH3cbAYfn2SbzOXp3ywWduZuXO4IGABxT0CWjuAvP2V45s719Rjn3EDQA
AEsSdMk86IGCTvwce+TylHOhr7KpObxw7MEYUz8sAwCAC4SgAQAgaAAAQNAA
ABA0AAAgaAAACJqgAQBYlKCvis24/VoAABgvSURBVApB44R5f6puVg/Va8f2
14dV9fB65HsCQNAAQR9J0K8P1bdv36oVQQMEDRB0oaAfNvKsy56s13VX8X3R
88br1tdYrW6qp1cJGiBogKCLBb26earev+x7rR5qqb53nK9PXU3cAEH7CEDQ
+4m2XVaZJu50k3ct4VK5JuoSNEDQPgIQdGmCbiXfhLBDse9ta1WO1SVoAAQN
DBJ0W8jv1dNN87pOwa19nU3cBXUJGiBoHwEIukcfdGPW9r7w+PXr/bSdvFaq
LkEDaYofN/m1zvX6Ic+ZcwbPjny5C6YSr9f+mGeKMUEDA0ZxrwKpvj/dfDZZ
3zxUDzfpKVlFdQka6BB0KMdgca1knesq6uh1/evq+u5H9bNd//o6/pzoavt8
6LvpFE3QAIDJ+dvfpxb0dvv1VrDROj+rH9dB0g7OeXfXEvjHtru7u/i1cvdB
0ACAU5FzUyYVdDshx+psU3I6QX/Uf7nbCfznj7vqx0tawhI0AECCLhJ0nZDb
gu7op46ds/5zXak+1/Z1VPT1+RKyJ2gAwEUyNEG3a961xF0LeVf/Q8zrfuiP
c+1tj54l3adN0AAAgt5Zt7rK9kGXnfOl7of+cbcZ8V0g+qlauQkaAHAGgi4Z
xV14zvVUqkgSb38BkKB7vpkr3zcA4DIEXTIPeqCgUz9X24Fhu2vqgyZoLP93
71//9iEAOFFB/3qu7m8fq7e9jW/V4+1tdVuX++fqV/E+gsay5NwUnCb//If/
O3Cpgn573Ih2T9C/quf72+rxrfH3fXXbvMjuI2icV4Im9uPLuSnARQn67bEW
8331/BYm6Doht17vJezcvmUK+q8///DbJEFL3xI0cGIJOibZ+vVe03Ut5Q+R
/+rY1+L79+975ViCruXcFBA0SSPGf3//jw8BJyLoutk7JeHcvpkTdIl0jyXo
q9ff/AZL0DgROTcFuIgEPbagQ8GWinduQddibhcsQ85zJeghSyb6AhBnzqZt
csbpCHphfdApyZZK91TknDrm3EQ/5/uZM0EPeeiAlJ6W81yDw0oSdGofsRP0
EQS9Gal9v43FsVHc8X3zJeilSDkm6EPlXrL9lOU89/uZcxS3BH2cBH2oKLvk
HBO4pnGCPpKgm6R8mvOgj9HvPIZ4cpI+1QS7lOvPJelRnwqE0RLwGNeQoHEc
Qc/xZmYexR2T89TCHiMZ5o4fcv5zT+Slgp2jmXv05+pitgQNEPSIsl1iqj5U
fiVy7nP+sD45T388OQMEfTGCHkuspyDnruPHStAX9w/pwD5efcQACHqiBD1X
n/QU/c8AAIJerKDH4BRWEiNnXAqW7zxXflY/rr8+anJd9p7VPPZlez6+8mwE
3XuE9/IEfQrLfUrQC//ypJl7VDkv/QEYBp2dmDQvUtC9n3S17AR9CpLGMuW8
9MVETu0LxNLlfGnzmyd5r0XSrBP3dfXjx90uaV//+MjZL8HrvXN+pvK79clf
qrvdtvkkfdTf4GFPuppP0H1Ee6kPzCD88QS3dDnP+QXi3JunL03Qk73fYkF/
iPXuZV/Ae6+bc9Qi/pD5z8j5NXG3Xh/5aVZDhHuJctZkvjzBTXWNOeV8Cc9n
vrQm7qMn6J+p14GU96j3EXT5k66OnKA9UlKCXrLgzmXd7aFynlLqYwtGH/Q0
gn65aw0aW6fk/oLeOwdBD0vQUwt6jFRNsphb+Ocw2Kwt2lLpxpJ3ybEldcZu
ou3z8AwinzNBb/qam9ZvTdwxQR+pD7rPs5/PJVlrpj5NEY+VlJci81CqsVIi
1K5jw3P0aU7vK+iuuqUPz/CwjBkFHZ5v/Tqx73IFPf/TrIY8+7mpG9t2Som4
Xf/QVcS69i3hi8CUa5kPkV5OtDkR95V0U6/9Z9e1S+/zUDmnJBzuK5Vv6f4h
CbpElmHdIYKVoI+RoOtTXrfmVd9Vd9dNom7mX1/IKO4qm5DnmwcdE25qe2k5
lmQOfVZ0n+NzdWP7htxbe43vIYINjy9d7rTr2NT9pKRXIuCUEMN97bW4SyQd
nidWuq7bdczYks4l4T7bY5JObQtlGAq2KTmhh8fnZF0iYAn6wlvPzurNjLDU
Zx8JH0vSXZIZaznQzqQ44D76yjVXhhxfkvpLy5f6BcIbmqDbcm7KWHJuH5/b
P6Wkc3I9tJSk9abEmpb7yDkl067zlh4Dgr5YQR8q6fA8Uwt6yP5SoZY+AKPv
dUofTZlKsTnxdqXhPl84umScvLeM7FKJOrc/JuIwQecSeqppO7xu132n/uxq
Iegzqrq0KTusk2vaLhF0mKDD5BtLxyUJuqvJuku85AyCHjk1z5meu5qWczLs
8wznvsfkmoKT6bPwkZV9EnWf5N07Lcfe34EpNRRnKi3HtpX2U5cm6pKm8K4W
gqHzmadK0SX910OS7hgJurQ+CPqiE3QsCZ9C83ZMhKkUmetjzSXYnHBL+5r7
pNI+ohx6nSHpeSek8D33aBLuqtOVoEu+AHQ1WafkmvvSUCL+oQm6S9C5pDw0
QYd/5sRYIstcv3NXgg7PSc4g6IkS9BSSLk2sQ9Pg0POV9hUPTahjXmeU64Zy
CrZNVdqJOiXppZShTNkPXVKvJEGnpNn32D5fBgiaoJdJ4RzoKRJ0e19fkU+Z
oLu25xJ07JhcX/EcCTp13S6hxt7rHAm6nXBzCTqVRvsk6Fjf85wJOnbOPgl6
TElfcoIGQS+P6JOuphE0cOzWH+XKv2HgFASdftLV9Al67GbukkRdOiq7zyCw
PvN4+/QFdx13aFPy0Gb7Pgk61erQWaaUU5BIY0l5rObtrvQeLdtWglTrQFeK
Ll0BbCrpdzV7t1Nrqrm72Z5KwkMTdGzEuCZunHQT91hPs6oFesyUMHTU8pj9
t+dQZkmiqetO2N9csn3ykuiDL5V0n5W/piiH/r03ch7aVz1m/zQI+iQEfcoJ
WlPmtF98JkvQuRaMjn7kkn0S9Ne+4bFWEhuaoMf4nZSgQdD6rwAs7T9Z/fAg
aIIGcJ5SB0FftKCnauIumXrV1RzbHD9FP3TuuNS+2H2POUjs0Kbmrs90lPtq
LVTSfl06xalv83PTrH20Ju4Bc71rYs3JMcZYEWyqJu4hc5b71impC4K+SEHP
IeeUpEtkUCJoZbqFVrokPXQQ1SGLlMw6SOyA99T3ARbHLG05H3swGEnjNAQ9
ExK0BC1BS9ASNAgaAAAQNAAABA0AAAgaAACCBgAABA0AAEEDAACCBgAABA0A
AEEDAACCBgCAoAEAAEEDAEDQAACAoAEAAEEDAEDQwHnx/lTdrB6q1/Dn3etV
9e3bt3V5ePVxASBo4MiCfq0eVjfV03u7Xus1ABA0cARBvz5UqyAyvz6sNim6
rnfzUD3crKrV3rHSNgCCBiZO0CGtRL2V8aeEY2k7dR4ABA1gJEG/V08fafmm
MXCngGthEzQAggYmFHQt25acM4Kum8CbJu4VQQMgaGAiQX9pyk4JeiPxXT1N
3AAIGphI0Ck5J0UevjbiGwBBA6ML+v3pZtdc3S67UdxBQm7XX21HeBvJDYCg
AQAgaAAAQNAAABA0AAAgaAAACPoI/Hqu7m8fq7e9jW/V4+1tdVuX++fqV8mb
ufJ9AwBA0OPw9riR8J6gf1XP97fV41vj7/vq9vFt8YL+688//EYBAE5f0G+P
tZjvq+e3MEHX6bn1OpqwlyXoWs5NAQDg9BN0TMD1671m7VrYHyIP2rm/f/++
VyRoAABBTynoutm7QNBLS9AAAEjQCxK05m0AwGUIeiF90H2ES9AAgAsQ9GYU
9/02Mh9jFHffVCxFAwAuQNBNij7uPOi+siXngb+Er7/5EABgkYIe681YqOQk
5dwUAABBJ9NvmILD7al6zTYp+jwT9NW//j1KnTH4298v4/fin//whRsEffGC
bvcftyUb2x6rF6uLM/q9+hBvUw6pM5acm3Lucm4KQNAStAQNCVqCBgh6iYIO
03Mo3VyKzqXrk/oM9QljxC8t5D8O//39P37hCHpB7B6iET5IY3xBlzRlDy2n
KmeSRpec52jSn0rOp9R8Xsu5KSDo4xNMu5pjHrQELUFDgpagQdAHCntKQY+V
qNuSP0SWh+w/5HrNz+E1Sq7ZV/Kpekv+knDKciLQ40p0SD2CJugF+zmeoMd6
mtWUTdxDJd0lubGTbqx5O7xGyTX7NpOn6i05yZ9y8+6pyPnURnCXNkMPqaeJ
m6AXnJy3z4wuWEpMgpagJWgJWoIGQc/K8h83CQDABQq6HtR9W3WNEyNoAABB
T2vj4AEZ0yfoqfufD2nubs6V6qsNf441T6d+DuunmrZzx+b6kHPHl/Sxp65b
sr3r+D5N6J3dDRP1S8fO2SxUMvRa7ePa9x1uTx0zB+0m7rA/ut1HnaqXayIP
9+Xqhs3MXXX7NkXHjmlea+LGYiPnemDYbh70tH3QUw8SO2TAWPvY3KCtIaXk
HDkZl54zdnzpdfrcT5/j+/STdw7YCyQ3lsxi52wv9TnkWrF7zZUpv3zkBDpW
yZ079jomz6b0qdtHzu1jUoPDDBIj6NN/MxK0BC1BS9ASNAj6vAQNAABBEzQA
AAQNAABBj03hMp+HCro+tikAABB0jt0TraYXdEzUZA0AIOgvbt5Or3qbJ0HH
RmCHsl7aoyZzo7Fj+1PHxo7LTY+KbQ/PPWRKV27d7666qfdX8lnEPo8hy4ym
Rjq3pyvltnWdt/6zmWKVOl/s53C6VOrc4T2lzlPynoeQGqkd7guP6bMsaMko
79T+kpHUuTrh6OzYFKvUcUZxE/QyyTRxz/mwjKGynlLOpfLLHTtmmfLcpdfv
+1kMqZ+SaEqApfONu87bFnTf+cxd85v7Hpt7z0Pl3Hdec9ec55JrdNULJdkl
4FSdrvnNqWPNgybokxX01An6UFlL0BK0BC1BS9Ag6Oo4o7j1WQMACHqBgiZr
AABBL1zQKVkDAEDQS/yQyRoAcHaC7inCU7hHwgYAEHQhx3rM5J6wMyOYm3PM
OTUqtW937yOca+wpVrF7m+S+6lHNzfHNdKNme8c0pty+cMR2e3rVHNOskvfW
em99rhF7tGOMPtOs+mzP7Q+31bQfLVk6JWqMOiV1QdAXKehjPgt69x9+kKwV
RVGaQtIELUEfIUF3pT0JWoKWoCVociZoAABA0AAAgKABACBoAABA0AAAEDQA
ACBoAAAIGgAAEDQAACBoAAAIGgAAEDQAAAQNAAAIGgAAggYAAAQNAAAIGijj
/am6WT1Ur+HPg86zqr59+7YuD68+WgAEDRxZ0K/Vw+qmenpvn7P1GgAIGhgq
6FqymwS8Wr9+r55uPiT79LBLxTe1cV+D12s/P1SrIDK/Pqw+U7R0DYCggbES
dC3o1ad4t5Ldf51K2+1EHUvXA5vRARA0QNDbBL1rpg5fB+Kt2vVWn+k6Km+C
BkDQwIyC3jSTx+RcN3k3TdwrggZA0MBMgt42gX/tX95Ie78/mqABEDQwvaCT
co4I2QhvAAQN9BR0MzCsPYq7QNDvTze75ut2+RxP9rl/dfNQPdysjOQGQNAA
ABA0AAAgaAAACBoAABA0AAAEDQAACPrgN3Pl+wYAgKAJGsBF8N/f/7MrAEET
NIAFiDmUNEDQBA1gIak5LABBEzSAIws6lDVwWYL+9Vzd3z5Wb3sb36rH29vq
ti73z9UvggYwMv/8x1Vneo6laeAyBP32uJHwnqB/Vc/3t9XjW+Pv++r28Y2g
AYwq56b0SdAEjYsQ9NtjLeb76vktTNB1em69jiZsggYwvqTrn3P9zwSNy0nQ
MQHXr/eatWthf4g8aOf+/v37XiFoAIdIul0IGgQdE3Td7F0gaAkawKFybv9Z
y3eInGPbiRwXnaAJGsAYybkt1b6Sjm2XtnG+gtYHDeAIkm7k3G7mDoVbmpbJ
GWcq6M0o7vttZDaKG8CUkg7F2k7QQ1JxqZxLz0X2BL0gQTcp2jxoAPMm6NLS
JdQuqfY9F0kT9Gm/GYIGMEKCDpNrTJRdYpWgQdAEDWAk+iToOcQpQRM0QZ/i
e339zW8vUJCIhwixZBR3LEmXTslKHStB4zQEvVsCNFwGlKBrOTcFwL6cc8t3
jpmeS4/pew0JGssWdDBozChuCRqYO0HnhBoeI0HjshJ0RtgSNDkDh8i75ClW
ubScOy4n/77bpWeCPgE/xxP0Ja7FrXkbGC7nsJTIuStFlwwc67MKWWlyB0Ev
IDlvn3hVMBFaggYgQYOgZ2X6tbj/+vOPdQm3dR3TZ/s5yn9I/ZIWgKm/hMTO
n7tm+56v/vXvef9Oguv97e/DztGUcFufax9brLltKfm2t8fqpJb6DOVZ2rec
k3v7nLl51aljQdDLVPTjbdU1TmyooBs5tyUdvs4d05Zy13FLlnJbniWC7Nvc
Hp4/ddzUzfix8+eu+eW+E2I75H6b84XnDcVay7kppQJtnyNVSqU+p4BDgcak
Gsq3ZHvJ4yX7PnJyyPzppc25BkGX2njWx02mpFqaoFOyPhU554oE/fWeU3Ie
+qWiS5jt121B9xFoLkHnjp9Lzl1To8ZM0CmB9xXtVHLuSuMg6KOzHhi2mwc9
fR/0oVI9l2btdppGvy8MUyToGO0m7kMFuuQm7Cm/DOSunWpaLumDTm3LLXBi
DjROTtCD3oylPgEABD1blC6aA03QAACCnovdcp8EDQAg6IW4edv3/DZPgk6N
vA5HeKcGg8X29/7LmKHfN9XHnBrkFNZP7U+da29wVcH22HVzdYunRnUM4EqN
6I69/76DqnL9y6UjqdvEBojlBn+1/wy3xwan9RnpXfL+h1A6NSrWn5xajCR2
rrBOrt946HrcJedqtod/6oMm6GWTaeIeayWxlFxTci4tfcU59Qphh4izj1iP
UUre65C6qSlWpbLNia5k9HZqmlXJ1KkpSpecx5iW1Wdq1NApVV3TrJZQ6v/P
mkLSBH1ygpagJWgJWoI+5QTdlnBYJGgQNABM+Z9sRsL+zwJBA8AR5Ov/JBA0
QQM4gogBgiZoAEdOwwBBH/gPbCiHjtg+ZFR3yUCo+vipRj/33Re776HnmmMU
9yT3VQ+caq3N3bwuHWzVNVK6/rM9GKz5eaxR3Kl7SN5bMIK99BqxUdYxDh2t
3XWOZn+XgFMPymj2HTpIzDrcIOgBci7pS1IU5fTLqU67ImmClqAlaAlagj7b
BF3yNCsJGgR9hD6kXGkWOlGmKz5jn7PPef4Cgj556n9s8Bn7nOFzBkH7x+Yz
hs/Z5wwQNAAABA0AAAgaAACCBgAABA0AAEFfHG/V4+1tdVuX++fql9+HcYmu
o+4zH/fz3X6WH+Xxzec8zX8Tj7vP+NbvMwh6lv/dquf7z//Ufj3fV7ePb34j
Rv9Prf0fms983C+X99Xzr7asm9c+56m+ZO5/lj5nEPSE/8G15NHjqVnocnMt
5g9ZvIWfqc981C9AgQzqz32zyec8j7B9ziDo6f6h7TVJBYkEo6cPn/lMidrn
POGvdCsl+5xB0BMmEP+45hW0z3yqD3rd1HrffJA+54l+l7ctQ7/8PoOgJWgJ
GkXJuSVnn7OWChD0efxD0380q6B95pOkuq/jknzOk/7Poa8fBD35/257zYJG
YM4haJ/59HL2OY9s40wzts8ZBD1xijaHcT5B+8zH+2jvW3NzY3Ohfc7TfNZh
E7bPGQQNAABBAwAAggYAAAQNAABBAwAAggYAgKABAABBAyfCy111dXVXvZTW
//lSvfxsfv5RXV9dVz9++hgBggZwPEETMgCCBggaAEED89AI7kctxatNuf5R
/fxS5+pz/0e527Pnz+rHdXt/S64xgX7Z9lLd7Y5t7qUt6OD8zcWD+7quT9g+
d/PzS6regPcDgKCB+QTdltBWTnsSjAm2kVpQf737+lPynYLeyPl6V6GRdeJ+
uu7vi6D3v3Cs7y26LXW94P0AIGhgTkHvJcgvksskyGhz9Eay63N2CTpy/J4w
Y+dvbysQ9Nf3lnm/Xe8HAEED8wk67MPdF9LLXaR5OUykkfKlyTlyzWg6bX8p
qIUZ3R80Y/8s2FeQ6DvfDwCCBpYi6HB7u1+2s/n3FAWtORsgaGAZgo41+eYG
RrUEvm4SzoyiHtDEvbctdv5eTdz9BN35fgAQNDCnoJODpGIC3ZPctn47de5J
LhwE1oyQ3t//2XTec5BYmPYPFXTn+wFA0MBsgg6mWYVt2y93QX9sKKtwWlJ8
1PfnlKZaqiNMs9rdXmv7wYIueD8ACBqYTdAEBICgAYIGAIIGCBoAQQMAAIIG
AAAEDQAAQQMAAIIGAICgAQAAQQMAQNAAAICgAQAAQQMAQNAAAICgAQAgaAAA
QNAAAFwU/x9PXhN+w9c0eQAAAABJRU5ErkJggg==
"" alt="On the left hand side scatter plots of expression of the specified genes in pseudotime. On the right hand side also the expression of specified genes as a function of pseudotime but shown with the cell type data superimposed. Again the trends are consistent with the previous biological interpretation and assignment. " width="488" height="550" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 17</strong>:</span>  A visualisation of how specified genes change in the pseudotime.</figcaption></figure>
<h2 id="regression-analysis---advanced">Regression analysis - advanced</h2>
<p>We will use a function which fits a generalized linear model for each gene in a CDS object. We have to specify the model formula which is any term that exists as a column in <code style="color: inherit">colData()</code>. We want to test genes that differ between cell types and batches, so we will use “~cell_type + batch” argument. Then, we extract a table of coefficients from each model. <code style="color: inherit">coefficient_table()</code> tests whether each coefficient differs significantly from zero under the <a href="https://en.wikipedia.org/wiki/Wald_test">Wald test</a>.</p>


In [ ]:
# fit a generalized linear model for each gene
gene_fits <- fit_models(cds_subset, model_formula_str = "~cell_type + batch")

In [ ]:
# extract a table of coefficients
fit_coefs <- coefficient_table(gene_fits)

In [ ]:
# preview the content of 'fit_coefs'
head(fit_coefs)

<p>If you inspect the <code style="color: inherit">fit_coefs</code> object, you will notice that the table includes one row for each term of each gene’s model. We generally don’t care about the intercept term, so we can filter it out. In this way, we will be able to control for the chosen factors. To focus on only one variable, you have to check the <code style="color: inherit">term</code> column in the <code style="color: inherit">fit_coefs</code> and pass this as an argument for filtering. Then, you should also filter the results with q_value &lt; 0.05 to control the false discovery rate.</p>


In [ ]:
# filter out Intercept term
no_intercept_coefs <- fit_coefs %>% dplyr::filter(term != "(Intercept)")

In [ ]:
# extract results for DP-M1 cells only
DP_M1_coefs <- fit_coefs %>% dplyr::filter(term == "cell_typeDP-M1")

In [ ]:
# control the false discovery rate and choose only several variables to store
DP_M1_coefs_filtered <- DP_M1_coefs %>% dplyr::filter (q_value < 0.05) %>%
  dplyr::select(gene_short_name, term, q_value, estimate)

In [ ]:
# view the resulting table
DP_M1_coefs_filtered

<p>The resulting table shows the genes that differ depending on the chosen term. Maybe this function is not very helpful in the case of our dataset, but may be useful when analysing unannotated data or choosing another term from <code style="color: inherit">colData()</code>.</p>
<h2 id="graph-autocorrelation-analysis---advanced">Graph-autocorrelation analysis - advanced</h2>
<p>Alongside regression analysis, Monocle also provides another way of finding genes that vary between groups of cells. The function <code style="color: inherit">graph_test()</code> uses a statistic from spatial autocorrelation analysis called <a href="https://en.wikipedia.org/wiki/Moran%27s_I">Moran’s I</a>, which <span class="citation"><a href="#Cao_2019">Cao <i>et al.</i> 2018</a></span> showed to be effective in finding genes that vary in single-cell RNA-seq datasets. Let’s try to perform this step on our full dataset (be patient!).</p>


In [ ]:
# run autocorrelation test
graph_test_res <- graph_test(cds_order, neighbor_graph="knn", cores=8)

<p>The output data frame has a bunch of statistical values that you can use to rank the genes by, for example <code style="color: inherit">morans_I</code> column, which ranges from -1 to +1. A value of 0 indicates no effect, while +1 indicates perfect positive autocorrelation and suggests that nearby cells have very similar values of a gene’s expression.</p>
<p>We will now try to associate genes with clusters by grouping genes into modules that have similar patterns of expression. You can call <code style="color: inherit">find_gene_modules()</code>, which essentially runs UMAP on the genes (as opposed to the cells) and then groups them into modules using Louvain community analysis.</p>


In [ ]:
# get gene IDs for which q-value < 0.05
pr_deg_ids <- row.names(subset(graph_test_res, q_value < 0.05))

In [ ]:
# group genes into modules
gene_module_df <- find_gene_modules(cds_order[pr_deg_ids,], resolution=1e-2)

<p>Now we can show the aggregate expression of all genes in each module across all the clusters. Monocle provides a simple utility function called <code style="color: inherit">aggregate_gene_expression</code> for this purpose:</p>


In [ ]:
# aggregate expression of genes in each module
cell_group_df <- tibble::tibble(cell=row.names(colData(cds_order)),
                                cell_group=clusters(cds_order)[colnames(cds_order)])
agg_mat <- aggregate_gene_expression(cds_order, gene_module_df, cell_group_df)
row.names(agg_mat) <- stringr::str_c("Module ", row.names(agg_mat))
colnames(agg_mat) <- stringr::str_c("Cluster ", colnames(agg_mat))

<p>We can now use this data to create a heatmap. Don’t worry if yours look a little bit different that the one shown in this tutorial. The general features should be maintained though.</p>


In [ ]:
# create a heatmap
pheatmap::pheatmap(agg_mat, cluster_rows=TRUE, cluster_cols=TRUE,
                   scale="column", clustering_method="ward.D2",
                   fontsize=6)

<figure id="figure-18"><img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAfQAAAK8CAMAAAAu670kAAACDVBMVEUAAAA6
Ojo6OmY6OpA6ZmY6ZpA6ZrY6kLZFdbRJebZOfbhSgrtXhr1ci79gj8JllMRm
OjpmOmZmOpBmZmZmZpBmZrZmkJBmkLZmkNtmtttmtv9pmMZunclzoct3ps58
qtCAr9KFs9WKuNeOvNmQOjqQOmaQZgCQZjqQZmaQZpCQkDqQkGaQkJCQkLaQ
kNuQtraQttuQtv+Q27aQ29uQ2/+TwNuYw92ZmZmcxt+hyuGmzeKr0OSw0+a0
1ui2ZgC2Zjq2Zma2ZpC2kDq2kGa2kLa2tma2tpC2tra2ttu225C227a229u2
2/+2/9u2//+52em+3OvD4O3I4+/M5vDR6fLW7PTXMCfZNSrbOy3bkDrbkGbb
kJDbtmbbtpDbtrbb25Db27bb29vb2//b7/bb/7bb/9vb///dQDDfRjPg8/fh
8/TiTDbj9PHkUTnl9e3mVzzn9eroXT/p9ubrYkLr9+PtaEXt+N/vbUjv+Nzw
+djxc0vy+tX0eU70+9L2flH2+874hFT4/Mv6ilf6/cf8j1r8lF38mWH8nmT8
o2f8qGv8rW78snH8/cT9t3X9vHj9wXv9xn/9y4L90IX91Yn92oz+34/+4ZL+
45X+5Zj+55v+6Z7+66H+7aP+76b+8Kn+8qz+9K/+9rL++LX++rf+/Lr+/r3+
/sD/tmb/tpD/25D/27b/29v//7b//9v///8jNwiTAAAACXBIWXMAAA7DAAAO
wwHHb6hkAAAcR0lEQVR4nO3dj38cx1nH8fPZTviN5JNrRfwGh5PrlMZwsEBp
JOeAAFWPH6UFq3Zst/VBSyihhJASQgmltIUATdqUpk2aNE0iztHu38jtSTrd
nLWzs8/Mzu7sfr6vVxL7pOfm5Lf3pNxzz0wnIa1Lp+oHQPwH9BamFuid0FL1
H5hlavH4a/EgCiS0x7ucWjz+WjyIAgnt8S6nFo+/kgfR4u8M7X0Q1Xzl7f3z
Xg7oftPeBwF6tQHdb9r7IECvNqD7TXsfBOjVBnS/ae+DAL3agO437X0QoFcb
0P2mvQ8C9GoDut+090GAXm1A95v2PgjQqw3ofuPpQdTwDUag13gZ0J0HdL8B
vezSSu63jqsuBXS/Ab3s0krut46rLgV0vwG97NJK7reOqy4FdL8BvezSSu63
ji9ELT/C2i8THnolpc1aBnTnqf8yoDtP/ZcB3Xnqv4xNaSU/UoFuvwylzlP/
ZSh1nvovQ6nz1H8ZSp2n/stQ6jz1X4ZS56n/MpQ6T/2XodR56r8Mpc5T/2Uo
dZ76L0Op89R/GUqdp/7LUOo89V+GUuep/zKUOk/9l6HUeeq/DKXOU/9lKHWe
+i9DqfPUfxlKnaf+y1DqPPVfhlLnqf8ylDpP/Zeh1HnqvwylzlP/ZSh1nvov
Q6nz1H8ZSp2n/stQ6jz1X4ZS56n/MpQ6T/2XodR56r8Mpc7jbpmIlJyCIB9T
cnjjtQ236M+KEz0lTvSCNNFfixM9J070BXFcoE9WQG80+p8pmd0UX7oOetvQ
dwd7oDca/c+VpLfEw97K6THogaHvnR0n+6ujgz/x+NIUcHr1Hglc24j7Z65o
0NM74EoPD707mP4zSuJPTM3vHKGnv5v9kHZtED84OgL5CyWLVAXRNbvtge4D
/T3rydXN0d7Z3zl7/fzw9HAQXxoOpr87f/RD2v65sXv07A+B7gP9gUufWt8e
bY+Sre4oBU//2brw2MXBwQ9p8cX5k33yl0pADxd9Y/fyYBl9+1duPz6e/ZD2
qdWdYxDQG4M+OT3eHk1W+/dNVi+fvt7tTZ/ip7+6L5l9cOtMb23+9P5XSkAP
Fr1IQG8huvqYQQfdFTopOQXR/0ZJSeivihN9RJzo69JE3xYn+qw48memF0AH
PTd/qwR00EFvJvrfKQE9WHRNly3tsCldNtCbgp7dZUs7bO9d7LL9vRKX6Atd
NtA9oGd32Q46bAtdthLR578C3Qd6dpdt1mFb7LL9gxLQw0XP7LJNv9XvzP4B
vXnomV22tMPWX+yy/aMS0INFLxLQW4j+T0pAB73Q4cPHhcs3VN2Dan4Kov+z
knvQze+os/yrBXSLTvEHxYn+R5roG+LYfKmviwM66Ln5FyWggw56M9H/VQno
waLrumybZ3fiYff4JbkS0Y8bLqB7QM/usu0O9s8NNw7+Iszyb0pcoh8HdB/o
mi5bPFzfGiTbI9Cbhq7psiW31jZHC+hfUgJ6uOjZXbYb09t6g3gT9OahZ3bZ
prfdf3e1vz4H+XcloAeLXiSgtxD9y0pKQiclp47onxYn+qQ40X9JY9NafUsc
mwZ0QfSvKAEddNCbif5VJaCDDnoz0f9DCejBoufMsiXJ1ZEv9IMuG+ge0PWz
bB/ud4/R/1OJDfoJb5E8vAPQfaDrZ9nujI/7LSehH061FkRfggbdN3rOLNtC
ky1Rv9zZTYd7x4IeFrp+li0P/XDvWNADQ9fOso0X0f9bycFtB1OtoAeFXiQn
oB9OtYLeWHS1qTi76XDvWAv0pZ/kq+5BNT/26HM7ObpyJ0n0UXFsniTkTfFX
xIleE8fjlf41JaCDDnoz0V9UAjrooDcT/SUloAeLrj2X7eoo3uxdmI81lY0+
67KB7gE9u8sW97ujyfqCkLrbinP0WUD3ga45l228PdrrrjwAeuPQdbNs26O7
iy++q18u6OGiZ3fZUvSP7yRb82/xoDcGXXMu2/T2bv/+OcjLSkAPFr1IQG8h
+jeVlIROSk4d0T8vjs05fl+RpqJtQi3O8Sso9L9K3KB3QG8d+j1loNcM/VtK
QAcd9Gaiq//HAXqw6Nou27UNT+gLO0aC7gE9u8uWTFZUdPUxO0Wf/xZ0H+ia
Ltul66A3Ez27y7Y72FPRv6ME9HDRM7ts8bC3cnq86AN6Y9A1XbalK1195R70
YNGLBPQWoqvtvZLQScmpI/rnxImeECf6sjQ2TXGLJ7XviVMU/btKQAddjL50
AiPo9UJX95B2hD6vBx100MtDz9k8cLfXW5uf0fWGEtDDRddvHjhKJufnIOWj
dzqg+0DXbx44Xji3J1GrS0HnSveCnrN54N7CBCPoTUHP2TzweKgpSd5UAnrA
6NrNA++km4CC3jD0Ivm+EtBBB72Z6OpWlU7R55tGVt2Dan5qg358PzZN5s+I
E31VGpt+usUJjG+IUxT9bSWzm+Jhdwf0lqHvbsxeugW9sejvKJndNP3f+HRP
GtDbhD4VBz04dN1Y07B7RSF5Qcnspt3B7MV50MNCz+6yTb9fP6S87/0E9Lur
/fXEIXoHdB/o2V227cvdgULyRSXqFeoGPeFK94Ke3WV7ZCNefOUd9OagZ3fZ
Dn8yP87zSkAPGD2zyzZZ3bxPAQG9IehFoi4MOuigNxNdnYQrCZ2UnDqiPyVO
9LI40dPSRO+IY3O5WqxaEEQd7wUddNCbia6+ZQd00EEPB103y7Z5dkfptD2j
BPRw0TVdtsH+ueFip61E9OMdI0H3gK6ZZYuH60qnTf1y3aIf/QJ0H+i6I7pu
rf3iYqcN9KagZ3fZbkxvO7XYaVP/jEEPGF3TZevfr3TaQG8IepGoU0Sggw56
M9GfVFISOik5dUS3+Dv8kjjy1rbN47XYy9biXQcFZdRq0EEHvZno6iA/6KCD
Hg56XpctuTqay6hvJiyC3skN6D7R9V22O/2uE3TNx44+4bDLBroHdH2XLR4v
DLmoB807R5/9AnQf6Pou2zgBvYHo+i6bMs72SSWgB4yu7bIloDcQvUg+qgR0
0F2hk5JTEP1PlZSEbrF35oviRM9LYzOFKF70eZtS0EHPy0eUgD77MwQd9Mah
/4kS0Gd/hqCDHgR6zrlsB522w/yxErfoaaNt1nAB3WRVi9Ik91y2y4P9c/NZ
thLRZ5+TfhboRqtalCYG57INj8/o+iMloM/+DINEzzmXbdZpAz37zzBIdP25
bAedtsP8oZKjW9Nv/aAHhq49l+3mrNOmRU+/9YMeFHqRfFDJ8e3750BvE/rj
j+2k3/odopOSUxD9t5TMbnr39jj91u8Q3eLvsEU//TlporfEsemnWzwf2qMn
B9/6QW8s+geULKOCDjrozUD/TSWgg26P3gHdE3ruLNu1jTnLbyhxjX7QagPd
B7p+lm08WfGJzpXuBz1nlu3S9WP0X1cCerjo+lm26V8A0JuHrp1le+RMb+X0
vLX6a0pADxg9Z5Zt4UoHvSHoRfKrSkAH3RU6KTkF0X9BSUno4mvuOZt+unzR
t8Wx+VItSkEHPS8/rwT02aKgg9449J9T4h49Ad10VYtS0NuJrumypbtF7vZ6
a/Nhtp9VUgZ6pwO60aoWpYm2yxbPdoucnJ+beEDnSjdb1aI00XbZZrtFxpuj
ucnPKAF9tmiQ6Lou2/TWveP5RdBPWjRI9Owu2wz9eJItSX5aCeizRcNEz+yy
pegL5y+CfuKiIaIXyU8pAX22KOgO0EnJKYj+k0pKQJ9+nk2TuZIr3WJvWIu3
DliUgg56Xn5CCeigg95M9B9XAjrooIeDnrNjpDLL9mNKSkHvgO4DXb9j5Ghx
lq189IQr3Qu6fsfIO4uzbD+qBPRw0fU7Ru4uzrLloRucqgl6LdC1O0bGw8VZ
th9RcgJ6JiXoNUPX7hg5XpxlA70h6EXyw0pAB90VOik5BdF/SElJ6N8WJ3pV
nOhpaaLXxYmeEsemFHTQ8/KDSkBvNfq1DdAbi/4DSuY3T1ZADws9Z/PAuL9w
RNfJ6OmuY6CHha7fPPDRwcFfhKw8/thO+kqtI/R0lg10D+j6zQO3BunEQ3be
vX0nfaXWFTpXuh90/eaBN7u9rg59FndXOuh+0LWbB249PF6cYNSigh4Qunbz
wMlafz3XCvTA0N0EdNBP/LjZ3YDeRnRSctyIJ07RLS7XV8SJnpEm+r448kWf
iZ4QB3TQLSJDP+n9k6A3Hf2EG0AHHfQy0HPGmuJhd2eZBvTg0fVjTZc39F02
x+idDug+0PVjTf2cLptjdK50P+j6sabp7aA3D1071jR9qi+xywZ6ZejasaZb
qyV22UCvCN1NQAcddNALoZOSU0f018SxudLl7ek3xZE38Z+2KQUddNBBBx10
0BuDruuybZ0ZLLOA3gh0TZfNrMPmEr3TAd0HenaXbbI2NHkJ1iF6wpXuBz27
y7a7fmvN75UOuif07C7b0gFdoDcIXTPLdnBQF+hNQ3cT0EE34gUd9MNU3YNq
ftyIJ07RvylO9EVx5Oco2kzZflacJvXTE9BBBx100EEHXYqed0TXsHvFF/ps
WBl0H+i5s2wP+Zplm/0WdB/o+lm2zctdk5ffQQ8LXT/L9shG/OAI9Kah62fZ
8vaGBT1MdO0s283VTZNOG+hBobtJLrrJuYygNws9u2r5t1W3I5ofN+KJU3SL
K118ot5z8s0+ozfEiT4tThDbhGZXLf8WdNBBbyi68nMd6C1BX/wt6KCDXga6
psuWJFdHXtFtu2ygm6Jnd9nifv5BTU7R03+B7gM9u8sWj01edgc9QHTduWyg
NxQ9u8sGenPRM7tsoDcV3U1AB90GnZScOqJbXOlviWPRT7cYYLS40kMYYAQd
dNBBBx100EEH3Tl6TpfN0blsZui2XTbQTdH1XbZdN+eyGaHPAroPdH2XzdGE
C+j1Qtd32RydywZ6vdD1XTZH57KBXjN0bZftrptz2UCvFbqbgA56gapkGZ2U
HDfiiVP0r4tjMwUqb4pbPL1YNMWfFQd00C0COugFqhLQQQcddNDLQM+dZTOa
Z3OBfjCuDLoPdH2XzXCeTYiu7Dd08B/QfaDru2yG82wy9IVi0L2i582ygd5A
9LxZNtCbiJ4zywZ689DdBHTQixfP/1N1D6r5cSOeOEV/URybJrN8V9kviGMz
ZWtR6kY8AR30wsWLr9GA3hb0+X2ADjro5aDnjTX1z5Y31gR6Vej6hsu1QYlj
Tfegp1020D2gVzjWdC86V7ofdH3DZdLtmfRWQQ8LXd9weXdc4lgT6JWhaxsu
k7USx5pArwjdTWboy0dugd589IQrHXTz4sPq2VNF1T2o5seVuRP0gzuIXhIn
+rw40QvSyCtfiJ4Rx+ZLdQJ+qJWADnqBYtBBBx100J2ja7pscf/MFdAbiZ7d
Zbs2iB8c+UXvdED3gZ7dZUuS/XMGbVWH6AlXuh90TZctvjjIggI9aPTsLtv+
qsmbZkAPET2zy7Z1prfG03sT0d3ECj0BHXTQW4dOSo41tsrmBP0b4sg3eH1b
PntqMyprMXpqsao1tsoGOugFqw8COuiggw66E3Rdl23z7E682btgONYEejjo
2V223cH+uZsm73p3gH40o94B3Qe6pssWD9f3uisPeEE/+jXoPtB1s2y31m6N
jQcYQQ8HPbvLdmN629pOsmXQXwU9MPTMLtv0tvsn3f79Zmygh4PuJqCDXrwa
9Fajk5LjwvuILQn6SpcPgYqfI96y6ac37EoHHXTQQQe9MegLO5eA7gE9Z5Yt
3jpjMvBgh55wpXtG18+yvXfDeMdIB+jVddlahq6fZbu5NjTdUswFOle6H3T9
LNvu+i2TKRfQw0LXz7Klt5feZQPdO7p2lu3m4d6RoDcJ3U1AB93iDqpuRzQ/
LrxVM+s7iF4WJ/q+OPIup3yD0S/aXOkW5/i58FbNrO8AdNBBbzL60dFsoLcH
fX43oIMOehnoebNsZrtGukLvdED3ga6fZXvUbNdIrvSw0PWzbIa7RoIeFrp+
lm1stmsk6GGh62fZftls10jQA0PXzrIZ7hoJelDobgI66BZ3U3UPqvmxxl7Q
ShyhW/TT3xHHop9u0cS3uFwtzh20xl7QSkAHXXg3oIMOOuiggw66DD2ny7bb
660ZvCbnsMsGugd0fZdtnEzOm2i5QU+40v2g53TZ4s0R6I1Dz+my7RltDgt6
WOj6LtvAZJIN9ODQtV02w9NWQQ8K3U1AB90iVfegmh8HSAdxiC6+5p6zuXLk
J+qJO/HP2nypHxMHdNAtAjroFgEddNBBB90JuqbLliTXNpLk6qhU9M5SQPeB
nt1lSyYrG3G/WzL60u9B94Ge3WWLL13fiMcmJ3SBHhh6dpdtd7A3fXoHvYHo
mV22eNhbOW12Fh/ooaFndtnSD3KlNxHdTUAH/Z4Pa7KMTkqON/S8yuPYTIG+
KU70OWmiV8SJnhAn+pI4oINuEdBBz5PVfAh00EEvAz2n4RIPuyY7DYEeFrq+
4bLr5IguHfpylw10D+j6hsvWwOglOTn6ckD3ga5vuExvB7156PqGy/T3JhOM
oAeGrm243F3tOziBEfR6obsJ6KCf/GGjgN5GdFJyrLFVVSfoHxUnelKc6NvS
RG+IYzOga/GkZo2tqoIO+vKHjQI66KCDDroTdN3mgf0zV+L+2XK7bKBXgZ7d
Zbs2iB9876DkLtv8U466bKB7QNdsHpjsn+v76rIdfQroPtA1mwfGFweTbs9k
ghH0sNCzNw/cX91J3h176rKB7hU9s8uWHs91c81Tlw10j+huAjroWaK59wB6
eOimA2s6dFJyHKNLP7z4KdEnxYk+L458W1mbYx9fEsemi2+NbagKesvRc74D
gN5IdP2ngA466GWga7tsRh020MND13bZTDpsztAPumyge0DP7rIZzrG5Q0//
C7oP9Owum2GHDfTw0LO7bIYdNtADRM8+osuswwZ6cOhuAjroug+DDvr85sNX
YqvuQTU/jshdoCcOrnSLvVbl7ennxYleFMemi+/CO0fyng+DDvryR0AHHXTQ
QXeCruuybXrssoHuEz27y7Y72D/nrcuWOOiygW6Krplli4f+XoY9Cug+0DWz
bMmtNZ9XOuje0LO7bDdGaUsd9CaiZ3fZVvv3m5mDHha6m4AOuu7DoLcF3Xyu
reoeVPPjC73Ap0bfESf6rjgWJzB+QByb0VOLtw7YSZtLFvhU0EEHHXTQQQdd
hp7TZYu3zpi8KCdDP2liGXQf6Pou26NFzmUrjH7Ch0H3ga7vsk3WjDptoIeF
ru+yDdeNOm2gh4Wu77J1zTptoAeGru2yHZ3QBnqT0N0EdNANP/VEdFJyrLF1
fPmfeiK6RdPwKXEsDuOzaAVbbBP6LXFAB90ioINu+Kmggz4L6KCDXgZ63rls
m70LJb8Mu9RlA90Duv5ctt83G2yyQlc+DLoPdP25bNe7Kw+A3jh0/blsd8cF
DuMDPRR0/blsH98pvcsGegXo+nPZukbzbEXQl+ceQPeO7iYF0O+pAh10UnKs
sVU+WdUS+tPiyPf6fCd6Rhp55TPR18SJvicO6KBbBHTQC1WBDjrooINeArqm
y5YkV0fuZ9l06GmXDXQP6NldtrjfHV1zPsumRedK94Oe3WWL05dnnc+ygV4D
dN0s2/Zo1/ksG+g1QM/usqXo285n2UCvA3pmly1Fdz/LBnr16G4COuiFqpbQ
Sclx4b0MV7hqCd3iyrE4F0/+9PJZcWwO43tdnMrRl/cNBb356PfUgt4i9IU3
zYHeGvTjX4IOOuhloOd12czOZnOF3umA7gNd32UzPJuNKz0sdH2XzfBsNtDD
Qtd32QzPZgM9LHR9l83wbDbQA0PP6bK5nmUDvXp0NwEd9KK1i+ik5LjwPtKy
qV1Etxg9fVsc+V6r8nnXz9kcxveaOKCDbgdnUws66KCDDrpr9LyGS//MFdAb
h54z1jSIHxz5Q+90QPeBntNwSXx22RKudD/oOQ2X+GJ5U6ugV4Wub7ikGwiC
3jx0bcMl3UCQp/emobuJFbry1nfQW4GecKW3Hp2UHDuqk9Usy2366a+KY3Gl
PyFO9E1xbJ7U7KhOVrMsBx100EEHHXTQZeiaLttur7dm9Hoc6KGhZ3fZkmRy
vriaHP3gXDbQPaBrjuiKN0de0dN/ge4DXdNl2zM7iQ/04NA1XTaTgSbQg0TP
7LIZvWcG9ADR3QR00GXloLcWnZQca2xVzbL8AN1ig1eLK11+uX5GnOgVcQLe
G3apHHTQQQcddNBBt0DPmWVLkmsbHtCPt4MG3Qe6fpYtmax4QF+gB90Hun6W
Lb503SN6wpXuBz3niK7BHujNQ9fOsn142Fs5XfJYE+gVoGtn2dJPMPECPSR0
NwEddOGdgN5WdFJyXFAdeDm5kxn6c+LYXOnyDV7FzxFPRN8SJ3pDHNBBt/Fy
cieggw466KA7Q9d02dLjuTwc0aXeC+g+0LO7bOnxXMPyj+hauJe0ywa6B3Td
LNtw3cMRXYvoXOl+0HVdtvR4rtKP6ALdP3p2ly09nus95R/RBXoF6JldtvR4
Lg9HdIHuHd1NQAddei8dGi7lxwnVjMvNvaToXxDH5oS6J6Wxaa2+LE5zWqug
gw466KA3Ez0BHXTQS0LPm2Ubdss+l23xbjod0H2g62fZdjfih7y99p5+Uwfd
B7p+lm37crfsI7pA94+un2V7ZKP0ExhB94+uP5dtazCbbQK9YejaWbbJ6uZ9
Jlqgh4TuSMvR3YDeTnRSctxQJc7Qp/cTPStONVe6uPJJrvSj+wEddNBBBx10
0EEH3RBdO8vWu2DQbHGK3gHdB3p2l21ifFiTM/SEK90LenaXba+78gDojUTP
7rLdHRs1W0APDz27y/bxHeOT2UAPDD17lq1rNMcGenDobgI66BapugfV/Dij
codOggnoLQzoLYwP9Bubaya7m7ktrWTRikqLxgP61oXbt7dMDhxwWVrJohWV
Fk756PuzPc4uGTaAHJVWsmhFpcUDevClxePl6X0cb5n3/dyUVrJoRaWF4+MH
udtJ8gnvpZUsWlFp0ZSPHj/+2DQf8ltayaIVlRaPhyv93dvTyL5ZyUsrWbSi
0sLxgH53+ldY+NXISytZtKLSwvHw0/va9K+w79JKFq2otHg8oL+vgtJKFq2o
tHg8oF88daore4FRXlrJohWVFo+H7+nXr1yX/v+nvLSSRSsqLRwPV/ovTe67
YfiOPWellSxaUWnx+EC//sCu9E9CWlrJohWVFo+Hp/cb5+/8nvfSShatqLRw
PKDfeuyxoemUlbPSShatqLRwPDy9n0u/HNnP4OLSShatqLR4fLwit9mTfrOS
l1ayaEWlhePjSr95YVv40424tJJFKyotnvLRJ+uT9f+TfTny0koWrai0eHy8
ieKhU2eFP6HISytZtKLSwvHUT5d9OfLSShatqLR4/PTTN0e+SytZtKLSwvEz
7HBD/v8i8tJKFq2otFiYcGlhQG9hPPx/+qVxEr9P+jOt9JXJycNJ/NvCd1pt
rvQelq1qkcnDd9/fnHfD7vdWR8nkvKx4+w96slc107PjZYvuX/rQjfUb4t72
vuwr3T833l8bmuzQ7yA+JlymArHo3UDx4xff/7srkj///fVksi5bdLI+fciy
0v2VU6dOdURy0wccX/qU8NIoGi9jTZOVU7LRvHe3PyR8X/DWmdOPGp1cdG8e
PbNy5VFPo4TKqhvCZ4nC4Qe5Fgb0Fgb0Fgb0Fgb0Fgb0Fgb0Fgb0Fgb0Fgb0
Fub/AbPuOgND78VcAAAAAElFTkSuQmCC
"" alt="A heatmap showing modules of co-regulated genes across the clusters. Modules listed vertically while clusters horizontally. Some modules are highly specific to certain partitions of cells, while others are shared across multiple partitions." width="500" height="700" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 18</strong>:</span> Heatmap showing modules of co-regulated genes across the clusters.</figcaption></figure>
<p>You can also visualise the modules using <code style="color: inherit">plot_cells()</code> function. We’ve chosen some modules to see how the differences on a heatmap correlate with the expression shown on our starting plot.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-4"><i class="far fa-question-circle" aria-hidden="true"></i> Question</div>
<p>Which modules to plot?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-5"></div>
<p>This is totally up to you! It might be the case that you got different numbering of modules, so then using the numbers specified in the code below won’t make much sense. Just look at your heatmap, compare the differences between modules and think which ones would be the most interesing to visualise.</p>
</details>
</blockquote>


In [ ]:
# see the chosen modules across the whole sample
plot_cells(cds_order,
           genes=gene_module_df %>% dplyr::filter(module %in% c(40, 39, 36, 17)),
           group_cells_by="cluster",
           color_cells_by="cluster",
           show_trajectory_graph=FALSE)

<figure id="figure-19"><img src="...
"" alt="Plots showing expression of the genes belonging to specified modules across whole sample." width="650" height="550" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 19</strong>:</span> Plots of expression of the genes belonging to specified modules across whole sample.</figcaption></figure>
<p>With the visualisation methods above, you can now come back to the generated data frame <code style="color: inherit">gene_module_df</code>, filter genes that belong to the module of interest and check their functions to get some more evidence for the correct biological interpretation.</p>
<h1 id="3d-plotting">3D plotting</h1>
<p>Let’s have some fun at the end! That was quite a long and insightful analysis – you definitely deserve to look at some nice, rotatable, cool plot now!
Essentially the workflow is the same as we followed in two dimensions. The crucial part is to specify the dimensionality of the reduced space with the <code style="color: inherit">max_components</code> parameter in <code style="color: inherit">reduce_dimension()</code> function. The default is 2, but if we want to see our data in 3D, we will change that value to 3. From there, you can just repeat the next steps in 3D… or just reward yourself for completing this tutorial by toggling this beautiful 3D plot!</p>


In [ ]:
# reduce dimension to 3D
cds_3d <- reduce_dimension(cds_order, preprocess_method = 'Aligned', max_components = 3)

In [ ]:
# see the resulting 3D plot
plot_cells_3d(cds_3d, color_cells_by="cell_type")

<h1 id="export-your-data-figures-and-notebook">Export your data, figures, and notebook</h1>
<p>Don’t forget to save and export your data! First, we will get Jupyter to see those as files.</p>
<h2 id="export-plots">Export plots</h2>
<p>If you want to export your plot, you have to make sure that you assigned it to an object. For example, if you want to save the plot of cells in pseudotime, simply assign the function you used to generate this plot to an object. Here we call this object <code style="color: inherit">plot_pseudotime</code>, like so:</p>


In [ ]:
plot_pseudotime <- plot_cells(cds_order,
           color_cells_by = "pseudotime",
           label_cell_groups=FALSE,
           label_leaves=FALSE,
           label_branch_points=FALSE)

<p>Then, if you want to save the plot as PDF:</p>


In [ ]:
pdf("plot_pseudotime.pdf")             # open the graphical device and specify the directory and the name of the output pdf file
plot_pseudotime                        # specify the object that your plot is assigned to
dev.off()                              # close the graphical device

<p>The procedure is very similar if you want to export the file as PNG (or analogically JPEG – just replace png with jpeg):</p>


In [ ]:
png("plot_pseudotime.png",            # open the graphical device and specify the directory and the name of the output png file
width=600, height=400)                # optionally you can specify the width and height of the final plot
plot_pseudotime                       # specify the object that your plot is assigned to
dev.off()                             # close the graphical device

<p>However, it often happens that the quality of the exported PNG and JPEG files is not perfect. For best results, we recommend exporting to SVG:</p>


In [ ]:
svg("plot_pseudotime.svg")             # open the graphical device and specify the directory and the name of the output svg file
plot_pseudotime                        # specify the object that your plot is assigned to
dev.off()                              # close the graphical device

<p>You can do the same with any plot that you want to save! You will find the saved figures in the left panel of your JupyterLab. You can right-click on them and download directly onto your computer. You can also push them into your Galaxy history. To do so, you have to change Kernel to Python3 (either click on <code style="color: inherit">Kernel</code> -&gt; <code style="color: inherit">Change Kernel...</code> in the upper left corner of your JupyterLab or click on the displayed current kernel in the upper right corner and change it).</p>
<figure id="figure-20"><img src="...
"" alt="Figure showing the JupyterLab interface with an arrow pointing to the left corner, showing the option &lt;code&gt;Kernel&lt;/code&gt; -&gt; &lt;code&gt;Change Kernel...&lt;/code&gt; and another arrow pointing to the right corner, showing the icon of the current kernel. The pop-up window asks which kernel should be chosen instead." width="1760" height="580" loading="lazy" /><figcaption><span class="figcaption-prefix"><strong>Figure 20</strong>:</span> Two ways of switching kernel.</figcaption></figure>
<p>Check in the upper right corner that selected kernel is Python3, and run the following:</p>
<div class="language-python highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="nf">put</span><span class="p">(</span><span class="s">"marker_file.txt"</span><span class="p">)</span>
<span class="nf">put</span><span class="p">(</span><span class="s">"plot_pseudotime.pdf"</span><span class="p">)</span>
<span class="nf">put</span><span class="p">(</span><span class="s">"plot_pseudotime.png"</span><span class="p">)</span>
</code></pre></div></div>
<p>In this way you can push all the files you’ve saved into your Galaxy history. You can also do the same with this notebook. The cell below will only work if you haven’t changed the name of the notebook. If you renamed it, simply type its new name in the parenthesis.</p>
<div class="language-python highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="nf">put</span><span class="p">(</span><span class="s">"single-cell-scrna-case_monocle3-rstudio.ipynb"</span><span class="p">)</span>
</code></pre></div></div>
<p>Now you can go check your Galaxy history to make sure your files have all made it back into your Galaxy history.</p>
<h1 id="after-jupyter">After Jupyter</h1>
<p>Congratulations! You’ve made it through Jupyter!</p>
<blockquote class="hands_on" style="border: 2px solid #dfe5f9; margin: 1em 0.2em">
<div class="box-title hands-on-title" id="hands-on-closing-jupyterlab"><i class="fas fa-pencil-alt" aria-hidden="true"></i> Hands-on: Closing JupyterLab</div>
<ol>
<li>Click <strong>User</strong>: <strong>Active Interactive Tools</strong></li>
<li>Tick the box of your Jupyter Interactive Tool, and click <strong>Stop</strong></li>
</ol>
</blockquote>
<p>If you want to run this notebook again, or share it with others, it now exists in your history. You can use this ‘finished’ version just the same way as you downloaded the directions file and uploaded it into the Jupyter environment.</p>
<h1 id="conclusion">Conclusion</h1>
<p>If you’re following the Case Study tutorials from the beginning, you have already experienced what it’s like to analyse and question a dataset, potentially without clear cut-offs or clear answers. The <a href="/training-material/topics/single-cell/tutorials/scrna-case_monocle3-trajectories/tutorial.html">Monocle in Galaxy tutorial</a> was focused more on explaining each step of the trajectory analysis and interpreting the results in the biological context. The current tutorial aims at showing the variety of methods that can be used when Galaxy’s Monocle tools are not enough. It shows the potential of batch correction, differential expression analysis and flexibility when using different functions. It’s also a guide for those who would like to understand what is happening ‘behind the scenes’ when clicking on Galaxy buttons.</p>


# Key Points

- Monocle3 in R gives more flexibility when it comes to differential expression analysis and plotting, but Galaxy offers great reproducibility and ease of analysis.
- Comparing the output of several different methods applied on the same dataset might be useful to confirm the results, to ensure that the findings are reliable and even sometimes to find a new piece of information.

# Congratulations on successfully completing this tutorial!

Please [fill out the feedback on the GTN website](https://training.galaxyproject.org/training-material/topics/single-cell/tutorials/scrna-case_monocle3-rstudio/tutorial.html#feedback) and check there for further resources!
